In [2]:
#Imports

import numpy as np
import os
import shutil
import openpyxl as oxl
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from copy import deepcopy

from datetime import datetime

from collections import deque

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
from PIL import Image
import cv2

import json

import collections

# This is needed if the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

#imports from object-detection module
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# This is needed to display the images.
%matplotlib inline

# Developing WM algorithm using keyPoint detection and YOLO LSTM

### Method Definitions

In [3]:
def draw_patch_keypoints(image, patch_keypoints, validKeypoints = []):
    colors = [[255, 255, 51], [0, 255, 0], [0, 0, 255], [255, 0, 0], [153, 255, 255]]
    circle_radius=5

    for i, joint_keypoint in enumerate(patch_keypoints):
        x, y = joint_keypoint
        
        if x == 0 or y == 0:
            continue
            
        if len(validKeypoints) > 1 and joint_keypoint not in validKeypoints:
            continue

        cv2.circle(image, (x, y), circle_radius, colors[i], -1)

    return image

In [4]:
def draw_fittedCurves_for1setOfKeypoints(image, setof_keypoints, esimatorFunction, colorNb=0):
    #colors = [[204, 0, 204], [255, 255, 51], [255, 51, 255], [153, 51, 255], [255, 102, 255], [178, 102, 255]]
    colors = [[255, 255, 51], [0, 255, 0], [0, 0, 255], [255, 0, 0], [153, 255, 255]]
    
    #pts = [[p[0], int(esimatorFunction(p[0]))] for p in setof_keypoints if(p[0] > 0 and p[1]>0)]
    pts = [[indx, int(esimatorFunction(indx))] for indx in range(10, 600, 50)]
    pts_np = np.array(pts, np.int32)
    pts_np = pts_np.reshape((-1,1,2))
    cv2.polylines(image,[pts_np],False,colors[colorNb])

    return image

In [5]:
def draw_box(image, box):
    colors = [(0, 255, 0), (0, 255, 0), (255, 0, 0), (0, 0, 255), (255, 255, 0),(0, 255, 255)]
    labels=["Tooth", "Toothline", "BucketBB", "MatInside", "WearArea"]
    image_h, image_w, _ = image.shape
    
    if len(box) > 3:
        xmin = int(box[0] * image_w)
        xmax = int(box[1] * image_w)
        ymin = int(box[2] * image_h)
        ymax = int(box[3] * image_h)
        label, score = box[4:6]
        #print(str(xmin) + '  ' + str(xmax) + '  '  + str(ymin) + '  ' + str(ymax) + '  ' + label)

        if xmin < 0: xmin = 0
        if ymin < 0: ymin = 0

        color = colors[labels.index(label)]
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 2)

        font_increase = 1.
        cv2.putText(
            image,
            str(score),
            (xmin, ymin - 6),
            cv2.FONT_HERSHEY_SIMPLEX,
            8e-4 * image_h * font_increase,
            color, 2)

    return image

In [6]:
def draw_all_boxes(image, boxesDict):
    for key in boxesDict.keys():
        if 'keypoints' not in key and key not in ['nbOfDetectedTeeth','fileName', 'fittedCurves', '2ndDerivFittedCurves']:
            image = draw_box(image, boxesDict[key])
            
    return image

In [7]:
def draw_all_keypoints_boxes_andCurves(image, resultsDict, numberOfTeeth, drawOnlyValidated=False, numberOflandmarksIncludingToothTip = 5):

    image = inImage
    esimatorFunctions = resultsDict['fittedCurves']
    
    for toothNb in range(numberOfTeeth):
        toothKeyPointKey = 'keypointsForTooth_' + str(toothNb + 1)
        
        if toothKeyPointKey in resultsDict:
            
            if drawOnlyValidated:
                image = draw_patch_keypoints(image, resultsDict[toothKeyPointKey], resultsDict['validatedKeypoints'])
            else:
                image = draw_patch_keypoints(image, resultsDict[toothKeyPointKey], [])

        
    for keypointNb in range(numberOflandmarksIncludingToothTip):
        toothKeyPointKey = 'keypoints_' + str(keypointNb + 1)
        
        if toothKeyPointKey in esimatorFunctions:
            image = draw_fittedCurves_for1setOfKeypoints(image, resultsDict[toothKeyPointKey], esimatorFunctions[toothKeyPointKey], colorNb=keypointNb)

        
    if not drawOnlyValidated:
        draw_all_boxes(image, resultsDict)

    return image

In [8]:
def dimg(framesDir, fileName, teethBoxes, keypoints):
    inImage = cv2.imread(framesDir + fileName.replace('.json','.png'))
    outImage = draw_all_keypoints(inImage, data[0], circle_radius=3)
    outImage = draw_boxes(outImage, data[1]) 
    plt.imshow(outImage)
    plt.show()

In [9]:
def getFrameTime(resultsDict, resultsKey, fileName):
    for timeKey in resultsDict[resultsKey].keys():
        if fileName in resultsDict[resultsKey][timeKey]['fileName']:
            print(timeKey)
            print(resultsDict[resultsKey][timeKey]['fileName'])
            print('\n')
            return timeKey
    
    print('could not find the requested file')
    return None

In [10]:
def loadResults(resultsDir, rawFramesDir=None):
    #Load JSON prediciton results from disc into dict
    resultsDirectories = []


    
    #for resultsDir in resultsDirectories:
    resultsDir = wmsDir
    resultKey = resultsDir.split('/')[-3]

    
    


    resultsDic = {}

    datetimemask = "%Y.%m.%d %H.%M.%S"
    
    resultsDic[resultKey] = {}

    zeroTimeRef = None

    for fileName in sorted(os.listdir(resultsDir)):
        if fileName and '.json' in fileName:

            fileNameAr = fileName[:len(fileName)-5].split('_')
            time = fileNameAr[1] + ' ' + fileNameAr[2]
            dateTime = datetime.strptime(time, datetimemask)

            curHourSince = 0

            if zeroTimeRef == None:
                zeroTimeRef = dateTime
            else:
                timeDif = dateTime - zeroTimeRef
                totalSeconds = timeDif.seconds
                totalDays = timeDif.days
                curHourSince = totalSeconds/3600 + totalDays*24



            with open(resultsDir + fileName, 'r') as fjson:
                data = tuple(json.load(fjson))
                #print(fileName)

                resultsDic[resultKey][curHourSince] = {}      
                resultsDic[resultKey][curHourSince]['time'] = time
                resultsDic[resultKey][curHourSince]['keypoints'] = data[0]
                resultsDic[resultKey][curHourSince]['teeth'] = data[1]
                resultsDic[resultKey][curHourSince]['buckets'] = data[2]
                resultsDic[resultKey][curHourSince]['nbOfDetectedTeeth'] = len(data[1])
                resultsDic[resultKey][curHourSince]['fileName'] = resultsDir + fileName.replace('.json', '.png')
                


            #print(resultsDic)
            #dimg(rawFramesDir, fileName, data)
            #break



    print("loaded the results for  " + str(len(resultsDic[resultKey])) + "   frames. For key: " + resultKey)
    return resultsDic

In [11]:
def parseResults(resultsDic, numberOfTeeth, numberOflandmarksIncludingToothTip = 5):
    paresedResultsDict = {}
    
    for resKey in resultsDic.keys():
        paresedResultsDict[resKey] = {}
        
        for time in resultsDic[resKey].keys():
            paresedResultsDict[resKey][time] = {}
            
            paresedResultsDict[resKey][time]['nbOfDetectedTeeth'] = resultsDic[resKey][time]['nbOfDetectedTeeth'] 
            paresedResultsDict[resKey][time]['fileName'] = resultsDic[resKey][time]['fileName']
            
            #parse the bucket objects
            for obj in resultsDic[resKey][time]['buckets']:
                paresedResultsDict[resKey][time][obj[4]] = obj
                
            #parse the teeth 
            toothNb = 1
            for obj in sorted(resultsDic[resKey][time]['teeth'], key=lambda rv: rv[0]):
                paresedResultsDict[resKey][time]['Tooth_' + str(toothNb)] = obj
                toothNb += 1
                
            toothNb = 1
            for keypointsSet in sorted(resultsDic[resKey][time]['keypoints'], key=lambda kv: kv[0]):           
                paresedResultsDict[resKey][time]['keypointsForTooth_' + str(toothNb)] = keypointsSet
                toothNb += 1
                
                
            for keypointsNb in range(numberOflandmarksIncludingToothTip):       
                paresedResultsDict[resKey][time]['keypoints_' + str(keypointsNb + 1)] = [keypointsSet[keypointsNb] for keypointsSet in sorted(resultsDic[resKey][time]['keypoints'], key=lambda kv: kv[0])]
            
                
    return paresedResultsDict

In [12]:
def filterParsedResults(parsedResultsDic, rejectedResultsDir, numberOfTeeth):
    filteredResults = deepcopy(parsedResultsDic)
    
    for resKey in parsedResultsDic.keys():
        deletedCount = 0
        for time in parsedResultsDic[resKey].keys():
            
            filePath = paresedResultsDict[resKey][time]['fileName']
            fileName = filePath.split('/')[-1]
            
            #reject1 not enough teeth boxes detected.
            if paresedResultsDict[resKey][time]['nbOfDetectedTeeth']  < numberOfTeeth:
                shutil.copy(filePath, rejectedResultsDir)
                print('\nrejected:\n' + str(filePath) + '\nbecause not enough teeth (tips) were detected.')
                del filteredResults[resKey][time]
                deletedCount += 1


            '''
            #reject2 no wearArea detected
            elif 'WearArea' not in paresedResultsDict[resKey][time].keys():
                shutil.copy(filePath, rejectedResultsDir)
                print('\nrejected:\n' + str(filePath) + '\nbecause wearArea was not detected.')
                del filteredResults[resKey][time]
                deletedCount += 1
            '''

                
        print('\nfor results set: ' + resKey + '  rejected ' + str(deletedCount) + ' logs from the parsedResultsDic which were not copied into filteredResults.')
        
        return filteredResults

In [13]:
def fitCurve2keypoints(keypoints, numberOfTeeth):
    numberOflandmarksIncludingToothTip = 5
    degreeOfPolyn = 2
    minNumberOflandmarksNeededToFitCurve = 3
    
    estimatedFunctions = {}
    
    for landmarkNb in range(numberOflandmarksIncludingToothTip):
        landmarkKey = 'keypoints_' + str(landmarkNb + 1)
        
        x = np.ndarray(shape=(1,))
        y = np.ndarray(shape=(1,))
        
        for point in keypoints[landmarkKey]:
            if(point[0] > 0 and point[1] > 0):
                x = np.vstack([x, point[0]])
                y = np.vstack([y, point[1]])
            
        x = x[1:,]
        y = y[1:,]
        x = x.reshape(-1)
        y = y.reshape(-1)
        
        if len(x) >= minNumberOflandmarksNeededToFitCurve and len(y) >= minNumberOflandmarksNeededToFitCurve:
            z = np.polyfit(x, y, degreeOfPolyn)

            estimatedFunctions[landmarkKey] = np.poly1d(z)
        
    
    return estimatedFunctions

In [14]:
def get2ndDerivativeOfCurves(fittC):
    secondDervs = {}
    for key in fittC.keys():
        secondDervs[key] = fittC[key].deriv().deriv().c[0]
        
    return secondDervs

In [15]:
def removeBadCurvesAndKeyPoints(filteredResultsDict, path2saveCurves, rejectedPredsDir, curvDerivTreshDic):
    
    cleanedUpResultsDict = deepcopy(filteredResultsDict)
    
    for resKey in filteredResultsDict.keys():
        deletedCount = 0
        for time in filteredResultsDict[resKey].keys():
            
            fileName = filteredResultsDict[resKey][time]['fileName'].split('/')[-1]
            filePath = path2saveCurves + fileName
            
            
            for keypointsKey in curvDerivTreshDic.keys():
                #reject1 no curve calculated for this keypoint type (because there was less than 2 of them)
                if keypointsKey not in filteredResultsDict[resKey][time]['fittedCurves']:
                    del(cleanedUpResultsDict[resKey][time][keypointsKey])

                    #shutil.copy(filePath, rejectedPredsDir)
                    print('\nrejected:\n' + str(filePath) + '\nbecause no curve was fitted for this keypoint type.')
                    deletedCount += 1
                    
                    
            #reject2 2nd derivative of fitted curve for this keypoint type doesnt satisfy thresholds        
            for keypointsKey in filteredResultsDict[resKey][time]['2ndDerivFittedCurves'].keys():
                derivative = filteredResultsDict[resKey][time]['2ndDerivFittedCurves'][keypointsKey]
                
                if derivative < curvDerivTreshDic[keypointsKey][0]\
                or derivative > curvDerivTreshDic[keypointsKey][1]:
                    
                    if keypointsKey in cleanedUpResultsDict[resKey][time]:
                        del(cleanedUpResultsDict[resKey][time][keypointsKey])
                        del(cleanedUpResultsDict[resKey][time]['2ndDerivFittedCurves'][keypointsKey])
                        del(cleanedUpResultsDict[resKey][time]['fittedCurves'][keypointsKey])

                        shutil.copy(filePath, rejectedPredsDir)
                        print('\nrejected:\n' + str(filePath) + '\nbecause derivative of fitted curve didnt fit the criteria.')
                        deletedCount += 1
                        
                        
                        
            #after the clean up, count all the valid (i.e remaining) keypoints
            cleanedUpResultsDict[resKey][time]['validatedKeypoints'] = []
            
            for keypointsKey in cleanedUpResultsDict[resKey][time]['2ndDerivFittedCurves'].keys():
                for keypoint in cleanedUpResultsDict[resKey][time][keypointsKey]:
                    cleanedUpResultsDict[resKey][time]['validatedKeypoints'].append(keypoint)
            


                
        print('\nfor results set: ' + resKey + '  rejected ' + str(deletedCount) + ' keypoints from the filteredResults which were not copied into cleanedUpResultsDict.')
        
        return cleanedUpResultsDict

In [16]:
def filterCleanedUpResults(cleanedUpResultsDict, rejectedResultsDir, numberOfTeeth):
    finalResultsDict = deepcopy(cleanedUpResultsDict)
    
    for resKey in cleanedUpResultsDict.keys():
        deletedCount = 0
        for time in cleanedUpResultsDict[resKey].keys():
            
            filePath = cleanedUpResultsDict[resKey][time]['fileName']
            fileName = filePath.split('/')[-1]
            
            #reject1 not enough teeth boxes + tips detected.
            if 'keypoints_1' not in cleanedUpResultsDict[resKey][time] or\
            len([x for x in cleanedUpResultsDict[resKey][time]['keypoints_1'] if (x[0] == 0 and x[1] == 0)]) > 0:
                
                shutil.copy(filePath, rejectedResultsDir)
                print('\nrejected:\n' + str(filePath) + '\nbecause not enough toothTips were detected.')
                del finalResultsDict[resKey][time]
                deletedCount += 1

                
        print('\nfor results set: ' + resKey + '  rejected ' + str(deletedCount) + ' logs from the cleanedUpResultsDict which were not copied into finalResultsDict.')
        
        return finalResultsDict

### Loading the json prediction results into resultsDic, parsing, and filtering

In [17]:
#Load JSON prediciton results from disc into dict
wmsDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/'

rawFramesDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/frame3Chan/'

rejectedPredsDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/rejected_notAllTeeth'


resultsDict = loadResults(wmsDir)

paresedResultsDict = parseResults(resultsDict, numberOfTeeth= 6)

filteredResultsDict = filterParsedResults(paresedResultsDict, rejectedPredsDir, numberOfTeeth= 6)

loaded the results for  366   frames. For key: wmdlLogs_Pinto

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/WMDL_2017.10.19_20.43.54.png
because not enough teeth (tips) were detected.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/WMDL_2017.10.20_04.31.19.png
because not enough teeth (tips) were detected.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/WMDL_2017.10.19_04.37.56.png
because not enough teeth (tips) were detected.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/WMDL_2017.10.20_02.29.12.png
because not enough teeth (tips) w

In [18]:
# For all frames: Fit curves to all landmarks, calculate curve derives, save the vis img, save res in filteredDict.
path2saveCurves = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/curves/'

numberOfTeeth = 6

for resKey in filteredResultsDict.keys():
    for time in filteredResultsDict[resKey].keys():
        fileName = filteredResultsDict[resKey][time]['fileName'].split('/')[-1]
        
        fittC = fitCurve2keypoints(filteredResultsDict[resKey][time], numberOfTeeth)
        
        filteredResultsDict[resKey][time]['fittedCurves'] = fittC
        filteredResultsDict[resKey][time]['2ndDerivFittedCurves'] = get2ndDerivativeOfCurves(fittC)
        
        inImage = cv2.imread(rawFramesDir + fileName)

        outImage =  draw_all_keypoints_boxes_andCurves(inImage, filteredResultsDict[resKey][time], numberOfTeeth)

        cv2.imwrite(path2saveCurves + fileName, outImage)

In [19]:
# Remove bad curves and keypoints  from  filteredResultsDict
path2saveRejectedCurves = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/rejectedCurves/'

curvDerivTreshDic = {
    'keypoints_1' : [0.0001, 0.002],
    'keypoints_2' : [0.0001, 0.002],
    'keypoints_3' : [0.0001, 0.002],
    'keypoints_4' : [0.0001, 0.002],
    'keypoints_5' : [-0.0001, 0.002],
}

cleanedUpResultsDict = removeBadCurvesAndKeyPoints(filteredResultsDict, path2saveCurves, path2saveRejectedCurves, curvDerivTreshDic)


rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/curves/WMDL_2017.10.18_17.31.10.png
because no curve was fitted for this keypoint type.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/curves/WMDL_2017.10.18_17.31.10.png
because no curve was fitted for this keypoint type.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/curves/WMDL_2017.10.18_18.53.08.png
because no curve was fitted for this keypoint type.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/curves/WMDL_2017.10.18_18.53.08.png
because no curve was fitted for this keypoint type.

rejected:
/media/hooman/961293e3-04a5-40c5-

In [20]:
# Remove logs where we don't have all toothTips
path2saveRejectedLogs = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/rejected_notAllToothTips/'

finalResultsDict = filterCleanedUpResults(cleanedUpResultsDict, path2saveRejectedLogs, numberOfTeeth= 6)


rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/WMDL_2017.10.19_00.36.33.png
because not enough toothTips were detected.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/WMDL_2017.10.19_01.31.32.png
because not enough toothTips were detected.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/WMDL_2017.10.19_02.32.57.png
because not enough toothTips were detected.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/WMDL_2017.10.19_03.32.58.png
because not enough toothTips were detected.

rejected:
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a706

In [21]:
# visualize the final cleaned up dict
path2vis = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/finalVis/'

for resKey in finalResultsDict.keys():
    for time in finalResultsDict[resKey].keys():
        fileName = finalResultsDict[resKey][time]['fileName'].split('/')[-1]
 
        inImage = cv2.imread(rawFramesDir + fileName)

        outImage =  draw_all_keypoints_boxes_andCurves(inImage, finalResultsDict[resKey][time], 6, drawOnlyValidated=True)

        cv2.imwrite(path2vis + fileName, outImage)

In [25]:
# (Debug) single image look at dic results
resultsKey = 'wmdlLogs_Pinto'
imgFileName = 'WMDL_2017.10.18_22.30.46.png'
frameTime = getFrameTime(filteredResultsDict, resultsKey, imgFileName)
cleanedUpResultsDict['wmdlLogs_Pinto'][frameTime]

4.993333333333333
/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/algorithmDev/wmAlgo_usingWearLandmarsk_optical_hydraulics/try1/wmdlLogs_Pinto/yolo_preds/WMDL_2017.10.18_22.30.46.png




{'2ndDerivFittedCurves': {'keypoints_1': 0.0016452016719914354,
  'keypoints_2': 0.0009081391797253049,
  'keypoints_3': 0.0016576678980770915,
  'keypoints_4': 0.001610305958132028,
  'keypoints_5': 1.1743131747709634e-05},
 'BucketBB': [-0.024749725787367116,
  0.9794890700891358,
  0.4046937651511791,
  0.9734920763846997,
  'BucketBB',
  '99'],
 'Tooth_1': [0.04790989180951992,
  0.10331736018806377,
  0.5009610478906885,
  0.6384907009630456,
  'Tooth',
  '59'],
 'Tooth_2': [0.1762337935233349,
  0.24772483154776037,
  0.4546936496334699,
  0.6126468968229917,
  'Tooth',
  '94'],
 'Tooth_3': [0.3358003254003191,
  0.4062401412791873,
  0.41599479719268884,
  0.565464877091523,
  'Tooth',
  '98'],
 'Tooth_4': [0.5002368499580939,
  0.5689993017260154,
  0.4301860424170575,
  0.5803755955586515,
  'Tooth',
  '94'],
 'Tooth_5': [0.659836438682653,
  0.7278405086666122,
  0.4464199842429066,
  0.6072742908215427,
  'Tooth',
  '96'],
 'Tooth_6': [0.8113445673166378,
  0.871200916248427

In [24]:
cleanedUpResultsDict['wmdlLogs_Pinto'][frameTime]['keypoints_1']

KeyError: 'keypoints_1'

In [26]:
resultsKey = 'wmdlLogs_Pinto'
time = 7.315

filePath = paresedResultsDict[resKey][time]['fileName']
fileName = filePath.split('/')[-1]

#reject1 not enough teeth boxes + tips detected.
if 'keypoints_1' not in paresedResultsDict[resKey][time] or len(paresedResultsDict[resKey][time]['keypoints_1'])  < numberOfTeeth:

    shutil.copy(filePath, rejectedResultsDir)
    print('\nrejected:\n' + str(filePath) + '\nbecause not enough toothTips were detected.')
    del finalResultsDict[resKey][time]
    deletedCount += 1

else:
    print(fileName)


WMDL_2017.10.19_00.50.04.png


### Looking at tooth Lengths

# <font color='red'> Sandbox Area (DO NOT DELETE)

In [26]:
di = {'keypoints_1': [[0, 0],
  [135, 228],
  [237, 197],
  [342, 203],
  [444, 219],
  [533, 238]]}

In [27]:
type(di['keypoints_1'])

list

In [28]:
di['keypoints_1'] == [0, 0]

False

In [29]:
k = len([x for x in di['keypoints_1'] if (x[0] == 0 and x[1] == 0)])

In [30]:
k

1

In [ ]:
# (Debug) single frame of above : Fit curves to landmarks and plot them for a single frame
resultsKey = 'wmdlLogs_Pinto'
imgFileName = 'WMDL_2017.10.18_22.53.03.png'
logNb = getFrameTime(filteredResultsDict, resultsKey, imgFileName)


fittC = fitCurve2keypoints(filteredResultsDict[resultsKey][logNb], 6)

inImage = cv2.imread(wmsDir + imgFileName)

outImage =  draw_all_keypoints_boxes_andCurves(inImage, filteredResultsDict[resultsKey][logNb], fittC, 6)

plt.imshow(outImage)
plt.show()

for key in fittC.keys():
    print(key)
    print(fittC[key].deriv().deriv().c[0])

In [ ]:
# calculate the second derivative of a curve. For validation.
secondDerivative = fittC['keypoints_4'].deriv().deriv().c[0]

# Developing WM algorithm using just SSD (tooth boxes)

## Capturing frames from video and Saving Results to Disk

In [ ]:
#open a video and save certain multiples of frames as images

#pathToVideo = '/home/hooman/videos/1059_Kearl_Thermal_Camera_CAT_7495HF/CombinedVideos(DL)/1059_ThermalBucyrus_Kear_4D17H15M.avi'
#locationToSaveAt = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/results_try6-51546_1059_ThermalBucyrus_Kear_4D17H15M/'


pathToVideo = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/1_20160729-145000_0001n0.avi'

locationToSaveAt = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/frames/'

scoreToUse = 0.5
framesToCaptureMultiple = 10





vidReader = cv2.VideoCapture(pathToVideo)

while(vidReader.isOpened()):
    
    ret, frame = vidReader.read()
    
    curPosTime = int(vidReader.get(cv2.CAP_PROP_POS_MSEC))
    curPosFrame = int(vidReader.get(cv2.CAP_PROP_POS_FRAMES))
    
    
    if ret and curPosFrame % framesToCaptureMultiple == 0:
        
        nameToSaveWith = str(curPosFrame) + '_' + str(curPosTime) + '_'

        #save the predictions on frame
        #predictAndSaveSingleImage(frame, nameToSaveWith, locationToSaveAt, scoreToUse)
        
        #save just the frames
        cv2.imwrite( locationToSaveAt + nameToSaveWith + '.png', frame)

    
vidReader.release()

In [ ]:
# copying over some of the captured frames
srcDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/every100Frame_1059_ThermalBucyrus_Kear_4D17H15M/'

destDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/selectedFrames_every18000Frames/'

framesToCopyMultiple = 18000


for fileName in os.listdir(srcDir):
    if fileName and '.png' in fileName:
        full_file_name = os.path.join(srcDir, fileName)
        if (os.path.isfile(full_file_name)):
            
            fileNameAr = fileName.split('_')
            frameNb = int(fileNameAr[0])
            
            if frameNb % framesToCopyMultiple == 0:
                shutil.copy(full_file_name, destDir)

## Peforming Predictions on images and Saving Results to Disk

### Helpers

In [ ]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
#PATH_TO_CKPT = '/home/hooman/WM_PROJECT/thermal_Bucyrus/try6-ssd-correctedBoxWidth/result_ckpt-51546/output_inference_graph/frozen_inference_graph.pb'

#PATH_TO_CKPT = '/home/hooman/WM_PROJECT/thermal_Bucyrus/try6-ssd-correctedBoxWidth/result_ckpt-3164009/output_inference_graph/frozen_inference_graph.pb'

#best so far
PATH_TO_CKPT = '/home/hooman/WM_PROJECT/thermal_Bucyrus/try6-ssd-correctedBoxWidth/result_ckpt-1128411/output_inference_graph/frozen_inference_graph.pb'




# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/home/hooman/WM_PROJECT/thermal_Bucyrus/try6-ssd-correctedBoxWidth/hs_label_map_multiclass.pbtxt'

NUM_CLASSES = 4


#Path to test images and labels
imagesPath = '/home/hooman/WM_PROJECT/dataPreparation/hsTrainingData/thermal/bucyrusPnH/trainingData-ThermalBucyrus-imagesWithTeethAndWearAndSceneLabels/removedThisIMagesBecauseOfBadLabels/'

#labelsPath = '/home/hooman/dataPreparation/hsTestSet/labels/'

savedResultsPath = '/home/hooman/WM_PROJECT/thermal_Bucyrus/try6-ssd-correctedBoxWidth/result_ckpt-3164009/predictions_score50/'

In [ ]:
#load the frozen detection graph

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
#load the label map

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
def predictAndSaveSingleImage(image, nameToSaveWith, locationToSaveAt, scoreToUse):

    STANDARD_COLORS = ['LightGoldenRodYellow', 'SpringGreen', 'Aqua', 'LightCyan']

    #image_np = load_image_into_numpy_array(image)
    image_np = image
    
    image_np_expanded = np.expand_dims(image_np, axis=0)

    graph = detection_graph

    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
              'num_detections', 'detection_boxes', 'detection_scores',
              'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                  tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)


            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                 feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
              'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]



    boxes   = output_dict['detection_boxes']
    classes = output_dict['detection_classes']
    scores  = output_dict['detection_scores']


    box_to_color_map = collections.defaultdict(str)
    box_to_display_str_map = collections.defaultdict(list)

    
    
    
    
    # hs Get best boxes
    jsonToSave = []
    
    class1Best = tuple()
    class1BestScore = -1
    
    class2Best = []
    class2BestScore = []
    
    class3Best = []
    class3BestScore = []

    for i in range(boxes.shape[0]):
        box = tuple()


        if scores[i] > scoreToUse:
            
            if classes[i] == 1 and scores[i] >= class1BestScore:
                
                class1Best = tuple(boxes[i].tolist())
                class1BestScore = scores[i]
                
                
            if classes[i] == 2:
                
                class2Best.append(tuple(boxes[i].tolist())) 
                class2BestScore.append(scores[i])
                
                
            if classes[i] == 3:
                
                class3Best.append(tuple(boxes[i].tolist()))
                class3BestScore.append(scores[i])
                
                
                
                
                
    #hs get the json data and desplay string for best boxes
    if len(class1Best) > 2:

        box_to_color_map[class1Best] = STANDARD_COLORS[1]

        display_str = ''
        class_name = category_index[1]['name']
        display_str = str(class_name)

        display_str = '{}: {}%'.format(display_str, int(100*class1BestScore))
        box_to_display_str_map[class1Best].append(display_str)


        #save json
        ymin, xmin, ymax, xmax = class1Best

        jsonToSave.append([xmin, xmax, ymin, ymax, str(class_name), str(int(100*class1BestScore))])
        
        
        
    for box, score in zip(class2Best, class2BestScore):
        
        box_to_color_map[box] = STANDARD_COLORS[2]

        display_str = ''
        class_name = category_index[2]['name']
        display_str = str(class_name)

        display_str = '{}: {}%'.format(display_str, int(100*score))
        box_to_display_str_map[box].append(display_str)


        #save json
        ymin, xmin, ymax, xmax = box

        jsonToSave.append([xmin, xmax, ymin, ymax, str(class_name), str(int(100*score))])


        
    for box, score in zip(class3Best, class3BestScore):
        
        box_to_color_map[box] = STANDARD_COLORS[3]

        display_str = ''
        class_name = category_index[3]['name']
        display_str = str(class_name)

        display_str = '{}: {}%'.format(display_str, int(100*score))
        box_to_display_str_map[box].append(display_str)


        #save json
        ymin, xmin, ymax, xmax = box

        jsonToSave.append([xmin, xmax, ymin, ymax, str(class_name), str(int(100*score))])
        
        
        


    #overlay the best boxes
    for box, color in box_to_color_map.items():
        ymin, xmin, ymax, xmax = box

        vis_util.draw_bounding_box_on_image_array(
            image_np,
            ymin,
            xmin,
            ymax,
            xmax,
            color=color,
            thickness=4,
            display_str_list=box_to_display_str_map[box],
            use_normalized_coordinates=True)


        
        
        
    #display final results
    #plt.imshow(image_np)
    #plt.title(nameToSaveWith)
    #plt.show()
    
    
    #save final results
    with open(locationToSaveAt + nameToSaveWith + '.json', 'w') as fjson:
        json.dump(jsonToSave, fjson)
    
    cv2.imwrite( locationToSaveAt + nameToSaveWith + '.png', image_np)

### Runing the actual detection

In [ ]:
### from a directoy of images run predictions, save both the overlay image and json bounding boxes
testImagesDirectory = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/selectedSequence/'

locationToSaveAt = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/pred_selectedSequence_try6-1128411_score50/'

scoreToUse = 0.5


for fileName in os.listdir(testImagesDirectory):

    image = Image.open(testImagesDirectory + fileName)
    image_np = load_image_into_numpy_array(image)
    
    nameToSaveWith = fileName.replace(".png", "")
    
    predictAndSaveSingleImage(image_np, nameToSaveWith, locationToSaveAt, scoreToUse)

In [ ]:
###  (no helpers) Whole process on images: Load, Test, Display, save the overlayed image
scoreToUse = 0.5

for filePath, fileName in zip(TEST_IMAGE_PATHS, TEST_IMAGE_IDS):

    image = Image.open(filePath)


    STANDARD_COLORS = ['LightGoldenRodYellow', 'SpringGreen', 'Aqua', 'LightCyan']

    #image_np = load_image_into_numpy_array(image) # just replacing the function call with its content
    (im_width, im_height) = image.size
    image_np = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
    
    
    image_np_expanded = np.expand_dims(image_np, axis=0)

    graph = detection_graph

    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
              'num_detections', 'detection_boxes', 'detection_scores',
              'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                  tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)


            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                 feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
              'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]



    boxes   = output_dict['detection_boxes']
    classes = output_dict['detection_classes']
    scores  = output_dict['detection_scores']





    box_to_color_map = collections.defaultdict(str)
    box_to_display_str_map = collections.defaultdict(list)



    for i in range(boxes.shape[0]):
        box = tuple()


        if scores is None or scores[i] > scoreToUse:
            box = tuple(boxes[i].tolist())

            #box_to_color_map[box] = 'LightGreen'
            box_to_color_map[box] = STANDARD_COLORS[classes[i]]


            display_str = ''
            if classes[i] in category_index.keys():
                class_name = category_index[classes[i]]['name']
            display_str = str(class_name)

            display_str = '{}: {}%'.format(display_str, int(100*scores[i]))
            box_to_display_str_map[box].append(display_str)




    for box, color in box_to_color_map.items():
        ymin, xmin, ymax, xmax = box

        vis_util.draw_bounding_box_on_image_array(
            image_np,
            ymin,
            xmin,
            ymax,
            xmax,
            color=color,
            thickness=4,
            display_str_list=box_to_display_str_map[box],
            use_normalized_coordinates=True)

    #plt.figure(figsize=IMAGE_SIZE)
    #plt.imshow(image_np)
    
    import cv2
    cv2.imwrite( savedResultsPath + fileName, image_np)


## <font color='blue'> Running the pipeline:

### Helpers

In [ ]:
def getMovingAverage(maDeque, currentLength):
    
    maDeque.append(currentLength)
    
    if len(maDeque) > 2:
        maDeque.popleft()
        
    rs = 0
    for l in maDeque:
        rs += l
        
    return rs / len(maDeque)

In [ ]:
def getMovingAverage2(maDeque, currentLength):
    
    if len(maDeque) > 0 and currentLength > maDeque[-1]:
        if (currentLength - maDeque[-1]) < 25:
            currentLength = maDeque[-1]
    
    
    maDeque.append(currentLength)
    
    if len(maDeque) > 7:
        maDeque.popleft()
        
    rs = 0
    for l in maDeque:
        rs += l
        
    return rs / len(maDeque)

In [ ]:
def getMovingAverage3(maDeque, currentLength, movingAvgLength, posJumpMin, negJumpMax):
    
    # if we have history
    if len(maDeque) > 0:
        # only big positive jumps are allowed
        if currentLength > maDeque[-1]:
            if (currentLength - maDeque[-1]) < posJumpMin:
                currentLength = maDeque[-1]
                
                
        # only small negetive jumps are allowed
        elif (maDeque[-1] - currentLength) > negJumpMax:
            currentLength = maDeque[-1]
    
    
    maDeque.append(currentLength)
    
    
    if len(maDeque) > movingAvgLength:
        maDeque.popleft()
        
    rs = 0
    for l in maDeque:
        rs += l
        
    return rs / len(maDeque)

In [ ]:
def getMovingAverage4(maDeque, currentLength, movingAvgLength, posJumpMin, negJumpMax):
    
    # if we have history
    if len(maDeque) > 0:

        temRs = 0
        for l in maDeque:
            temRs += l

        curMA = temRs / len(maDeque)
        
        
        
        # only big positive jumps are allowed
        if currentLength > maDeque[-1]:
            if (currentLength - maDeque[-1]) < posJumpMin:
                currentLength = curMA
                
                
        # only small negetive jumps are allowed
        elif (maDeque[-1] - currentLength) > negJumpMax:
            currentLength = curMA
    
    
    maDeque.append(currentLength)
    
    
    if len(maDeque) > movingAvgLength:
        maDeque.popleft()
        
    rs = 0
    for l in maDeque:
        rs += l
        
    return rs / len(maDeque)

In [ ]:
def getMovingAverage5(maDeque, currentLength, movingAvgLength, posJumpMin, posJumpMax, negJumpMax):
    
    # if we have history
    if len(maDeque) > 0:

        temRs = 0
        for l in maDeque:
            temRs += l

        curMA = temRs / len(maDeque)
        
        
        
        # only big positive jumps are allowed
        if currentLength > maDeque[-1]:
            if (currentLength - maDeque[-1]) < posJumpMin or (currentLength - maDeque[-1]) > posJumpMax:
                currentLength = curMA
                
                
        # only small negetive jumps are allowed
        elif (maDeque[-1] - currentLength) > negJumpMax:
            currentLength = curMA
    
    
    maDeque.append(currentLength)
    
    
    if len(maDeque) > movingAvgLength:
        maDeque.popleft()
        
    rs = 0
    for l in maDeque:
        rs += l
        
    return rs / len(maDeque)

In [ ]:
def getAv(maDeque):
    rs = 0
    for l in maDeque:
        rs += l
        
    return rs / len(maDeque)

### Loading the json prediction results into resultsDic

In [ ]:
#single load JSON results in resultsDic fro both LipShroud and WM

wmsDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/wmdlLogs_Liebherr/hsPreds/'

lsDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/wmdlLogs_Liebherr/hsPredsLS/'



resultsDirectories = []


resultsDic = {}


datetimemask = "%Y.%m.%d %H.%M.%S"


resultsDir = wmsDir
#for resultsDir in resultsDirectories:
    
resDirAr = resultsDir.split('/')

resultKey = resDirAr[len(resDirAr) - 3]

resultsDic[resultKey] = {}

zeroTimeRef = None


#for fileName in sorted(os.listdir(resultsDir)):
for fileName in sorted(os.listdir('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/wmdlLogs_Liebherr/theOnesWeGetLSFor/')):
    if fileName and '.json' in fileName:

        fileNameAr = fileName[:len(fileName)-5].split('_')
        time = fileNameAr[1] + ' ' + fileNameAr[2]
        dateTime = datetime.strptime(time, datetimemask)

        curHourSince = 0

        if zeroTimeRef == None:
            zeroTimeRef = dateTime
        else:
            timeDif = dateTime - zeroTimeRef
            totalSeconds = timeDif.seconds
            totalDays = timeDif.days
            curHourSince = totalSeconds/3600 + totalDays*24





        #timeConverted = int(''.join(c for c in time if c.isdigit()))
        timeConverted = curHourSince

        with open(resultsDir + fileName, 'r') as fjson:
            data = tuple(json.load(fjson))
            #print(revis_data)


            if len(data) > 0:
                teethLine = {}
                teeth = []
                wms = []

                for el in data:
                    tooth = {}
                    wm = {}

                    if el[4] == 'teethLine':
                        teethLine['box'] = (el[0], el[1], el[2],el[3])
                        teethLine['score'] = el[5]

                    if el[4] == 'wearMark':
                        wm['box'] = (el[0], el[1], el[2],el[3])
                        wm['score'] = el[5]
                        wms.append(wm)

                    if el[4] == 'tooth':
                        tooth['box'] = (el[0], el[1], el[2],el[3])
                        tooth['score'] = el[5]
                        teeth.append(tooth)


                resultsDic[resultKey][timeConverted] = {}      
                resultsDic[resultKey][timeConverted]['time'] = time
                resultsDic[resultKey][timeConverted]['teethLine'] = teethLine
                resultsDic[resultKey][timeConverted]['wearMarks'] = wms
                resultsDic[resultKey][timeConverted]['teeth'] = teeth


print("loaded the results for  " + str(len(resultsDic[resultKey])) + "   frames. For key: " + resultKey)

In [ ]:
#MULTIPLE load JSON results in resultsDic (Keyed by converted time)

#Theraml_Bucycrys datasets:
'''
'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogsFromNov2016/hsPreds/',


'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogsFromDec2016/hsPreds/'

'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogsFromDec2016_18-23/hsPreds/'


'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogsFromMay2017/hsPreds/',

'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogsFromMahdi_Kearl/hsPreds/'

'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogsFromMahdi_Kearl_Aug20-Sep11/hsPreds/'



'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogs_CNRL_Dec2nd/hsPreds/'

'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogs_CNRL_Aug10-Sep10/hsPreds/'
'''

#Optical_Hydraulics datasets:
'''
'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/wmdlLogs_aitik/hsPreds/'

'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/wmdlLogs_Phukham/hsPreds/'

'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/wmdlLogs_Pinto/hsPreds/'

'/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/wmdlLogs_Liebherr/hsPreds/'
'''
resultsDirectories = ['/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/wmdlLogs_Liebherr/hsPreds/']


resultsDic = {}


datetimemask = "%Y.%m.%d %H.%M.%S"

for resultsDir in resultsDirectories:
    
    resDirAr = resultsDir.split('/')
    
    resultKey = resDirAr[len(resDirAr) - 3]
    
    resultsDic[resultKey] = {}
    
    zeroTimeRef = None
    

    for fileName in sorted(os.listdir(resultsDir)):
        if fileName and '.json' in fileName:

            fileNameAr = fileName[:len(fileName)-5].split('_')
            time = fileNameAr[1] + ' ' + fileNameAr[2]
            dateTime = datetime.strptime(time, datetimemask)
            
            curHourSince = 0
            
            if zeroTimeRef == None:
                zeroTimeRef = dateTime
            else:
                timeDif = dateTime - zeroTimeRef
                totalSeconds = timeDif.seconds
                totalDays = timeDif.days
                curHourSince = totalSeconds/3600 + totalDays*24

 


            
            #timeConverted = int(''.join(c for c in time if c.isdigit()))
            timeConverted = curHourSince

            with open(resultsDir + fileName, 'r') as fjson:
                data = tuple(json.load(fjson))
                #print(revis_data)


                if len(data) > 0:
                    teethLine = {}
                    teeth = []
                    wms = []

                    for el in data:
                        tooth = {}
                        wm = {}

                        if el[4] == 'teethLine':
                            teethLine['box'] = (el[0], el[1], el[2],el[3])
                            teethLine['score'] = el[5]

                        if el[4] == 'wearMark':
                            wm['box'] = (el[0], el[1], el[2],el[3])
                            wm['score'] = el[5]
                            wms.append(wm)

                        if el[4] == 'tooth':
                            tooth['box'] = (el[0], el[1], el[2],el[3])
                            tooth['score'] = el[5]
                            teeth.append(tooth)


                    resultsDic[resultKey][timeConverted] = {}      
                    resultsDic[resultKey][timeConverted]['time'] = time
                    resultsDic[resultKey][timeConverted]['teethLine'] = teethLine
                    resultsDic[resultKey][timeConverted]['wearMarks'] = wms
                    resultsDic[resultKey][timeConverted]['teeth'] = teeth


    print("loaded the results for  " + str(len(resultsDic[resultKey])) + "   frames. For key: " + resultKey)

In [ ]:
#load JSON results in resultsDic (Keyed by frame#)

#resultsDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/selectedFrames_10000_predictions_try6-1128411/'

#resultsDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/preds_selectedFrames_every18000Frames_try6-3164009_score50/'

resultsDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/predictions_every100Frame_try6-1128411_score50/'



#resultsDir = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/pred_selectedSequenceAll/'

resultsDic = {}

for fileName in os.listdir(resultsDir):
    if fileName and '.json' in fileName:
        
        fileNameAr = fileName.split('_')
        frameNb = int(fileNameAr[0])
        frameTime = int(fileNameAr[1])
        
        with open(resultsDir + fileName, 'r') as fjson:
            data = tuple(json.load(fjson))
            #print(revis_data)
            
            
            if len(data) > 0:
                teethLine = {}
                teeth = []
                wms = []
                
                for el in data:
                    tooth = {}
                    wm = {}
                    
                    if el[4] == 'teethLine':
                        teethLine['box'] = (el[0], el[1], el[2],el[3])
                        teethLine['score'] = el[5]

                    if el[4] == 'wearMark':
                        wm['box'] = (el[0], el[1], el[2],el[3])
                        wm['score'] = el[5]
                        wms.append(wm)

                    if el[4] == 'tooth':
                        tooth['box'] = (el[0], el[1], el[2],el[3])
                        tooth['score'] = el[5]
                        teeth.append(tooth)
                        
                        
                resultsDic[frameNb] = {}      
                resultsDic[frameNb]['time'] = frameTime
                resultsDic[frameNb]['teethLine'] = teethLine
                resultsDic[frameNb]['wearMarks'] = wms
                resultsDic[frameNb]['teeth'] = teeth
                
                
print("loaded the results for  " + str(len(resultsDic)) + "   frames")

In [ ]:
resultsDic

### Seperating individual teeth, and recording teethLine and WM statistics

In [ ]:
#MULTIPLE V2 Record width and Ycentre of teethLine (only detected) ...

imWidth = 720
imHeight = 480
numberOfTeeth = 6 #9


teethSeperated_resultsDic = {}

for resultKey in resultsDic:

    processedFrames = 0
    teethSeperated_resultsDic[resultKey] = {}

    for key in sorted(resultsDic[resultKey]):

        if 'box' in resultsDic[resultKey][key]['teethLine']:

            #1. Record TeethLine
            xmin, xmax, ymin, ymax = resultsDic[resultKey][key]['teethLine']['box']


            xminConv = xmin * imWidth
            xmaxConv = xmax * imWidth

            yminConv = ymin * imHeight
            ymaxConv = ymax * imHeight

            middleOfTeethLineY = (ymaxConv - yminConv)/2 + yminConv

            teethSeperated_resultsDic[resultKey][key] = {}
            teethSeperated_resultsDic[resultKey][key]['teethLine'] = {}
            teethSeperated_resultsDic[resultKey][key]['teethLine']['box'] = (xminConv, xmaxConv, yminConv, ymaxConv)
            teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle'] = middleOfTeethLineY
            teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] = (xmaxConv - xminConv)




            #2. Calculate locations teeth should be at
            toothLocations = []

            xRange = xmaxConv - xminConv
            xRangeInc = xRange / numberOfTeeth

            for i in range(numberOfTeeth):
                toothLocations.append(( int(xminConv + i*xRangeInc), int(xminConv + (i+1)*xRangeInc) ))




            #3. Seperate the teeth and record them
            foundAtLeast1Tooth = False

            for tooth in resultsDic[resultKey][key]['teeth']:
                xminT, xmaxT, yminT, ymaxT = tooth['box']

                xminConvT = xminT * imWidth
                xmaxConvT = xmaxT * imWidth

                yminConvT = yminT * imHeight
                ymaxConvT = ymaxT * imHeight

                toothWidth = (xmaxConvT - xminConvT)
                middleOfTooth = xminConvT + toothWidth/2


                for i in range(len(toothLocations)):

                    if(middleOfTooth >= toothLocations[i][0] and middleOfTooth <= toothLocations[i][1]):
                        toothKey = 'tooth' + str(i)

                        if toothKey in teethSeperated_resultsDic[resultKey][key]:
                            print("found a duplicate for:  " + toothKey + "\n")
                            #print(teethSeperated_resultsDic[key])

                        else:
                            teethSeperated_resultsDic[resultKey][key][toothKey] = (xminConvT, xmaxConvT, yminConvT, ymaxConvT)
                            foundAtLeast1Tooth = True




            #4. Record the average WM height, max WM heigh and number of WM detections
            wmHeights = []
            sumOfWmHeights = 0
            maxWMHeight = -1
            for wm in resultsDic[resultKey][key]['wearMarks']:
                xminTT, xmaxTT, yminTT, ymaxTT = wm['box']

                xminConvTT = xminTT * imWidth
                xmaxConvTT = xmaxTT * imWidth

                yminConvTT = yminTT * imHeight
                ymaxConvTT = ymaxTT * imHeight

                wmHeight = (ymaxConvTT - yminConvTT)

                wmHeights.append(wmHeight)

                sumOfWmHeights += wmHeight

                maxWMHeight = max(maxWMHeight, wmHeight)


            if(len(wmHeights) > 0):
                averageWmHeight = sumOfWmHeights / len(wmHeights)
                teethSeperated_resultsDic[resultKey][key]['wms_avgHeight'] = averageWmHeight
                teethSeperated_resultsDic[resultKey][key]['wms_maxHeight'] =  maxWMHeight
                teethSeperated_resultsDic[resultKey][key]['wms_nbDetections'] = len(wmHeights)




            if foundAtLeast1Tooth:
                processedFrames += 1
        #else:
            #print("found no teethLine for frame:   " + str(key) + "\n")


    print("Found teeth for  " + str(processedFrames)+"  of  "+str(len(teethSeperated_resultsDic[resultKey])) +"  frames  for result:  " + resultKey + '\n\n\n') 

### AutoCalibrating and plotting results

In [ ]:
# Full (all offsets included) Auto Calbrating, Normalizing and Plotting results

'''
#This works for Kearl
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44}
}


#This kinda works for CNRL
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 80},
}


#I was playing around with this for Kearl
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth6':{'posJumpMinPercentile':99.9   , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 4, 'lengthOffset': 0},
    
    'tooth7':{'posJumpMinPercentile':99.9   , 'negJumpMaxPercentile':50, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 4, 'lengthOffset': 0},
    
    'tooth8':{'posJumpMinPercentile':99.9   , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 5, 'lengthOffset': 0}
}


#Used this for Pinto
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0}
}
'''

#Used this for wmdlLogs_Liebherr
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 2, 'lengthOffset': 0},
    
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':45, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth2':{'posJumpMinPercentile':90.0 , 'negJumpMaxPercentile':45, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth3':{'posJumpMinPercentile':99.0 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 2, 'lengthOffset': 0},
    
    'tooth5':{'posJumpMinPercentile':99.0 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0}
}



'''
#For thermal_bucyrus
ymiddleLimitMin = 180
ymiddleLimitMax = 210 

'''
#For optical_Hydraulics
ymiddleLimitMin = 260
ymiddleLimitMax = 290 


widthToFilterFor = 450
widthTolerance = 500000

imHeight = 480





#############################################################################################################
###### Process the lengths to get a sequence for each tooth #######
###################################################################
finalResults = {}

for resultKey in teethSeperated_resultsDic:
    finalResults[resultKey] = {}
    
    for toothKey in configs.keys():
        finalResults[resultKey][toothKey] = {}
        finalResults[resultKey][toothKey]['origLengths'] = []
        finalResults[resultKey][toothKey]['origLengths'].append(configs[toothKey]['start'])
        
        finalResults[resultKey][toothKey]['normalizedLengths'] = []
        finalResults[resultKey][toothKey]['times'] = []
        finalResults[resultKey][toothKey]['times'].append(-1)
        finalResults[resultKey][toothKey]['maDeque'] = deque()
        
        
    
    for timeKey in sorted(teethSeperated_resultsDic[resultKey]):
        
        if 'teethLine' in teethSeperated_resultsDic[resultKey][timeKey] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][timeKey]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['width'] 
            
            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin and abs(widthToFilterFor - widthTeethLine) < widthTolerance:

                for toothKey in configs.keys():
                    if toothKey in teethSeperated_resultsDic[resultKey][timeKey]:
                        actualLength = teethSeperated_resultsDic[resultKey][timeKey][toothKey][3] - teethSeperated_resultsDic[resultKey][timeKey][toothKey][2]
                        
                        finalResults[resultKey][toothKey]['times'].append(timeKey)
                        finalResults[resultKey][toothKey]['origLengths'].append(actualLength)
#############################################################################################################








#############################################################################################################
########### FINDING CALIBRATION PARAMETERS ####################
###################################################################
calibResults = {}
for resultKey in finalResults:
    print('\n\nshowing learned calib params for  ' + str(resultKey) + ':')
    calibResults[resultKey] = {}
    
    for toothKey in sorted(finalResults[resultKey].keys()):
        print('\nfor ' + str(toothKey) + ': ') 
        
        calibResults[resultKey][toothKey] = {}
        calibResults[resultKey][toothKey]['jumps'] = []
        calibResults[resultKey][toothKey]['posJumps'] = []
        calibResults[resultKey][toothKey]['negJumps'] = []
        
        for i in range(len(finalResults[resultKey][toothKey]['origLengths']) - 1):
            diff = finalResults[resultKey][toothKey]['origLengths'][i+1] - finalResults[resultKey][toothKey]['origLengths'][i]
          
        
            calibResults[resultKey][toothKey]['jumps'].append(diff)

            if(diff > 0):
                calibResults[resultKey][toothKey]['posJumps'].append(diff)
            else:
                calibResults[resultKey][toothKey]['negJumps'].append(abs(diff))

                
                
                
        posJumps_np = np.array(calibResults[resultKey][toothKey]['posJumps'])
        negJumps_np = np.array(calibResults[resultKey][toothKey]['negJumps'])
        
        
        calibResults[resultKey][toothKey][90.0] = np.percentile(posJumps_np, 90.0)
        calibResults[resultKey][toothKey][99.0] = np.percentile(posJumps_np, 99.0)
        calibResults[resultKey][toothKey][99.5] = np.percentile(posJumps_np, 99.5)
        calibResults[resultKey][toothKey][99.9] = np.percentile(posJumps_np, 99.9)
        
        
        calibResults[resultKey][toothKey][65] = np.percentile(negJumps_np, 65)
        calibResults[resultKey][toothKey][60] = np.percentile(negJumps_np, 60)
        calibResults[resultKey][toothKey][55] = np.percentile(negJumps_np, 55)
        calibResults[resultKey][toothKey][50] = np.percentile(negJumps_np, 50)
        calibResults[resultKey][toothKey][45] = np.percentile(negJumps_np, 45)
        calibResults[resultKey][toothKey][40] = np.percentile(negJumps_np, 40)
        calibResults[resultKey][toothKey][35] = np.percentile(negJumps_np, 35)
        calibResults[resultKey][toothKey][30] = np.percentile(negJumps_np, 30)
        calibResults[resultKey][toothKey][25] = np.percentile(negJumps_np, 25)
        calibResults[resultKey][toothKey][20] = np.percentile(negJumps_np, 20)
        calibResults[resultKey][toothKey][15] = np.percentile(negJumps_np, 15)
        calibResults[resultKey][toothKey][10] = np.percentile(negJumps_np, 10)
        
        print('99.5th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.5]))
        print('99.99th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.9]))
        print('50th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][50]))
        print('35th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][35]))
        print('30th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][30]))
#############################################################################################################








#############################################################################################################
################ Normalizing tooth lengths ####################
###################################################################
for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in finalResults[resultKey].keys():
        
        
        posJumpMin = calibResults[resultKey][toothKey][configs[toothKey]['posJumpMinPercentile']] + configs[toothKey]['posJumpMinOffset']
        
        
        negJumpMax = calibResults[resultKey][toothKey][configs[toothKey]['negJumpMaxPercentile']]  
        movingAvgLength = configs[toothKey]['movingAvgLength']
        
        
        
        print('\nfor ' + str(toothKey) + ': ') 
        
        print('posJumpMin = ' + str(posJumpMin) + ' which was ' + str(configs[toothKey]['posJumpMinPercentile']) + 'th percentile of positiveJumps')
        
        print('negJumpMax = ' + str(negJumpMax) + ' which was ' + str(configs[toothKey]['negJumpMaxPercentile']) + 'th percentile of negetiveJumps')
        
        print('movingAvgLength = ' + str('movingAvgLength'))
        
        
        
        for toothLen in finalResults[resultKey][toothKey]['origLengths']:
                    
            normToothLen = getMovingAverage4(finalResults[resultKey][toothKey]['maDeque'], toothLen, movingAvgLength,posJumpMin,negJumpMax) + configs[toothKey]['lengthOffset']
            
            finalResults[resultKey][toothKey]['normalizedLengths'].append(normToothLen)
#############################################################################################################









                        
                        
                        
'''                        
#############################################################################################################
############# Plotting Original Lengths Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):

        print('showing original lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['origLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################     '''                   
                        
                
'''                       
#############################################################################################################
############# Plotting Final Results Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):        
        print('showing normalized lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['normalizedLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################
'''


#############################################################################################################
############# Plotting Final Results Same Graphs ##########################
###############################################################################
for toothKey in sorted(configs.keys()):
    print('\n\n\nshowing normalized lengths for:  ' + str(toothKey))
    
    plt.figure(figsize=(30,10))
    plt.axis([0, 800, 20, 100])
    ax = plt.axes()
    ax.grid()

    for resultKey in finalResults:

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


    ax.legend()
    
    plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/'+toothKey+'.png')
    
    plt.show()
#############################################################################################################

In [ ]:
# (starting point included) Auto Calbrating, Normalizing and Plotting results

'''
configs = {
    'tooth0':{'posJumpMinPercentile':99.5 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2}
}

#This works
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44}
}




configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
}

configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2, },
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
}



    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 80},
    
    
    configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth4':{'posJumpMinPercentile':-1 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':25, 'movingAvgLength':2, 'start': 44},
}


'''
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44}
}




imHeight = 480

ymiddleLimitMax = 210 
ymiddleLimitMin = 180

widthToFilterFor = 450
widthTolerance = 500000






#############################################################################################################
###### Process the lengths to get a sequence for each tooth #######
###################################################################
finalResults = {}

for resultKey in teethSeperated_resultsDic:
    finalResults[resultKey] = {}
    
    for toothKey in configs.keys():
        finalResults[resultKey][toothKey] = {}
        finalResults[resultKey][toothKey]['origLengths'] = []
        finalResults[resultKey][toothKey]['origLengths'].append(configs[toothKey]['start'])
        
        finalResults[resultKey][toothKey]['normalizedLengths'] = []
        finalResults[resultKey][toothKey]['times'] = []
        finalResults[resultKey][toothKey]['times'].append(-1)
        finalResults[resultKey][toothKey]['maDeque'] = deque()
        
        
    
    for timeKey in sorted(teethSeperated_resultsDic[resultKey]):
        
        if 'teethLine' in teethSeperated_resultsDic[resultKey][timeKey] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][timeKey]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['width'] 
            
            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin and abs(widthToFilterFor - widthTeethLine) < widthTolerance:

                for toothKey in configs.keys():
                    if toothKey in teethSeperated_resultsDic[resultKey][timeKey]:
                        actualLength = teethSeperated_resultsDic[resultKey][timeKey][toothKey][3] - teethSeperated_resultsDic[resultKey][timeKey][toothKey][2]
                        
                        finalResults[resultKey][toothKey]['times'].append(timeKey)
                        finalResults[resultKey][toothKey]['origLengths'].append(actualLength)
#############################################################################################################








#############################################################################################################
########### FINDING CALIBRATION PARAMETERS ####################
###################################################################
calibResults = {}
for resultKey in finalResults:
    print('\n\nshowing learned calib params for  ' + str(resultKey) + ':')
    calibResults[resultKey] = {}
    
    for toothKey in sorted(finalResults[resultKey].keys()):
        print('\nfor ' + str(toothKey) + ': ') 
        
        calibResults[resultKey][toothKey] = {}
        calibResults[resultKey][toothKey]['jumps'] = []
        calibResults[resultKey][toothKey]['posJumps'] = []
        calibResults[resultKey][toothKey]['negJumps'] = []
        
        for i in range(len(finalResults[resultKey][toothKey]['origLengths']) - 1):
            diff = finalResults[resultKey][toothKey]['origLengths'][i+1] - finalResults[resultKey][toothKey]['origLengths'][i]
          
        
            calibResults[resultKey][toothKey]['jumps'].append(diff)

            if(diff > 0):
                calibResults[resultKey][toothKey]['posJumps'].append(diff)
            else:
                calibResults[resultKey][toothKey]['negJumps'].append(abs(diff))

                
                
                
        posJumps_np = np.array(calibResults[resultKey][toothKey]['posJumps'])
        negJumps_np = np.array(calibResults[resultKey][toothKey]['negJumps'])
        
        calibResults[resultKey][toothKey][99.5] = np.percentile(posJumps_np, 99.5)
        calibResults[resultKey][toothKey][99.9] = np.percentile(posJumps_np, 99.9)+ 1
        calibResults[resultKey][toothKey][-1] = np.percentile(posJumps_np, 99.9) + 3
        
        calibResults[resultKey][toothKey][65] = np.percentile(negJumps_np, 65)
        calibResults[resultKey][toothKey][60] = np.percentile(negJumps_np, 60)
        calibResults[resultKey][toothKey][50] = np.percentile(negJumps_np, 50)
        calibResults[resultKey][toothKey][45] = np.percentile(negJumps_np, 45)
        calibResults[resultKey][toothKey][40] = np.percentile(negJumps_np, 40)
        calibResults[resultKey][toothKey][35] = np.percentile(negJumps_np, 35)
        calibResults[resultKey][toothKey][30] = np.percentile(negJumps_np, 30)
        calibResults[resultKey][toothKey][25] = np.percentile(negJumps_np, 25)
        calibResults[resultKey][toothKey][20] = np.percentile(negJumps_np, 20)
        calibResults[resultKey][toothKey][15] = np.percentile(negJumps_np, 15)
        calibResults[resultKey][toothKey][10] = np.percentile(negJumps_np, 10)
        
        print('99.5th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.5]))
        print('99.99th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.9]))
        print('50th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][50]))
        print('35th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][35]))
        print('30th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][30]))
#############################################################################################################








#############################################################################################################
################ Normalizing tooth lengths ####################
###################################################################
for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in finalResults[resultKey].keys():
        
        posJumpMin = calibResults[resultKey][toothKey][configs[toothKey]['posJumpMinPercentile']]  
        negJumpMax = calibResults[resultKey][toothKey][configs[toothKey]['negJumpMaxPercentile']]  
        movingAvgLength = configs[toothKey]['movingAvgLength']
        
        
        print('\nfor ' + str(toothKey) + ': ') 
        
        print('posJumpMin = ' + str(posJumpMin) + ' which was ' + str(configs[toothKey]['posJumpMinPercentile']) + 'th percentile of positiveJumps')
        
        print('negJumpMax = ' + str(negJumpMax) + ' which was ' + str(configs[toothKey]['negJumpMaxPercentile']) + 'th percentile of negetiveJumps')
        
        print('movingAvgLength = ' + str('movingAvgLength'))
        
        
        
        for toothLen in finalResults[resultKey][toothKey]['origLengths']:
                    
            normToothLen = getMovingAverage4(finalResults[resultKey][toothKey]['maDeque'], toothLen, movingAvgLength,posJumpMin,negJumpMax)
            
            finalResults[resultKey][toothKey]['normalizedLengths'].append(normToothLen)
#############################################################################################################









                        
                        
                        
'''                        
#############################################################################################################
############# Plotting Original Lengths Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):

        print('showing original lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['origLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################     '''                   
                        
                
'''                       
#############################################################################################################
############# Plotting Final Results Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):        
        print('showing normalized lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['normalizedLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################
'''


#############################################################################################################
############# Plotting Final Results Same Graphs ##########################
###############################################################################
for toothKey in sorted(configs.keys()):
    print('\n\n\nshowing normalized lengths for:  ' + str(toothKey))
    
    plt.figure(figsize=(30,10))
    plt.axis([0, 800, 20, 100])
    ax = plt.axes()
    ax.grid()

    for resultKey in finalResults:

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


    ax.legend()
    
    plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/'+toothKey+'.png')
    
    plt.show()
#############################################################################################################

In [ ]:
# (no starting points)Auto Calbrating, Normalizing and Plotting results

'''
configs = {
    'tooth0':{'posJumpMinPercentile':99.5 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2}
}
configs = {
    'tooth0':{'posJumpMinPercentile':99.5 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2}
}

configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44}
}

configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
}

configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
}
'''
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
}




imHeight = 480

#ymiddleLimitMin = 180
#ymiddleLimitMax = 210 
ymiddleLimitMin = 260
ymiddleLimitMax = 290 


widthToFilterFor = 450
widthTolerance = 500000






#############################################################################################################
###### Process the lengths to get a sequence for each tooth #######
###################################################################
finalResults = {}

for resultKey in teethSeperated_resultsDic:
    finalResults[resultKey] = {}
    
    for toothKey in configs.keys():
        finalResults[resultKey][toothKey] = {}
        finalResults[resultKey][toothKey]['origLengths'] = []
        finalResults[resultKey][toothKey]['normalizedLengths'] = []
        finalResults[resultKey][toothKey]['times'] = []
        finalResults[resultKey][toothKey]['maDeque'] = deque()
        
        
    
    for timeKey in sorted(teethSeperated_resultsDic[resultKey]):
        
        if 'teethLine' in teethSeperated_resultsDic[resultKey][timeKey] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][timeKey]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['width'] 
            
            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin and abs(widthToFilterFor - widthTeethLine) < widthTolerance:

                for toothKey in configs.keys():
                    if toothKey in teethSeperated_resultsDic[resultKey][timeKey]:
                        actualLength = teethSeperated_resultsDic[resultKey][timeKey][toothKey][3] - teethSeperated_resultsDic[resultKey][timeKey][toothKey][2]
                        
                        finalResults[resultKey][toothKey]['times'].append(timeKey)
                        finalResults[resultKey][toothKey]['origLengths'].append(actualLength)
#############################################################################################################








#############################################################################################################
########### FINDING CALIBRATION PARAMETERS ####################
###################################################################
calibResults = {}
for resultKey in finalResults:
    print('\n\nshowing learned calib params for  ' + str(resultKey) + ':')
    calibResults[resultKey] = {}
    
    for toothKey in sorted(finalResults[resultKey].keys()):
        print('\nfor ' + str(toothKey) + ': ') 
        
        calibResults[resultKey][toothKey] = {}
        calibResults[resultKey][toothKey]['jumps'] = []
        calibResults[resultKey][toothKey]['posJumps'] = []
        calibResults[resultKey][toothKey]['negJumps'] = []
        
        for i in range(len(finalResults[resultKey][toothKey]['origLengths']) - 1):
            diff = finalResults[resultKey][toothKey]['origLengths'][i+1] - finalResults[resultKey][toothKey]['origLengths'][i]
          
        
            calibResults[resultKey][toothKey]['jumps'].append(diff)

            if(diff > 0):
                calibResults[resultKey][toothKey]['posJumps'].append(diff)
            else:
                calibResults[resultKey][toothKey]['negJumps'].append(abs(diff))

                
                
                
        posJumps_np = np.array(calibResults[resultKey][toothKey]['posJumps'])
        negJumps_np = np.array(calibResults[resultKey][toothKey]['negJumps'])
        
        calibResults[resultKey][toothKey][85.0] = np.percentile(posJumps_np, 85.0)
        calibResults[resultKey][toothKey][99.0] = np.percentile(posJumps_np, 99.0)
        calibResults[resultKey][toothKey][99.5] = np.percentile(posJumps_np, 99.5)
        calibResults[resultKey][toothKey][99.9] = np.percentile(posJumps_np, 99.9)
        calibResults[resultKey][toothKey][-1] = np.percentile(posJumps_np, 99.9) + 1
        
        calibResults[resultKey][toothKey][75] = np.percentile(negJumps_np, 75)
        calibResults[resultKey][toothKey][70] = np.percentile(negJumps_np, 70)
        calibResults[resultKey][toothKey][65] = np.percentile(negJumps_np, 65)
        calibResults[resultKey][toothKey][60] = np.percentile(negJumps_np, 60)
        calibResults[resultKey][toothKey][50] = np.percentile(negJumps_np, 50)
        calibResults[resultKey][toothKey][40] = np.percentile(negJumps_np, 40)
        calibResults[resultKey][toothKey][35] = np.percentile(negJumps_np, 35)
        calibResults[resultKey][toothKey][30] = np.percentile(negJumps_np, 30)
        calibResults[resultKey][toothKey][25] = np.percentile(negJumps_np, 25)
        calibResults[resultKey][toothKey][20] = np.percentile(negJumps_np, 20)
        calibResults[resultKey][toothKey][15] = np.percentile(negJumps_np, 15)
        calibResults[resultKey][toothKey][10] = np.percentile(negJumps_np, 10)
        
        print('99.5th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.5]))
        print('99.99th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.9]))
        print('50th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][50]))
        print('35th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][35]))
        print('30th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][30]))
#############################################################################################################








#############################################################################################################
################ Normalizing tooth lengths ####################
###################################################################
for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in finalResults[resultKey].keys():
        
        posJumpMin = calibResults[resultKey][toothKey][configs[toothKey]['posJumpMinPercentile']]  
        negJumpMax = calibResults[resultKey][toothKey][configs[toothKey]['negJumpMaxPercentile']]  
        movingAvgLength = configs[toothKey]['movingAvgLength']
        
        
        print('\nfor ' + str(toothKey) + ': ') 
        
        print('posJumpMin = ' + str(posJumpMin) + ' which was ' + str(configs[toothKey]['posJumpMinPercentile']) + 'th percentile of positiveJumps')
        
        print('negJumpMax = ' + str(negJumpMax) + ' which was ' + str(configs[toothKey]['negJumpMaxPercentile']) + 'th percentile of negetiveJumps')
        
        print('movingAvgLength = ' + str('movingAvgLength'))
        
        
        
        for toothLen in finalResults[resultKey][toothKey]['origLengths']:
                    
            normToothLen = getMovingAverage4(finalResults[resultKey][toothKey]['maDeque'], toothLen, movingAvgLength,posJumpMin,negJumpMax)
            
            finalResults[resultKey][toothKey]['normalizedLengths'].append(normToothLen)
#############################################################################################################









                        
                        
                        
'''                      
#############################################################################################################
############# Plotting Original Lengths Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):

        print('showing original lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['origLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################     
'''
                 
                        
                
'''                       
#############################################################################################################
############# Plotting Final Results Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):        
        print('showing normalized lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['normalizedLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################
'''


#############################################################################################################
############# Plotting Final Results Same Graphs ##########################
###############################################################################
for toothKey in sorted(configs.keys()):
    print('\n\n\nshowing normalized lengths for:  ' + str(toothKey))
    
    plt.figure(figsize=(30,10))
    ax = plt.axes()
    ax.grid()

    for resultKey in finalResults:

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


    ax.legend()
    
    plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/'+toothKey+'.png')
    
    plt.show()
#############################################################################################################

In [ ]:
calibResults

In [ ]:
# Just Auto Calibration


'''
configs = {
    'tooth0':{'posJumpMinPercentile':99.5 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2}
}
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44}
}




configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
}

configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2, },
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
}

configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
}

    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 80},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 80},
}

'''
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 80},
}




imHeight = 480

ymiddleLimitMax = 210 
ymiddleLimitMin = 180

widthToFilterFor = 450
widthTolerance = 500000






#############################################################################################################
###### Process the lengths to get a sequence for each tooth #######
###################################################################
finalResults = {}

for resultKey in teethSeperated_resultsDic:
    finalResults[resultKey] = {}
    
    for toothKey in configs.keys():
        finalResults[resultKey][toothKey] = {}
        finalResults[resultKey][toothKey]['origLengths'] = []
        finalResults[resultKey][toothKey]['origLengths'].append(configs[toothKey]['start'])
        
        finalResults[resultKey][toothKey]['normalizedLengths'] = []
        finalResults[resultKey][toothKey]['times'] = []
        finalResults[resultKey][toothKey]['times'].append(-1)
        finalResults[resultKey][toothKey]['maDeque'] = deque()
        
        
    
    for timeKey in sorted(teethSeperated_resultsDic[resultKey]):
        
        if 'teethLine' in teethSeperated_resultsDic[resultKey][timeKey] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][timeKey]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['width'] 
            
            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin and abs(widthToFilterFor - widthTeethLine) < widthTolerance:

                for toothKey in configs.keys():
                    if toothKey in teethSeperated_resultsDic[resultKey][timeKey]:
                        actualLength = teethSeperated_resultsDic[resultKey][timeKey][toothKey][3] - teethSeperated_resultsDic[resultKey][timeKey][toothKey][2]
                        
                        finalResults[resultKey][toothKey]['times'].append(timeKey)
                        finalResults[resultKey][toothKey]['origLengths'].append(actualLength)
#############################################################################################################








#############################################################################################################
########### FINDING CALIBRATION PARAMETERS ####################
###################################################################
calibResults = {}
for resultKey in finalResults:
    print('\n\nshowing learned calib params for  ' + str(resultKey) + ':')
    calibResults[resultKey] = {}
    
    for toothKey in sorted(finalResults[resultKey].keys()):
        print('\nfor ' + str(toothKey) + ': ') 
        
        calibResults[resultKey][toothKey] = {}
        calibResults[resultKey][toothKey]['jumps'] = []
        calibResults[resultKey][toothKey]['posJumps'] = []
        calibResults[resultKey][toothKey]['negJumps'] = []
        
        for i in range(len(finalResults[resultKey][toothKey]['origLengths']) - 1):
            diff = finalResults[resultKey][toothKey]['origLengths'][i+1] - finalResults[resultKey][toothKey]['origLengths'][i]
          
        
            calibResults[resultKey][toothKey]['jumps'].append(diff)

            if(diff > 0):
                calibResults[resultKey][toothKey]['posJumps'].append(diff)
            else:
                calibResults[resultKey][toothKey]['negJumps'].append(abs(diff))

                
                
                
        posJumps_np = np.array(calibResults[resultKey][toothKey]['posJumps'])
        negJumps_np = np.array(calibResults[resultKey][toothKey]['negJumps'])
        
        calibResults[resultKey][toothKey][99.5] = np.percentile(posJumps_np, 99.5)
        calibResults[resultKey][toothKey][99.9] = np.percentile(posJumps_np, 99.9) + 1
        
        calibResults[resultKey][toothKey][65] = np.percentile(negJumps_np, 65)
        calibResults[resultKey][toothKey][60] = np.percentile(negJumps_np, 60)
        calibResults[resultKey][toothKey][50] = np.percentile(negJumps_np, 50)
        calibResults[resultKey][toothKey][40] = np.percentile(negJumps_np, 40)
        calibResults[resultKey][toothKey][35] = np.percentile(negJumps_np, 35)
        calibResults[resultKey][toothKey][30] = np.percentile(negJumps_np, 30)
        calibResults[resultKey][toothKey][25] = np.percentile(negJumps_np, 25)
        calibResults[resultKey][toothKey][20] = np.percentile(negJumps_np, 20)
        calibResults[resultKey][toothKey][10] = np.percentile(negJumps_np, 10)
        
        print('99.5th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.5]))
        print('99.99th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.9]))
        print('50th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][50]))
        print('35th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][35]))
        print('30th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][30]))
#############################################################################################################

In [ ]:
# Just Normalizing and Plotting results

'''
configs = {
    'tooth0':{'posJumpMinPercentile':99.5 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2}
}
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44}
}




configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
}

configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2, },
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':10, 'movingAvgLength':2},
}

configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
}

    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 80},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 80},
}

'''
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':15, 'movingAvgLength':2, 'start': 70},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':15, 'movingAvgLength':2, 'start': 70},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':15, 'movingAvgLength':2, 'start': 70},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':15, 'movingAvgLength':2, 'start': 70},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':15, 'movingAvgLength':2, 'start': 70},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':15, 'movingAvgLength':2, 'start': 70},
}




imHeight = 480

ymiddleLimitMax = 210 
ymiddleLimitMin = 180

widthToFilterFor = 450
widthTolerance = 500000






#############################################################################################################
###### Process the lengths to get a sequence for each tooth #######
###################################################################
finalResults = {}

for resultKey in teethSeperated_resultsDic:
    finalResults[resultKey] = {}
    
    for toothKey in configs.keys():
        finalResults[resultKey][toothKey] = {}
        finalResults[resultKey][toothKey]['origLengths'] = []
        finalResults[resultKey][toothKey]['origLengths'].append(configs[toothKey]['start'])
        
        finalResults[resultKey][toothKey]['normalizedLengths'] = []
        finalResults[resultKey][toothKey]['times'] = []
        finalResults[resultKey][toothKey]['times'].append(-1)
        finalResults[resultKey][toothKey]['maDeque'] = deque()
        
        
    
    for timeKey in sorted(teethSeperated_resultsDic[resultKey]):
        
        if 'teethLine' in teethSeperated_resultsDic[resultKey][timeKey] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][timeKey]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['width'] 
            
            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin and abs(widthToFilterFor - widthTeethLine) < widthTolerance:

                for toothKey in configs.keys():
                    if toothKey in teethSeperated_resultsDic[resultKey][timeKey]:
                        actualLength = teethSeperated_resultsDic[resultKey][timeKey][toothKey][3] - teethSeperated_resultsDic[resultKey][timeKey][toothKey][2]
                        
                        finalResults[resultKey][toothKey]['times'].append(timeKey)
                        finalResults[resultKey][toothKey]['origLengths'].append(actualLength)
#############################################################################################################






#############################################################################################################
################ Normalizing tooth lengths ####################
###################################################################
for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in finalResults[resultKey].keys():
        
        posJumpMin = calibResults[resultKey][toothKey][configs[toothKey]['posJumpMinPercentile']]  
        negJumpMax = calibResults[resultKey][toothKey][configs[toothKey]['negJumpMaxPercentile']]  
        movingAvgLength = configs[toothKey]['movingAvgLength']
        
        
        print('\nfor ' + str(toothKey) + ': ') 
        
        print('posJumpMin = ' + str(posJumpMin) + ' which was ' + str(configs[toothKey]['posJumpMinPercentile']) + 'th percentile of positiveJumps')
        
        print('negJumpMax = ' + str(negJumpMax) + ' which was ' + str(configs[toothKey]['negJumpMaxPercentile']) + 'th percentile of negetiveJumps')
        
        print('movingAvgLength = ' + str('movingAvgLength'))
        
        
        
        for toothLen in finalResults[resultKey][toothKey]['origLengths']:
                    
            normToothLen = getMovingAverage4(finalResults[resultKey][toothKey]['maDeque'], toothLen, movingAvgLength,posJumpMin,negJumpMax)
            
            finalResults[resultKey][toothKey]['normalizedLengths'].append(normToothLen)
#############################################################################################################









                        
                        
                        
'''                        
#############################################################################################################
############# Plotting Original Lengths Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):

        print('showing original lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['origLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################     '''                   
                        
                
'''                       
#############################################################################################################
############# Plotting Final Results Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):        
        print('showing normalized lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['normalizedLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################
'''


#############################################################################################################
############# Plotting Final Results Same Graphs ##########################
###############################################################################
for toothKey in sorted(configs.keys()):
    print('\n\n\nshowing normalized lengths for:  ' + str(toothKey))
    
    plt.figure(figsize=(30,10))
    plt.axis([0, 800, 20, 100])
    ax = plt.axes()
    ax.grid()

    for resultKey in finalResults:

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


    ax.legend()
    
    plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/'+toothKey+'.png')
    
    plt.show()
#############################################################################################################

## <font color='blue'/> R&D Optical_Hydraulics

In [ ]:
# R&D  Full (all offsets included) Auto Calbrating, Normalizing and Plotting results

'''
#This works for Kearl
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44}
}


#This kinda works for CNRL
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 85},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':40, 'movingAvgLength':2, 'start': 85},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 85},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 80},
}


#I was playing around with this for Kearl
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':60, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth6':{'posJumpMinPercentile':99.9   , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 4, 'lengthOffset': 0},
    
    'tooth7':{'posJumpMinPercentile':99.9   , 'negJumpMaxPercentile':50, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 4, 'lengthOffset': 0},
    
    'tooth8':{'posJumpMinPercentile':99.9   , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 5, 'lengthOffset': 0}
}


#Used this for Pinto
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0}
}




configs = {
    'tooth0':{'posJumpMinPercentile':80.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth1':{'posJumpMinPercentile':70.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth2':{'posJumpMinPercentile':60.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth3':{'posJumpMinPercentile':80.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth4':{'posJumpMinPercentile':80.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth5':{'posJumpMinPercentile':80.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0}
}



#Used this for wmdlLogs_Liebherr

configs = {
    'tooth0':{'posJumpMinPercentile':93.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth1':{'posJumpMinPercentile':93.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth2':{'posJumpMinPercentile':75.0 , 'negJumpMaxPercentile':45, 'movingAvgLength':6, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth3':{'posJumpMinPercentile':82.0 , 'negJumpMaxPercentile':20, 'movingAvgLength':4, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth4':{'posJumpMinPercentile':93.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0},
    
    'tooth5':{'posJumpMinPercentile':93.0 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 44, 'posJumpMinOffset': 0, 'lengthOffset': 0}
}

'''
configs = {
    'tooth0':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0},
    
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':65, 'movingAvgLength':2, 'start': 64, 'posJumpMinOffset': 1, 'lengthOffset': 0}
}

'''
#For thermal_bucyrus
ymiddleLimitMin = 180
ymiddleLimitMax = 210 

'''
#For optical_Hydraulics
ymiddleLimitMin = -1#260
ymiddleLimitMax = 100000#290 


widthToFilterFor = 450
widthTolerance = 500000

maxLength = 433363

imHeight = 480

averageLength = 3




#############################################################################################################
###### Process the lengths to get a sequence for each tooth #######
###################################################################
finalResults = {}

for resultKey in teethSeperated_resultsDic:
    finalResults[resultKey] = {}
    
    for toothKey in configs.keys():
        finalResults[resultKey][toothKey] = {}
        finalResults[resultKey][toothKey]['origLengths'] = []
       # finalResults[resultKey][toothKey]['origLengths'].append(configs[toothKey]['start'])
        
        finalResults[resultKey][toothKey]['normalizedLengths'] = []
        finalResults[resultKey][toothKey]['normalizedTimes'] = []
        finalResults[resultKey][toothKey]['times'] = []
       # finalResults[resultKey][toothKey]['times'].append(-1)
        finalResults[resultKey][toothKey]['maDeque'] = deque()
        
        finalResults[resultKey][toothKey]['averagedLengths'] = []
        finalResults[resultKey][toothKey]['averagedTimes'] = []
        
        
    
    for timeKey in sorted(teethSeperated_resultsDic[resultKey]):
        
        if 'teethLine' in teethSeperated_resultsDic[resultKey][timeKey] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][timeKey]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['width'] 
            
            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin and abs(widthToFilterFor - widthTeethLine) < widthTolerance: #and len(resultsDic[resultKey][timeKey]['wearMarks']) > 1:

                for toothKey in configs.keys():
                    if toothKey in teethSeperated_resultsDic[resultKey][timeKey]:
                        
                        #if toothKey == 'tooth2' or toothKey == 'tooth3':
                        actualLength = teethSeperated_resultsDic[resultKey][timeKey][toothKey][3] - teethSeperated_resultsDic[resultKey][timeKey][toothKey][2]
                        #else:
                        #actualLength = teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['box'][3] - teethSeperated_resultsDic[resultKey][timeKey][toothKey][2]
                        
                        if actualLength < maxLength:
                            finalResults[resultKey][toothKey]['times'].append(timeKey)
                            finalResults[resultKey][toothKey]['origLengths'].append(actualLength)
#############################################################################################################




 





#############################################################################################################
################ Averaging tooth lengths ####################
###################################################################
for resultKey in finalResults:
    for toothKey in finalResults[resultKey].keys():
        finalResults[resultKey][toothKey]['maDeque'].clear()



for resultKey in finalResults:
    for toothKey in finalResults[resultKey].keys():
                
        
        for i in range(averageLength):
            
            finalResults[resultKey][toothKey]['maDeque'].append(finalResults[resultKey][toothKey]['origLengths'][i])
            
        finalResults[resultKey][toothKey]['maDeque'].append(finalResults[resultKey][toothKey]['origLengths'][averageLength])
        
        
        for i in range(averageLength+1, 2*averageLength + 1):
            finalResults[resultKey][toothKey]['maDeque'].append(finalResults[resultKey][toothKey]['origLengths'][i])
        
        

        for i in range(averageLength, len(finalResults[resultKey][toothKey]['origLengths']) - averageLength - 1, 1):
       
            normToothLen = getAv(finalResults[resultKey][toothKey]['maDeque'])#sum(finalResults[resultKey]            
            finalResults[resultKey][toothKey]['averagedLengths'].append(normToothLen)
            finalResults[resultKey][toothKey]['averagedTimes'].append(finalResults[resultKey][toothKey]['times'][i])
            
            
            finalResults[resultKey][toothKey]['maDeque'].popleft()
            finalResults[resultKey][toothKey]['maDeque'].append(finalResults[resultKey][toothKey]['origLengths'][i + averageLength + 1])
#############################################################################################################     '''          
                    





'''
#############################################################################################################
########### FINDING CALIBRATION PARAMETERS ####################
###################################################################
calibResults = {}
for resultKey in finalResults:
    print('\n\nshowing learned calib params for  ' + str(resultKey) + ':')
    calibResults[resultKey] = {}
    
    for toothKey in sorted(finalResults[resultKey].keys()):
        print('\nfor ' + str(toothKey) + ': ') 
        
        calibResults[resultKey][toothKey] = {}
        calibResults[resultKey][toothKey]['jumps'] = []
        calibResults[resultKey][toothKey]['posJumps'] = []
        calibResults[resultKey][toothKey]['negJumps'] = []
        
        for i in range(len(finalResults[resultKey][toothKey]['origLengths']) - 1):
            diff = finalResults[resultKey][toothKey]['origLengths'][i+1] - finalResults[resultKey][toothKey]['origLengths'][i]
          
        
            calibResults[resultKey][toothKey]['jumps'].append(diff)

            if(diff > 0):
                calibResults[resultKey][toothKey]['posJumps'].append(diff)
            else:
                calibResults[resultKey][toothKey]['negJumps'].append(abs(diff))

                
                
                
        posJumps_np = np.array(calibResults[resultKey][toothKey]['posJumps'])
        negJumps_np = np.array(calibResults[resultKey][toothKey]['negJumps'])
        
        
        calibResults[resultKey][toothKey][60.0] = np.percentile(posJumps_np, 60.0)
        calibResults[resultKey][toothKey][70.0] = np.percentile(posJumps_np, 70.0)
        calibResults[resultKey][toothKey][80.0] = np.percentile(posJumps_np, 80.0)
        calibResults[resultKey][toothKey][82.0] = np.percentile(posJumps_np, 82.0)
        calibResults[resultKey][toothKey][85.0] = np.percentile(posJumps_np, 85.0)
        calibResults[resultKey][toothKey][90.0] = np.percentile(posJumps_np, 90.0)
        calibResults[resultKey][toothKey][95.0] = np.percentile(posJumps_np, 97.0)
        calibResults[resultKey][toothKey][93.0] = np.percentile(posJumps_np, 94.0)
        calibResults[resultKey][toothKey][99.0] = np.percentile(posJumps_np, 99.0)
        calibResults[resultKey][toothKey][99.5] = np.percentile(posJumps_np, 99.5)
        calibResults[resultKey][toothKey][99.9] = np.percentile(posJumps_np, 99.9)
        
        
        calibResults[resultKey][toothKey][85] = np.percentile(negJumps_np, 85)
        calibResults[resultKey][toothKey][75] = np.percentile(negJumps_np, 75)
        calibResults[resultKey][toothKey][65] = np.percentile(negJumps_np, 65)
        calibResults[resultKey][toothKey][60] = np.percentile(negJumps_np, 60)
        calibResults[resultKey][toothKey][55] = np.percentile(negJumps_np, 55)
        calibResults[resultKey][toothKey][50] = np.percentile(negJumps_np, 50)
        calibResults[resultKey][toothKey][45] = np.percentile(negJumps_np, 45)
        calibResults[resultKey][toothKey][43] = np.percentile(negJumps_np, 42)
        calibResults[resultKey][toothKey][40] = np.percentile(negJumps_np, 40)
        calibResults[resultKey][toothKey][35] = np.percentile(negJumps_np, 35)
        calibResults[resultKey][toothKey][30] = np.percentile(negJumps_np, 30)
        calibResults[resultKey][toothKey][25] = np.percentile(negJumps_np, 25)
        calibResults[resultKey][toothKey][20] = np.percentile(negJumps_np, 20)
        calibResults[resultKey][toothKey][15] = np.percentile(negJumps_np, 15)
        calibResults[resultKey][toothKey][10] = np.percentile(negJumps_np, 10)
        
        print('99.5th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.5]))
        print('99.99th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.9]))
        print('50th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][50]))
        print('35th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][35]))
        print('30th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][30]))
#############################################################################################################








#############################################################################################################
################ Normalizing tooth lengths ####################
###################################################################
for resultKey in finalResults:
    for toothKey in finalResults[resultKey].keys():
        finalResults[resultKey][toothKey]['maDeque'].clear()
        
        
for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in finalResults[resultKey].keys():
        
        
        posJumpMin = calibResults[resultKey][toothKey][configs[toothKey]['posJumpMinPercentile']] + configs[toothKey]['posJumpMinOffset']
        
        
        negJumpMax = calibResults[resultKey][toothKey][configs[toothKey]['negJumpMaxPercentile']]  
        movingAvgLength = configs[toothKey]['movingAvgLength']
        
        
        
        print('\nfor ' + str(toothKey) + ': ') 
        
        print('posJumpMin = ' + str(posJumpMin) + ' which was ' + str(configs[toothKey]['posJumpMinPercentile']) + 'th percentile of positiveJumps')
        
        print('negJumpMax = ' + str(negJumpMax) + ' which was ' + str(configs[toothKey]['negJumpMaxPercentile']) + 'th percentile of negetiveJumps')
        
        print('movingAvgLength = ' + str('movingAvgLength'))
        
        
        
        for toothLen in finalResults[resultKey][toothKey]['averagedLengths']:
                    
            normToothLen = getMovingAverage4(finalResults[resultKey][toothKey]['maDeque'], toothLen, movingAvgLength,posJumpMin,negJumpMax) + configs[toothKey]['lengthOffset']
            
            #normToothLen = getMovingAverage5(finalResults[resultKey][toothKey]['maDeque'], toothLen, movingAvgLength,posJumpMin,posJumpMax,negJumpMax) + configs[toothKey]['lengthOffset']
            
            finalResults[resultKey][toothKey]['normalizedLengths'].append(normToothLen)
#############################################################################################################




                    
                        

        
        
        
        
        
        
        
        
        





#############################################################################################################
############# Plotting Original Lengths Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):

        print('showing original lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['origLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'],'ro')
        
        plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/'+toothKey+'.png')

        ax.legend()
        plt.show()
############################################################################################################# 



#############################################################################################################
############# Plotting Final Results Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):        
        print('showing normalized lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['normalizedLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################
               
'''                    



#############################################################################################################
############# Plotting Average Lengths Seperate Graphs ##########################
###############################################################################            
for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in sorted(finalResults[resultKey].keys()):

        print('\n\n\nshowing normalized lengths for:  ' + str(toothKey))

        plt.figure(figsize=(30,10))
        #plt.axis([0, 800, 20, 100])
        ax = plt.axes()
        ax.grid()

        for resultKey in finalResults:

            plt.plot(finalResults[resultKey][toothKey]['averagedTimes'],finalResults[resultKey][toothKey]['averagedLengths'])
            plt.plot(finalResults[resultKey][toothKey]['averagedTimes'], finalResults[resultKey][toothKey]['averagedLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################


'''
#############################################################################################################
############# Plotting Final Results Same Graphs ##########################
###############################################################################
for toothKey in sorted(configs.keys()):
    print('\n\n\nshowing normalized lengths for:  ' + str(toothKey))
    
    plt.figure(figsize=(30,10))
    #plt.axis([0, 800, 20, 100])
    ax = plt.axes()
    ax.grid()

    for resultKey in finalResults:

        plt.plot(finalResults[resultKey][toothKey]['averagedTimes'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['averagedTimes'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


    ax.legend()
    
    plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/'+toothKey+'.png')
    
    plt.show()
#############################################################################################################
'''

In [ ]:
#playground

for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in finalResults[resultKey].keys():
        
        finalResults[resultKey][toothKey]['maDeque'].clear()




for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in finalResults[resultKey].keys():
        
        
        
        for i in range(averageLength):
            #print(i)
            finalResults[resultKey][toothKey]['maDeque'].append(finalResults[resultKey][toothKey]['origLengths'][i])
            
        finalResults[resultKey][toothKey]['maDeque'].append(finalResults[resultKey][toothKey]['origLengths'][averageLength])
        
        #print('\n\n\n')
        for i in range(averageLength+1, 2*averageLength + 1):
            #print(i)
            finalResults[resultKey][toothKey]['maDeque'].append(finalResults[resultKey][toothKey]['origLengths'][i])
        
        
        print('\n\n\n')
        for i in range(averageLength, len(finalResults[resultKey][toothKey]['origLengths']) - averageLength - 1, 1):
            print(i)
            
            print('normtoothLen:\n')
            

            normToothLen = getAv(finalResults[resultKey][toothKey]['maDeque'])#sum(finalResults[resultKey][toothKey]['maDeque'])/len(finalResults[resultKey][toothKey]['maDeque'])
            
            print(normToothLen)
            
            finalResults[resultKey][toothKey]['normalizedLengths'].append(normToothLen)
            finalResults[resultKey][toothKey]['normalizedTimes'].append(finalResults[resultKey][toothKey]['times'][i])
            
            
            finalResults[resultKey][toothKey]['maDeque'].popleft()
            finalResults[resultKey][toothKey]['maDeque'].append(finalResults[resultKey][toothKey]['origLengths'][i + averageLength + 1])
                
                    

            
            
for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in finalResults[resultKey].keys():

        print('\n\n\nshowing normalized lengths for:  ' + str(toothKey))

        plt.figure(figsize=(30,10))
        #plt.axis([0, 800, 20, 100])
        ax = plt.axes()
        ax.grid()

        for resultKey in finalResults:

            #plt.plot(finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
            plt.plot(finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


        ax.legend()

        plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT/wm_algoDevelopment_optical_hydraulics__try1-ssd_result-ckpt-170025/'+toothKey+'.png')

        plt.show()

In [ ]:
#Bining the lengths yCentre and width TeethLine, and getting statistics
            
#resultKey = 'wmdlLogs_aitik'           
resultKey = 'wmdlLogs_Phukham'

imHeight = 480


yCentreTeethLine_buckets = []
for i in range(50,500, 30):
#for i in range(50,500, 4):
    yCentreTeethLine_buckets.append(i)
    

widthTeethLine_buckets = []
for i in range(10,1000, 50):
#for i in range(150,700, 50):
    widthTeethLine_buckets.append(i)

        
#teethKeys_toLookFor = ['tooth0','tooth1','tooth2','tooth3','tooth4','tooth5','tooth6','tooth7','tooth8']
teethKeys_toLookFor = ['tooth0','tooth1','tooth2','tooth3','tooth4','tooth5']




    
#setup toothLengthsDic
toothLengthsDic = {}

for toothKey in teethKeys_toLookFor:
    toothLengthsDic[toothKey] = {}
    
    for i in range(len(yCentreTeethLine_buckets)-1):
        yCen = yCentreTeethLine_buckets[i]
        toothLengthsDic[toothKey][yCen] = {}
        
        for j in range(len(widthTeethLine_buckets)-1):
            toothLengthsDic[toothKey][yCen][widthTeethLine_buckets[j]] = {}
            toothLengthsDic[toothKey][yCen][widthTeethLine_buckets[j]]['originalLength'] = []
            toothLengthsDic[toothKey][yCen][widthTeethLine_buckets[j]]['times'] = []
            toothLengthsDic[toothKey][yCen][widthTeethLine_buckets[j]]['deque'] = deque()
        

        
        
# Record info
noBucketCount = 0
for time in sorted(teethSeperated_resultsDic[resultKey]):
    
    #1. Find yCentreTeethLine bucket
    curYCen = teethSeperated_resultsDic[resultKey][time]['teethLine']['yOfMiddle']
    curYCen_bucket = -1
    for i in range(len(yCentreTeethLine_buckets)-1):
        if curYCen >= yCentreTeethLine_buckets[i] and curYCen < yCentreTeethLine_buckets[i+1]:
            curYCen_bucket = yCentreTeethLine_buckets[i]


    #2. Find widthTeethLine bucket
    curWid =  teethSeperated_resultsDic[resultKey][time]['teethLine']['width']
    curWid_bucket = -1
    for i in range(len(widthTeethLine_buckets)-1):
        if curWid >= widthTeethLine_buckets[i] and curWid < widthTeethLine_buckets[i+1]:
            curWid_bucket = widthTeethLine_buckets[i]
    
    
    
    if not (curYCen_bucket == -1 or curWid_bucket == -1):
        
        #3. Record original and smoothed tooth lengths
        for toothKey in teethKeys_toLookFor:

            if toothKey in teethSeperated_resultsDic[resultKey][time]:

                
                
                
                origToothLength = teethSeperated_resultsDic[resultKey][time][toothKey][3] -teethSeperated_resultsDic[resultKey][time][toothKey][2]
                
                
                toothLengthsDic[toothKey][curYCen_bucket][curWid_bucket]['times'].append(time)
                
                toothLengthsDic[toothKey][curYCen_bucket][curWid_bucket]['originalLength'].append(origToothLength)
                
                
    else:
        #print('could not find curYCen_bucket or curWid_bucket for frame: ' + str(time) + '\n\n')
        noBucketCount +=1
            
            
            
            
            
#4. Print statistics on bin distribution

for key in sorted(toothLengthsDic.keys()):
    print("\n\n")
    print(key)
    
    toothCounter = 0
    for yBuck in toothLengthsDic[key].keys():
        
        yLocCounter = 0
        for wBuck in toothLengthsDic[key][yBuck]:
            yLocCounter += len(toothLengthsDic[key][yBuck][wBuck]['times'])
            
        if yLocCounter > 10:
            print('\nyLoc  ' + str(yBuck) + ' :')
            print('yLoc total:  ' + str(yLocCounter))
        
        toothCounter += yLocCounter
        
    print('\n ****tooth total:  ' + str(toothCounter))


## <font color='blue'/> R&D Thermal_Bucyrus

In [ ]:
# Testing minimum calibration length

# Auto Calbrating, Normalizing and Plotting results

configs = {
    'tooth0':{'posJumpMinPercentile':99.5 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth1':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':30, 'movingAvgLength':2},
    'tooth2':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth3':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth4':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth5':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth6':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':50, 'movingAvgLength':2},
    'tooth7':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2},
    'tooth8':{'posJumpMinPercentile':99.9 , 'negJumpMaxPercentile':35, 'movingAvgLength':2}
}





imHeight = 480

ymiddleLimitMax = 210 
ymiddleLimitMin = 180

widthToFilterFor = 450
widthTolerance = 500000






#############################################################################################################
###### Process the lengths to get a sequence for each tooth #######
###################################################################
finalResults = {}

for resultKey in teethSeperated_resultsDic:
    finalResults[resultKey] = {}
    
    for toothKey in configs.keys():
        finalResults[resultKey][toothKey] = {}
        finalResults[resultKey][toothKey]['origLengths'] = []
        finalResults[resultKey][toothKey]['normalizedLengths'] = []
        finalResults[resultKey][toothKey]['times'] = []
        finalResults[resultKey][toothKey]['maDeque'] = deque()
        
        
    
    for timeKey in sorted(teethSeperated_resultsDic[resultKey]):
        
        if 'teethLine' in teethSeperated_resultsDic[resultKey][timeKey] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][timeKey]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][timeKey]['teethLine']['width'] 
            
            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin and abs(widthToFilterFor - widthTeethLine) < widthTolerance:

                for toothKey in configs.keys():
                    if toothKey in teethSeperated_resultsDic[resultKey][timeKey]:
                        actualLength = teethSeperated_resultsDic[resultKey][timeKey][toothKey][3] - teethSeperated_resultsDic[resultKey][timeKey][toothKey][2]
                        
                        finalResults[resultKey][toothKey]['times'].append(timeKey)
                        finalResults[resultKey][toothKey]['origLengths'].append(actualLength)
#############################################################################################################








#############################################################################################################
########### FINDING CALIBRATION PARAMETERS ####################
###################################################################
calibResults = {}
for resultKey in finalResults:
    print('\n\nshowing learned calib params for  ' + str(resultKey) + ':')
    calibResults[resultKey] = {}
    
    for toothKey in sorted(finalResults[resultKey].keys()):
        print('\nfor ' + str(toothKey) + ': ') 
        
        calibResults[resultKey][toothKey] = {}
        calibResults[resultKey][toothKey]['jumps'] = []
        calibResults[resultKey][toothKey]['posJumps'] = []
        calibResults[resultKey][toothKey]['negJumps'] = []
        
        #for i in range(len(finalResults[resultKey][toothKey]['origLengths']) - 1):
        for i in range( int(len(finalResults[resultKey][toothKey]['origLengths'])*0.9) ):
            diff = finalResults[resultKey][toothKey]['origLengths'][i+1] - finalResults[resultKey][toothKey]['origLengths'][i]
          
        
            calibResults[resultKey][toothKey]['jumps'].append(diff)

            if(diff > 0):
                calibResults[resultKey][toothKey]['posJumps'].append(diff)
            else:
                calibResults[resultKey][toothKey]['negJumps'].append(abs(diff))

                
                
                
        posJumps_np = np.array(calibResults[resultKey][toothKey]['posJumps'])
        negJumps_np = np.array(calibResults[resultKey][toothKey]['negJumps'])
        
        calibResults[resultKey][toothKey][99.5] = np.percentile(posJumps_np, 99.5)
        calibResults[resultKey][toothKey][99.9] = np.percentile(posJumps_np, 99.9)
        calibResults[resultKey][toothKey][50] = np.percentile(negJumps_np, 50)
        calibResults[resultKey][toothKey][35] = np.percentile(negJumps_np, 35)
        calibResults[resultKey][toothKey][30] = np.percentile(negJumps_np, 30)
        calibResults[resultKey][toothKey][10] = np.percentile(negJumps_np, 10)
        
        print('99.5th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.5]))
        print('99.99th percentile of posJumps:  ' + str(calibResults[resultKey][toothKey][99.9]))
        print('50th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][50]))
        print('35th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][35]))
        print('30th percentile of negJumps:  ' + str(calibResults[resultKey][toothKey][30]))
#############################################################################################################








#############################################################################################################
################ Normalizing tooth lengths ####################
###################################################################
for resultKey in finalResults:
    print('\n\nshowing final normalization params for  ' + str(resultKey) + ':')
    
    for toothKey in finalResults[resultKey].keys():
        
        posJumpMin = calibResults[resultKey][toothKey][configs[toothKey]['posJumpMinPercentile']]  
        negJumpMax = calibResults[resultKey][toothKey][configs[toothKey]['negJumpMaxPercentile']]  
        movingAvgLength = configs[toothKey]['movingAvgLength']
        
        
        print('\nfor ' + str(toothKey) + ': ') 
        
        print('posJumpMin = ' + str(posJumpMin) + ' which was ' + str(configs[toothKey]['posJumpMinPercentile']) + 'th percentile of positiveJumps')
        
        print('negJumpMax = ' + str(negJumpMax) + ' which was ' + str(configs[toothKey]['negJumpMaxPercentile']) + 'th percentile of negetiveJumps')
        
        print('movingAvgLength = ' + str('movingAvgLength'))
        
        
        
        for toothLen in finalResults[resultKey][toothKey]['origLengths']:
                    
            normToothLen = getMovingAverage4(finalResults[resultKey][toothKey]['maDeque'], toothLen, movingAvgLength,posJumpMin,negJumpMax)
            
            finalResults[resultKey][toothKey]['normalizedLengths'].append(normToothLen)
#############################################################################################################









                        
                        
                        
'''                        
#############################################################################################################
############# Plotting Original Lengths Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):

        print('showing original lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['origLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################     '''                   
                        
                
'''                       
#############################################################################################################
############# Plotting Final Results Seperate Graphs ##########################
###############################################################################
for resultKey in finalResults:
    print('\n\n\nshowing original plots for:  ' + str(resultKey))
    
    for toothKey in sorted(finalResults[resultKey].keys()):        
        print('showing normalized lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['normalizedLengths'])) + ' dataPoints')

        plt.figure(figsize=(30,10))
        ax = plt.axes()
        ax.grid()

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


        ax.legend()
        plt.show()
#############################################################################################################
'''


#############################################################################################################
############# Plotting Final Results Same Graphs ##########################
###############################################################################
for toothKey in sorted(configs.keys()):
    print('\n\n\nshowing normalized lengths for:  ' + str(toothKey))
    
    plt.figure(figsize=(30,10))
    ax = plt.axes()
    ax.grid()

    for resultKey in finalResults:

        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'], label=resultKey + ': ' +toothKey)
        plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'ro')


    ax.legend()
    
    plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/'+toothKey+'.png')
    
    plt.show()
#############################################################################################################

### FInding the maximum error compared to GT

In [ ]:
# Comparing with grounTruth All cells

fullCsvPath = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogsFromDec2016/hsGtAll.csv'

dec16_gt = {}
dec16_gt['tooth0'] = {'times':[], 'lengs':[]}
dec16_gt['tooth1'] = {'times':[], 'lengs':[]}
dec16_gt['tooth2'] = {'times':[], 'lengs':[]}
dec16_gt['tooth3'] = {'times':[], 'lengs':[]}
dec16_gt['tooth4'] = {'times':[], 'lengs':[]}
dec16_gt['tooth5'] = {'times':[], 'lengs':[]}
dec16_gt['tooth6'] = {'times':[], 'lengs':[]}
dec16_gt['tooth7'] = {'times':[], 'lengs':[]}
dec16_gt['tooth8'] = {'times':[], 'lengs':[]}


csv_file = open(fullCsvPath, "r") 
data = csv_file.read()
csv_file.close()

rows = data.split('\n')

for row in rows[1 : len(rows)-1]:
    vals = row.split(',')

    
    if vals[1] != '' and float(vals[1]) > 0:
        dec16_gt['tooth0']['times'].append(float(vals[0]))
        dec16_gt['tooth0']['lengs'].append(float(vals[1]))
        
    if vals[2] != '' and float(vals[2]) > 0:
        dec16_gt['tooth1']['times'].append(float(vals[0]))
        dec16_gt['tooth1']['lengs'].append(float(vals[2]))
        
    if vals[3] != '' and float(vals[3]) > 0:
        dec16_gt['tooth2']['times'].append(float(vals[0]))
        dec16_gt['tooth2']['lengs'].append(float(vals[3]))
        
    if vals[4] != '' and float(vals[4]) > 0:
        dec16_gt['tooth3']['times'].append(float(vals[0]))
        dec16_gt['tooth3']['lengs'].append(float(vals[4]))
        
    if vals[5] != '' and float(vals[5]) > 0:
        dec16_gt['tooth4']['times'].append(float(vals[0]))
        dec16_gt['tooth4']['lengs'].append(float(vals[5]))
        
    if vals[6] != '' and float(vals[6]) > 0:
        dec16_gt['tooth5']['times'].append(float(vals[0]))
        dec16_gt['tooth5']['lengs'].append(float(vals[6]))
        
    if vals[7] != '' and float(vals[7]) > 0:
        dec16_gt['tooth6']['times'].append(float(vals[0]))
        dec16_gt['tooth6']['lengs'].append(float(vals[7]))
        
    if vals[8] != '' and float(vals[8]) > 0:
        dec16_gt['tooth7']['times'].append(float(vals[0]))
        dec16_gt['tooth7']['lengs'].append(float(vals[8]))
        
    if vals[9] != '' and float(vals[9]) > 0:
        dec16_gt['tooth8']['times'].append(float(vals[0]))
        dec16_gt['tooth8']['lengs'].append(float(vals[9]))
        




print("read " + str(len(dec16_gt['tooth0']['times'])) + " lines")


resultKey = 'wmdlLogsFromDec2016'
    
for toothKey in sorted(dec16_gt.keys()):

    print('showing original lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['origLengths'])) + ' dataPoints')

    plt.figure(figsize=(30,10))
    ax = plt.axes()
    ax.grid()
    
    plt.plot(dec16_gt[toothKey]['times'], dec16_gt[toothKey]['lengs'], label='groundTruth')
    plt.plot(dec16_gt[toothKey]['times'], dec16_gt[toothKey]['lengs'], 'ro')
    
    plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'],label='predictions')
    plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['origLengths'],'bo')

    plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/'+toothKey+'.png')

    ax.legend()
    plt.show()

In [ ]:
# Comparing with grounTruth Scaled

fullCsvPath = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogsFromDec2016/hsGtMeasured.csv'

dec16_gt = {}
dec16_gt['tooth0'] = {'times':[], 'lengs':[]}
dec16_gt['tooth1'] = {'times':[], 'lengs':[]}
dec16_gt['tooth2'] = {'times':[], 'lengs':[]}
dec16_gt['tooth3'] = {'times':[], 'lengs':[]}
dec16_gt['tooth4'] = {'times':[], 'lengs':[]}
dec16_gt['tooth5'] = {'times':[], 'lengs':[]}
dec16_gt['tooth6'] = {'times':[], 'lengs':[]}
dec16_gt['tooth7'] = {'times':[], 'lengs':[]}
dec16_gt['tooth8'] = {'times':[], 'lengs':[]}


csv_file = open(fullCsvPath, "r") 
data = csv_file.read()
csv_file.close()

rows = data.split('\n')

for row in rows[1 : len(rows)-1]:
    vals = row.split(',')

    
    if vals[1] != '':
        dec16_gt['tooth0']['times'].append(float(vals[0]))
        dec16_gt['tooth0']['lengs'].append(float(vals[11]))
        
    if vals[2] != '':
        dec16_gt['tooth1']['times'].append(float(vals[0]))
        dec16_gt['tooth1']['lengs'].append(float(vals[12]))
        
    if vals[3] != '':
        dec16_gt['tooth2']['times'].append(float(vals[0]))
        dec16_gt['tooth2']['lengs'].append(float(vals[13]))
        
    if vals[4] != '':
        dec16_gt['tooth3']['times'].append(float(vals[0]))
        dec16_gt['tooth3']['lengs'].append(float(vals[14]))
        
    if vals[5] != '':
        dec16_gt['tooth4']['times'].append(float(vals[0]))
        dec16_gt['tooth4']['lengs'].append(float(vals[15]))
        
    if vals[6] != '':
        dec16_gt['tooth5']['times'].append(float(vals[0]))
        dec16_gt['tooth5']['lengs'].append(float(vals[16]))
        
    if vals[7] != '':
        dec16_gt['tooth6']['times'].append(float(vals[0]))
        dec16_gt['tooth6']['lengs'].append(float(vals[17]))
        
    if vals[8] != '':
        dec16_gt['tooth7']['times'].append(float(vals[0]))
        dec16_gt['tooth7']['lengs'].append(float(vals[18]))
        
    if vals[9] != '':
        dec16_gt['tooth8']['times'].append(float(vals[0]))
        dec16_gt['tooth8']['lengs'].append(float(vals[19]))
        




print("read " + str(len(dec16_gt['tooth0']['times'])) + " lines")

In [ ]:
# Comparing with grounTruth Just Measurments

fullCsvPath = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/wmdlLogsFromDec2016/hsGtMeasured.csv'

dec16_gt = {}
dec16_gt['tooth0'] = {'times':[], 'lengs':[]}
dec16_gt['tooth1'] = {'times':[], 'lengs':[]}
dec16_gt['tooth2'] = {'times':[], 'lengs':[]}
dec16_gt['tooth3'] = {'times':[], 'lengs':[]}
dec16_gt['tooth4'] = {'times':[], 'lengs':[]}
dec16_gt['tooth5'] = {'times':[], 'lengs':[]}
dec16_gt['tooth6'] = {'times':[], 'lengs':[]}
dec16_gt['tooth7'] = {'times':[], 'lengs':[]}
dec16_gt['tooth8'] = {'times':[], 'lengs':[]}


csv_file = open(fullCsvPath, "r") 
data = csv_file.read()
csv_file.close()

rows = data.split('\n')

for row in rows[1 : len(rows)-1]:
    vals = row.split(',')

    
    if vals[1] != '':
        dec16_gt['tooth0']['times'].append(float(vals[0]))
        dec16_gt['tooth0']['lengs'].append(float(vals[1]))
        
    if vals[2] != '':
        dec16_gt['tooth1']['times'].append(float(vals[0]))
        dec16_gt['tooth1']['lengs'].append(float(vals[2]))
        
    if vals[3] != '':
        dec16_gt['tooth2']['times'].append(float(vals[0]))
        dec16_gt['tooth2']['lengs'].append(float(vals[3]))
        
    if vals[4] != '':
        dec16_gt['tooth3']['times'].append(float(vals[0]))
        dec16_gt['tooth3']['lengs'].append(float(vals[4]))
        
    if vals[5] != '':
        dec16_gt['tooth4']['times'].append(float(vals[0]))
        dec16_gt['tooth4']['lengs'].append(float(vals[5]))
        
    if vals[6] != '':
        dec16_gt['tooth5']['times'].append(float(vals[0]))
        dec16_gt['tooth5']['lengs'].append(float(vals[6]))
        
    if vals[7] != '':
        dec16_gt['tooth6']['times'].append(float(vals[0]))
        dec16_gt['tooth6']['lengs'].append(float(vals[7]))
        
    if vals[8] != '':
        dec16_gt['tooth7']['times'].append(float(vals[0]))
        dec16_gt['tooth7']['lengs'].append(float(vals[8]))
        
    if vals[9] != '':
        dec16_gt['tooth8']['times'].append(float(vals[0]))
        dec16_gt['tooth8']['lengs'].append(float(vals[9]))
        




print("read " + str(len(dec16_gt['tooth0']['times'])) + " lines")





resultKey = 'wmdlLogsFromDec2016'


for toothKey in sorted(dec16_gt.keys()):
    
    scalingFactor = 2.2
    #scalingFactor = max(finalResults[resultKey][toothKey]['normalizedLengths']) / max(dec16_gt[toothKey]['lengs'])
    
    #scalingFactor = min(finalResults[resultKey][toothKey]['normalizedLengths']) / min(dec16_gt[toothKey]['lengs'])
    
    #scalingFactor = (sum(finalResults[resultKey][toothKey]['normalizedLengths']) / len(finalResults[resultKey][toothKey]['normalizedLengths'])) / (sum(dec16_gt[toothKey]['lengs']) / len(dec16_gt[toothKey]['lengs']) )
    
    for i in range(len(dec16_gt[toothKey]['lengs'])):
        dec16_gt[toothKey]['lengs'][i] = dec16_gt[toothKey]['lengs'][i] * scalingFactor


    
for toothKey in sorted(dec16_gt.keys()):

    print('showing original lengths for:  ' + str(toothKey) + '  ' + str(len(finalResults[resultKey][toothKey]['origLengths'])) + ' dataPoints')

    plt.figure(figsize=(30,10))
    ax = plt.axes()
    ax.grid()
    
    plt.plot(dec16_gt[toothKey]['times'], dec16_gt[toothKey]['lengs'], label='groundTruth')
    plt.plot(dec16_gt[toothKey]['times'], dec16_gt[toothKey]['lengs'], 'ro')
    
    plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],label='predictions')
    plt.plot(finalResults[resultKey][toothKey]['times'], finalResults[resultKey][toothKey]['normalizedLengths'],'bo')

    plt.savefig('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/'+toothKey+'.png')

    ax.legend()
    plt.show()

###  Filter out based on yCentre only, and smooth each tooth out with tooth specific config

In [ ]:
#MULTIPLE Final Tooth 1, 35th

imHeight = 480

posJumpMin = 19.731351256370612
negJumpMax = 3.54824137687683 #2.941786766052246#4.940700531005859 
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth0Lengths = {}
tooth0Lengths_normalized = {}
tooth0Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth0Lengths[resultKey] = {}
    tooth0Lengths_normalized[resultKey] = {}
    tooth0Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth0' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth0'][3] - teethSeperated_resultsDic[resultKey][key]['tooth0'][2]

                    tooth0Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth0Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth0Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth0Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth0Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth0Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth0Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth0Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth0Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
20,3, 2

In [ ]:
# Get statistics on posJumpMin and negJumpMax, for tooth 1
imHeight = 480

posJumpMin = 20 #14
negJumpMax = 3 #4
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth0Lengths = {}
tooth0Lengths_normalized = {}
tooth0Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth0Lengths[resultKey] = {}
    tooth0Lengths_normalized[resultKey] = {}
    tooth0Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth0' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth0'][3] - teethSeperated_resultsDic[resultKey][key]['tooth0'][2]

                    tooth0Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage(tooth0Lengths_movingAverage[resultKey], actualLength)


                    tooth0Lengths_normalized[resultKey][key] = movingAbverage






#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth0Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))


print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n30th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

print('\n\n35th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 35)
print(perc_neg)

In [ ]:
#MULTIPLE Final Tooth 2, 35th, better with 30th

imHeight = 480

posJumpMin = 21.806332254409845
negJumpMax = 3.12546157836914 #2.578182220458984 #4.686956405639648 
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth1Lengths = {}
tooth1Lengths_normalized = {}
tooth1Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth1Lengths[resultKey] = {}
    tooth1Lengths_normalized[resultKey] = {}
    tooth1Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth1' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth1'][3] - teethSeperated_resultsDic[resultKey][key]['tooth1'][2]

                    tooth1Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth1Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth1Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth1Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth1Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth1Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth1Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth1Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth1Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
17,2,1

In [ ]:
# Get statistics on posJumpMin and negJumpMax, for tooth 2
imHeight = 480

posJumpMin = 17 #20 #30
negJumpMax = 2 #4  #3
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth1Lengths = {}
tooth1Lengths_normalized = {}
tooth1Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth1Lengths[resultKey] = {}
    tooth1Lengths_normalized[resultKey] = {}
    tooth1Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth1' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth1'][3] - teethSeperated_resultsDic[resultKey][key]['tooth1'][2]

                    tooth1Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage(tooth1Lengths_movingAverage[resultKey], actualLength)


                    tooth1Lengths_normalized[resultKey][key] = movingAbverage






#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth1Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))

print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n30th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

print('\n\n35th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 35)
print(perc_neg)

In [ ]:
#MULTIPLE Final Tooth 3, 50th only 

imHeight = 480

posJumpMin = 22.992304515838597
negJumpMax = 4.878816604614258 #2.75390625  
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth2Lengths = {}
tooth2Lengths_normalized = {}
tooth2Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth2Lengths[resultKey] = {}
    tooth2Lengths_normalized[resultKey] = {}
    tooth2Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth2' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth2'][3] - teethSeperated_resultsDic[resultKey][key]['tooth2'][2]

                    tooth2Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth2Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth2Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth2Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth2Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth2Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth2Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth2Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth2Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
20,3, 2

In [ ]:
# Get statistics on posJumpMin and negJumpMax, for eath tooth

imHeight = 480

posJumpMin = 20  #20#15
negJumpMax = 3  #4#3
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth2Lengths = {}
tooth2Lengths_normalized = {}
tooth2Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth2Lengths[resultKey] = {}
    tooth2Lengths_normalized[resultKey] = {}
    tooth2Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth2' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth2'][3] - teethSeperated_resultsDic[resultKey][key]['tooth2'][2]

                    tooth2Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth2Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth2Lengths_normalized[resultKey][key] = movingAbverage




#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth3Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))

print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n30th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

In [ ]:
#MULTIPLE Final Tooth 4, 50th or 30th 

imHeight = 480

posJumpMin = 22.992304515838597  #20 
negJumpMax = 4.878816604614258   #2.75390625
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth3Lengths = {}
tooth3Lengths_normalized = {}
tooth3Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth3Lengths[resultKey] = {}
    tooth3Lengths_normalized[resultKey] = {}
    tooth3Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth3' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth3'][3] - teethSeperated_resultsDic[resultKey][key]['tooth3'][2]

                    tooth3Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth3Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth3Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth3Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth3Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth3Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth3Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth3Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth3Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
23,5,2

In [ ]:
# Get statistics on posJumpMin and negJumpMax, for tooth 4

imHeight = 480

posJumpMin = 22.992304515838597  #20 
negJumpMax = 4.878816604614258   #4
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth3Lengths = {}
tooth3Lengths_normalized = {}
tooth3Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth3Lengths[resultKey] = {}
    tooth3Lengths_normalized[resultKey] = {}
    tooth3Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth3' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth3'][3] - teethSeperated_resultsDic[resultKey][key]['tooth3'][2]

                    tooth3Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage(tooth3Lengths_movingAverage[resultKey], actualLength)


                    tooth3Lengths_normalized[resultKey][key] = movingAbverage










#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth3Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))

print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n30th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

In [ ]:
#MULTIPLE Final Tooth 5, 50th only

imHeight = 480

posJumpMin = 25.580396175384514
negJumpMax = 4.9451494216918945 #2.878652572631836 
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth4Lengths = {}
tooth4Lengths_normalized = {}
tooth4Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth4Lengths[resultKey] = {}
    tooth4Lengths_normalized[resultKey] = {}
    tooth4Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth4' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth4'][3] - teethSeperated_resultsDic[resultKey][key]['tooth4'][2]

                    tooth4Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth4Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth4Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth4Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth4Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth4Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth4Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth4Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth4Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
21,4,2

In [ ]:
# Get statistics on posJumpMin and negJumpMax, for tooth 5



imHeight = 480

posJumpMin = 21#20 for 2ndDec #18
negJumpMax = 4 #7  for 2ndDec #5
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth4Lengths = {}
tooth4Lengths_normalized = {}
tooth4Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth4Lengths[resultKey] = {}
    tooth4Lengths_normalized[resultKey] = {}
    tooth4Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth4' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth4'][3] - teethSeperated_resultsDic[resultKey][key]['tooth4'][2]

                    tooth4Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage(tooth4Lengths_movingAverage[resultKey], actualLength)


                    tooth4Lengths_normalized[resultKey][key] = movingAbverage

                    

#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth4Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))

print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n30th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

In [ ]:
#MULTIPLE Final Tooth 6, 50th

imHeight = 480

posJumpMin = 24.54784412384033 
negJumpMax = 4.940700531005859  #2.2492504119873047
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth5Lengths = {}
tooth5Lengths_normalized = {}
tooth5Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth5Lengths[resultKey] = {}
    tooth5Lengths_normalized[resultKey] = {}
    tooth5Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth5' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth5'][3] - teethSeperated_resultsDic[resultKey][key]['tooth5'][2]

                    tooth5Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth5Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth5Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth5Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth5Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth5Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth5Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth5Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth5Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
20,4,1

In [ ]:
# Get statistics on posJumpMin and negJumpMax, for tooth 6

imHeight = 480

posJumpMin = 20#18 for 2ndDec  #20 #-1
negJumpMax = 4#8  for 2ndDec  #4  #1000
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth5Lengths = {}
tooth5Lengths_normalized = {}
tooth5Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth5Lengths[resultKey] = {}
    tooth5Lengths_normalized[resultKey] = {}
    tooth5Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth5' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth5'][3] - teethSeperated_resultsDic[resultKey][key]['tooth5'][2]

                    tooth5Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage(tooth5Lengths_movingAverage[resultKey], actualLength)


                    tooth5Lengths_normalized[resultKey][key] = movingAbverage
                    
                    
                    
                    
                    

#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth5Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))

print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n40th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

In [ ]:
#MULTIPLE Final Tooth 7, 50th

imHeight = 480

posJumpMin = 23.465128326416014
negJumpMax = 4.755220413208008 #2.60617733001709
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth6Lengths = {}
tooth6Lengths_normalized = {}
tooth6Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth6Lengths[resultKey] = {}
    tooth6Lengths_normalized[resultKey] = {}
    tooth6Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth6' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth6'][3] - teethSeperated_resultsDic[resultKey][key]['tooth6'][2]

                    tooth6Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth6Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth6Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth6Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth6Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth6Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth6Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth6Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth6Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
24,5,2

In [ ]:
# Get statistics on posJumpMin and negJumpMax, for tooth 7

imHeight = 480

posJumpMin = 24 #20 for Dec all  #-1
negJumpMax = 5 #4 for Dec all  #1000
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth6Lengths = {}
tooth6Lengths_normalized = {}
tooth6Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth6Lengths[resultKey] = {}
    tooth6Lengths_normalized[resultKey] = {}
    tooth6Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth6' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth6'][3] - teethSeperated_resultsDic[resultKey][key]['tooth6'][2]

                    tooth6Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage(tooth6Lengths_movingAverage[resultKey], actualLength)


                    tooth6Lengths_normalized[resultKey][key] = movingAbverage
                    
                    
                    

#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth6Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))

print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n99.9th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.9)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n30th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

In [ ]:
#MULTIPLE Final Tooth 8, 50th, best with 35th

imHeight = 480

posJumpMin = 22.909574890136753
negJumpMax = 2.5489425659179688#3.3023643493652344 #4.118413925170898 #2.214338779449463
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth7Lengths = {}
tooth7Lengths_normalized = {}
tooth7Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth7Lengths[resultKey] = {}
    tooth7Lengths_normalized[resultKey] = {}
    tooth7Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth7' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth7'][3] - teethSeperated_resultsDic[resultKey][key]['tooth7'][2]

                    tooth7Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth7Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth7Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth7Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth7Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth7Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth7Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth7Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth7Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
23,3,2

In [ ]:
# Get statistics on posJumpMin and negJumpMax, for eath tooth
imHeight = 480

posJumpMin = 23 #20  for Dec2nd#20
negJumpMax = 3  #5   for Dec2nd#4
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth7Lengths = {}
tooth7Lengths_normalized = {}
tooth7Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth7Lengths[resultKey] = {}
    tooth7Lengths_normalized[resultKey] = {}
    tooth7Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth7' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth7'][3] - teethSeperated_resultsDic[resultKey][key]['tooth7'][2]

                    tooth7Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage(tooth7Lengths_movingAverage[resultKey], actualLength)


                    tooth7Lengths_normalized[resultKey][key] = movingAbverage
                    
                    

#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth7Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))

print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n35th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 35)
print(perc_neg)

print('\n\n30th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

In [ ]:
#MULTIPLE Final Tooth 9, 35th

imHeight = 480

posJumpMin = 21.20268073081969
negJumpMax = 3.2632112503051753 #4.67867374420166 #3.0071840286254874
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth8Lengths = {}
tooth8Lengths_normalized = {}
tooth8Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth8Lengths[resultKey] = {}
    tooth8Lengths_normalized[resultKey] = {}
    tooth8Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth8' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth8'][3] - teethSeperated_resultsDic[resultKey][key]['tooth8'][2]

                    tooth8Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth8Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth8Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth8Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth8Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth8Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth8Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth8Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth8Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
24,4,2

In [ ]:
# Get statistics on posJumpMin and negJumpMax, for eath tooth
imHeight = 480

posJumpMin = 24 #18  for Dec2nd   #22 #-1
negJumpMax = 4  #4   for Dec2nd   #2  #1000
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth8Lengths = {}
tooth8Lengths_normalized = {}
tooth8Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth8Lengths[resultKey] = {}
    tooth8Lengths_normalized[resultKey] = {}
    tooth8Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth8' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth8'][3] - teethSeperated_resultsDic[resultKey][key]['tooth8'][2]

                    tooth8Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth8Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth8Lengths_normalized[resultKey][key] = movingAbverage

                    
                    

#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth8Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))

print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n99.99th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.99)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n35th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 35)
print(perc_neg)

print('\n\n30th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

## <font color='blue'>  Learning Calibration Parameters

In [ ]:
#copied

imHeight = 480

posJumpMin = 24 #18  for Dec2nd   #22 #-1
negJumpMax = 4  #4   for Dec2nd   #2  #1000
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth8Lengths = {}
tooth8Lengths_normalized = {}
tooth8Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth8Lengths[resultKey] = {}
    tooth8Lengths_normalized[resultKey] = {}
    tooth8Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth8' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth8'][3] - teethSeperated_resultsDic[resultKey][key]['tooth8'][2]

                    tooth8Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth8Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth8Lengths_normalized[resultKey][key] = movingAbverage


In [ ]:
# Get statistics on posJumpMin and negJumpMax, for eath tooth
imHeight = 480

posJumpMin = 24 #18  for Dec2nd   #22 #-1
negJumpMax = 4  #4   for Dec2nd   #2  #1000
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113



tooth8Lengths = {}
tooth8Lengths_normalized = {}
tooth8Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth8Lengths[resultKey] = {}
    tooth8Lengths_normalized[resultKey] = {}
    tooth8Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth8' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth8'][3] - teethSeperated_resultsDic[resultKey][key]['tooth8'][2]

                    tooth8Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth8Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth8Lengths_normalized[resultKey][key] = movingAbverage

                    
                    

#for resultKey in resultsDic:
resultKey = 'wmdlLogsFromMahdi_Kearl'
    
tempLengths = []
tempLengths_avg = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths[resultKey]):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[resultKey][frameNb])
    tempLengths_avg.append(tooth8Lengths_normalized[resultKey][frameNb])
    


    
posJumps = []
negJumps = []
allJumps = []

posJumps_avg = []
negJumps_avg = []
allJumps_avg = []

for i in range(len(tempLengths) - 1):
    diff = tempLengths[i+1] - tempLengths[i]
    diff_avg = tempLengths_avg[i+1] - tempLengths_avg[i]
    
    allJumps.append(diff)
    allJumps_avg.append(diff_avg)
    
    
    if(diff > 0):
        posJumps.append(diff)
    else:
        negJumps.append(abs(diff))
    
    
    if(diff_avg > 0):
        posJumps_avg.append(diff_avg)
    else:
        negJumps_avg.append(abs(diff_avg))
        


print(max(posJumps))
print(sum(posJumps)/len(posJumps))
print('\nNegJumps')
print(min(negJumps))
print(sum(negJumps)/len(negJumps))

print('\n\nUsing MovingAverage:')
print(max(posJumps_avg))
print(sum(posJumps_avg)/len(posJumps_avg))
print('\nNegJumps')
print(min(negJumps_avg))
print(sum(negJumps_avg)/len(negJumps_avg))

print('\n\n99.5th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.5)
print(perc)

print('\n\n99.99th percentile of posJumps:')
posJumps_np = np.array(posJumps)
perc = np.percentile(posJumps_np, 99.99)
print(perc)

print('\n\n50th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 50)
print(perc_neg)

print('\n\n35th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 35)
print(perc_neg)

print('\n\n30th percentile of negJumps:')
negJumps_np = np.array(negJumps)
perc_neg = np.percentile(negJumps_np, 30)
print(perc_neg)

In [ ]:
for i in range(len(allJumps)):
    if allJumps[i] > 0:
        print(str(allJumps[i]) + '  ' + str(tempFrameNbs[i]))

### Filtering out frames (or/and normalizing them) based on the y of centre and width of teethLine

In [ ]:
# Filtering out individual tooth lengths by yCentre and width teethLine, and calculating the Moving Averages. 

imHeight = 480

posJumpMin = 22 #-1
negJumpMax = 2  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100

normalizationMultiple_mid = 1
normalizationMultiple_width = 1
finalNormalizedLengthMultiple = 1

tooth0Lengths = {}
tooth1Lengths = {}
tooth2Lengths = {}
tooth3Lengths = {}
tooth4Lengths = {}
tooth5Lengths = {}
tooth6Lengths = {}
tooth7Lengths = {}
tooth8Lengths = {}

tooth0Lengths_normalized = {}
tooth1Lengths_normalized = {}
tooth2Lengths_normalized = {}
tooth3Lengths_normalized = {}
tooth4Lengths_normalized = {}
tooth5Lengths_normalized = {}
tooth6Lengths_normalized = {}
tooth7Lengths_normalized = {}
tooth8Lengths_normalized = {}

tooth0Lengths_movingAverage = deque()
tooth1Lengths_movingAverage = deque()
tooth2Lengths_movingAverage = deque()
tooth3Lengths_movingAverage = deque()
tooth4Lengths_movingAverage = deque()
tooth5Lengths_movingAverage = deque()
tooth6Lengths_movingAverage = deque()
tooth7Lengths_movingAverage = deque()
tooth8Lengths_movingAverage = deque()


processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle'] * normalizationMultiple_mid
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] * normalizationMultiple_width
        
        print(yMiddleOfTeethLine / widthTeethLine)

        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth0' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth0'][3] - teethSeperated_resultsDic[key]['tooth0'][2]
                
                tooth0Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage3(tooth0Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth0Lengths_normalized[key] = movingAbverage


            if 'tooth1' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth1'][3] - teethSeperated_resultsDic[key]['tooth1'][2]

                tooth1Lengths[key] = actualLength

                movingAbverage = getMovingAverage3(tooth1Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth1Lengths_normalized[key] = movingAbverage


            if 'tooth2' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth2'][3] - teethSeperated_resultsDic[key]['tooth2'][2]
                
                tooth2Lengths[key] = actualLength

                movingAbverage = getMovingAverage3(tooth2Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth2Lengths_normalized[key] = movingAbverage


            if 'tooth3' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth3'][3] - teethSeperated_resultsDic[key]['tooth3'][2]
                
                tooth3Lengths[key] = actualLength

                movingAbverage = getMovingAverage3(tooth3Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth3Lengths_normalized[key] = movingAbverage


            if 'tooth4' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth4'][3] - teethSeperated_resultsDic[key]['tooth4'][2]
                
                tooth4Lengths[key] = actualLength


                movingAbverage = getMovingAverage3(tooth4Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth4Lengths_normalized[key] = movingAbverage


            if 'tooth5' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth5'][3] - teethSeperated_resultsDic[key]['tooth5'][2]
                
                tooth5Lengths[key] = actualLength

                movingAbverage = getMovingAverage3(tooth5Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth5Lengths_normalized[key] = movingAbverage


            if 'tooth6' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth6'][3] - teethSeperated_resultsDic[key]['tooth6'][2]
                
                tooth6Lengths[key] = actualLength

                movingAbverage = getMovingAverage3(tooth6Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth6Lengths_normalized[key] = movingAbverage


            if 'tooth7' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth7'][3] - teethSeperated_resultsDic[key]['tooth7'][2]
                
                tooth7Lengths[key] = actualLength

                movingAbverage = getMovingAverage3(tooth7Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth7Lengths_normalized[key] = movingAbverage


            if 'tooth8' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth8'][3] - teethSeperated_resultsDic[key]['tooth8'][2]

                tooth8Lengths[key] = actualLength

                movingAbverage = getMovingAverage3(tooth8Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth8Lengths_normalized[key] = movingAbverage


            processedFrames+=1



print("processes: " + str(processedFrames) + " frames and Found:\n\n" + str(len(tooth0Lengths)) + '  tooth0Lengths\n' + str(len(tooth1Lengths)) + '  tooth1Lengths\n' + str(len(tooth2Lengths)) + '  tooth2Lengths\n' + str(len(tooth3Lengths)) + '  tooth3Lengths\n' + str(len(tooth4Lengths)) + '  tooth4Lengths\n' + str(len(tooth5Lengths)) + '  tooth5Lengths\n' + str(len(tooth6Lengths)) + '  tooth6Lengths\n' + str(len(tooth7Lengths)) + '  tooth7Lengths\n' + str(len(tooth8Lengths)) + '  tooth8Lengths\n\n\n' + str(len(tooth1Lengths_normalized)) + '  tooth1Lengths_normalized\n'+str(len(tooth1Lengths_normalized)) + '  tooth1Lengths_normalized\n' + str(len(tooth2Lengths_normalized)) + '  tooth2Lengths_normalized\n' + str(len(tooth3Lengths_normalized)) + '  tooth3Lengths_normalized\n' + str(len(tooth4Lengths_normalized)) + '  tooth4Lengths_normalized\n' + str(len(tooth5Lengths_normalized)) + '  tooth5Lengths_normalized\n' + str(len(tooth6Lengths_normalized)) + '  tooth6Lengths_normalized\n' + str(len(tooth7Lengths_normalized)) + '  tooth7Lengths_normalized\n' + str(len(tooth8Lengths_normalized)) + '  tooth8Lengths_normalized\n')

In [ ]:
# Filtering out individual tooth lengths by yCentre and width teethLine, and normalizing them using FittedFunction

imHeight = 480

ymiddleLimitMax = 120#400 #115
ymiddleLimitMin = 110 #60 #113

widthToFilterFor = 450
widthTolerance = 500000#50

normalizationMultiple_mid = 1
normalizationMultiple_width = 1
finalNormalizedLengthMultiple = 1

tooth0Lengths = {}
tooth1Lengths = {}
tooth2Lengths = {}
tooth3Lengths = {}
tooth4Lengths = {}
tooth5Lengths = {}
tooth6Lengths = {}
tooth7Lengths = {}
tooth8Lengths = {}

tooth0Lengths_normalized = {}
tooth1Lengths_normalized = {}
tooth2Lengths_normalized = {}
tooth3Lengths_normalized = {}
tooth4Lengths_normalized = {}
tooth5Lengths_normalized = {}
tooth6Lengths_normalized = {}
tooth7Lengths_normalized = {}
tooth8Lengths_normalized = {}


processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle'] * normalizationMultiple_mid
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] * normalizationMultiple_width
        
        print(yMiddleOfTeethLine / widthTeethLine)

        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth0' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth0'][3] - teethSeperated_resultsDic[key]['tooth0'][2]
                
                tooth0Lengths[key] = actualLength

            
                normalizationFactor = polyNomialFunction_y2WidthRatio(yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth0Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple


            if 'tooth1' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth1'][3] - teethSeperated_resultsDic[key]['tooth1'][2]

                tooth1Lengths[key] = actualLength

                normalizationFactor = polyNomialFunction_y2WidthRatio(yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth1Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple


            if 'tooth2' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth2'][3] - teethSeperated_resultsDic[key]['tooth2'][2]
                
                tooth2Lengths[key] = actualLength

                normalizationFactor = polyNomialFunction_y2WidthRatio(yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth2Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple


            if 'tooth3' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth3'][3] - teethSeperated_resultsDic[key]['tooth3'][2]
                
                tooth3Lengths[key] = actualLength

                normalizationFactor = polyNomialFunction_y2WidthRatio(yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth3Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple


            if 'tooth4' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth4'][3] - teethSeperated_resultsDic[key]['tooth4'][2]
                
                tooth4Lengths[key] = actualLength


                normalizationFactor = polyNomialFunction_y2WidthRatio(yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth4Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            if 'tooth5' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth5'][3] - teethSeperated_resultsDic[key]['tooth5'][2]
                
                tooth5Lengths[key] = actualLength

                normalizationFactor = polyNomialFunction_y2WidthRatio(yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth5Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            if 'tooth6' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth6'][3] - teethSeperated_resultsDic[key]['tooth6'][2]
                
                tooth6Lengths[key] = actualLength

                normalizationFactor = polyNomialFunction_y2WidthRatio(yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth6Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            if 'tooth7' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth7'][3] - teethSeperated_resultsDic[key]['tooth7'][2]
                
                tooth7Lengths[key] = actualLength

                normalizationFactor = polyNomialFunction_y2WidthRatio(yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth7Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            if 'tooth8' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth8'][3] - teethSeperated_resultsDic[key]['tooth8'][2]

                tooth8Lengths[key] = actualLength

                normalizationFactor = polyNomialFunction_y2WidthRatio(yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth8Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            processedFrames+=1



print("processes: " + str(processedFrames) + " frames and Found:\n\n" + str(len(tooth0Lengths)) + '  tooth0Lengths\n' + str(len(tooth1Lengths)) + '  tooth1Lengths\n' + str(len(tooth2Lengths)) + '  tooth2Lengths\n' + str(len(tooth3Lengths)) + '  tooth3Lengths\n' + str(len(tooth4Lengths)) + '  tooth4Lengths\n' + str(len(tooth5Lengths)) + '  tooth5Lengths\n' + str(len(tooth6Lengths)) + '  tooth6Lengths\n' + str(len(tooth7Lengths)) + '  tooth7Lengths\n' + str(len(tooth8Lengths)) + '  tooth8Lengths\n\n\n' + str(len(tooth1Lengths_normalized)) + '  tooth1Lengths_normalized\n'+str(len(tooth1Lengths_normalized)) + '  tooth1Lengths_normalized\n' + str(len(tooth2Lengths_normalized)) + '  tooth2Lengths_normalized\n' + str(len(tooth3Lengths_normalized)) + '  tooth3Lengths_normalized\n' + str(len(tooth4Lengths_normalized)) + '  tooth4Lengths_normalized\n' + str(len(tooth5Lengths_normalized)) + '  tooth5Lengths_normalized\n' + str(len(tooth6Lengths_normalized)) + '  tooth6Lengths_normalized\n' + str(len(tooth7Lengths_normalized)) + '  tooth7Lengths_normalized\n' + str(len(tooth8Lengths_normalized)) + '  tooth8Lengths_normalized\n')

In [ ]:
# Filtering out individual tooth lengths by yCentre and width teethLine, normalizing tooth lengths usingYCentreAndWidthDirectly

imHeight = 480

ymiddleLimitMax = 120 #400 #115
ymiddleLimitMin = 100 #60 #113

widthToFilterFor = 450
widthTolerance = 500000#50

normalizationMultiple_mid = 1
normalizationMultiple_width = 1
finalNormalizedLengthMultiple = 1

tooth0Lengths = {}
tooth1Lengths = {}
tooth2Lengths = {}
tooth3Lengths = {}
tooth4Lengths = {}
tooth5Lengths = {}
tooth6Lengths = {}
tooth7Lengths = {}
tooth8Lengths = {}

tooth0Lengths_normalized = {}
tooth1Lengths_normalized = {}
tooth2Lengths_normalized = {}
tooth3Lengths_normalized = {}
tooth4Lengths_normalized = {}
tooth5Lengths_normalized = {}
tooth6Lengths_normalized = {}
tooth7Lengths_normalized = {}
tooth8Lengths_normalized = {}


processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle'] * normalizationMultiple_mid
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] * normalizationMultiple_width
        
        print(yMiddleOfTeethLine / widthTeethLine)

        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth0' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth0'][3] - teethSeperated_resultsDic[key]['tooth0'][2]
                
                tooth0Lengths[key] = actualLength

            
                normalizationFactor = (yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth0Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple


            if 'tooth1' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth1'][3] - teethSeperated_resultsDic[key]['tooth1'][2]

                tooth1Lengths[key] = actualLength

                normalizationFactor = (yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth1Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple


            if 'tooth2' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth2'][3] - teethSeperated_resultsDic[key]['tooth2'][2]
                
                tooth2Lengths[key] = actualLength

                normalizationFactor = (yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth2Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple


            if 'tooth3' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth3'][3] - teethSeperated_resultsDic[key]['tooth3'][2]
                
                tooth3Lengths[key] = actualLength

                normalizationFactor = (yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth3Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple


            if 'tooth4' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth4'][3] - teethSeperated_resultsDic[key]['tooth4'][2]
                
                tooth4Lengths[key] = actualLength


                normalizationFactor = (yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth4Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            if 'tooth5' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth5'][3] - teethSeperated_resultsDic[key]['tooth5'][2]
                
                tooth5Lengths[key] = actualLength

                normalizationFactor = (yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth5Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            if 'tooth6' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth6'][3] - teethSeperated_resultsDic[key]['tooth6'][2]
                
                tooth6Lengths[key] = actualLength

                normalizationFactor = (yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth6Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            if 'tooth7' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth7'][3] - teethSeperated_resultsDic[key]['tooth7'][2]
                
                tooth7Lengths[key] = actualLength

                normalizationFactor = (yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth7Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            if 'tooth8' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth8'][3] - teethSeperated_resultsDic[key]['tooth8'][2]

                tooth8Lengths[key] = actualLength

                normalizationFactor = (yMiddleOfTeethLine / widthTeethLine)

                if (normalizationFactor > 0):
                    tooth8Lengths_normalized[key] = (normalizationFactor * actualLength) * finalNormalizedLengthMultiple 


            processedFrames+=1



print("processes: " + str(processedFrames) + " frames and Found:\n\n" + str(len(tooth0Lengths)) + '  tooth0Lengths\n' + str(len(tooth1Lengths)) + '  tooth1Lengths\n' + str(len(tooth2Lengths)) + '  tooth2Lengths\n' + str(len(tooth3Lengths)) + '  tooth3Lengths\n' + str(len(tooth4Lengths)) + '  tooth4Lengths\n' + str(len(tooth5Lengths)) + '  tooth5Lengths\n' + str(len(tooth6Lengths)) + '  tooth6Lengths\n' + str(len(tooth7Lengths)) + '  tooth7Lengths\n' + str(len(tooth8Lengths)) + '  tooth8Lengths\n\n\n' + str(len(tooth1Lengths_normalized)) + '  tooth1Lengths_normalized\n'+str(len(tooth1Lengths_normalized)) + '  tooth1Lengths_normalized\n' + str(len(tooth2Lengths_normalized)) + '  tooth2Lengths_normalized\n' + str(len(tooth3Lengths_normalized)) + '  tooth3Lengths_normalized\n' + str(len(tooth4Lengths_normalized)) + '  tooth4Lengths_normalized\n' + str(len(tooth5Lengths_normalized)) + '  tooth5Lengths_normalized\n' + str(len(tooth6Lengths_normalized)) + '  tooth6Lengths_normalized\n' + str(len(tooth7Lengths_normalized)) + '  tooth7Lengths_normalized\n' + str(len(tooth8Lengths_normalized)) + '  tooth8Lengths_normalized\n')

In [ ]:
# copy over the selected frames

destToCopyTo = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/frames_every100_centureOfTeethLineAt113-115_andWidth400-500/'

for fileName in os.listdir(resultsDir):
    if fileName and '.png' in fileName:
        
        fileNameAr = fileName.split('_')
        frameNb = int(fileNameAr[0])
        frameTime = int(fileNameAr[1])
        
        if frameNb in listOfFrameNbs:
            full_file_name = os.path.join(resultsDir, fileName)
            if (os.path.isfile(full_file_name)):
                shutil.copy(full_file_name, destToCopyTo)

### Bining the lengths yCentre and width TeethLine,  and getting statistics

In [ ]:
# V2 seperate teeth lengths based on yCentre and width TeethLine, and record them, into toothLengthsDic

posJumpMin = 7 #for t0:7 ,t3: 7
negJumpMax = 10  #for t0:7 ,t3: 10
movingAvgLength = 1 #for t0:1 ,t3: 1

imHeight = 480

yCentreTeethLine_buckets = []
#for i in range(90,194, 4):
#for i in range(112,120, 4): # 112-116
for i in range(50,500, 4):
    yCentreTeethLine_buckets.append(i)
    

widthTeethLine_buckets = []
#for i in range(300,600, 50):
#for i in range(400,550, 50): #400-500
for i in range(150,700, 50):
    widthTeethLine_buckets.append(i)

    
#teethKeys_toLookFor = ['tooth0','tooth1','tooth2','tooth3','tooth4','tooth5','tooth6','tooth7','tooth8']
teethKeys_toLookFor = ['tooth0','tooth1','tooth2','tooth3','tooth4','tooth5']


    
#setup toothLengthsDic
toothLengthsDic = {}

for toothKey in teethKeys_toLookFor:
    toothLengthsDic[toothKey] = {}
    
    for i in range(len(yCentreTeethLine_buckets)-1):
        yCen = yCentreTeethLine_buckets[i]
        toothLengthsDic[toothKey][yCen] = {}
        
        for j in range(len(widthTeethLine_buckets)-1):
            toothLengthsDic[toothKey][yCen][widthTeethLine_buckets[j]] = {}
            toothLengthsDic[toothKey][yCen][widthTeethLine_buckets[j]]['originalLength'] = []
            toothLengthsDic[toothKey][yCen][widthTeethLine_buckets[j]]['smoothedLength'] = []
            toothLengthsDic[toothKey][yCen][widthTeethLine_buckets[j]]['frameNbs'] = []
            toothLengthsDic[toothKey][yCen][widthTeethLine_buckets[j]]['deque'] = deque()
        

        
        
# Record info
noBucketCount = 0
for frameNb in sorted(teethSeperated_resultsDic):
    
    #1. Find yCentreTeethLine bucket
    curYCen = teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle']
    curYCen_bucket = -1
    for i in range(len(yCentreTeethLine_buckets)-1):
        if curYCen >= yCentreTeethLine_buckets[i] and curYCen < yCentreTeethLine_buckets[i+1]:
            curYCen_bucket = yCentreTeethLine_buckets[i]


    #2. Find widthTeethLine bucket
    curWid =  teethSeperated_resultsDic[frameNb]['teethLine']['width']
    curWid_bucket = -1
    for i in range(len(widthTeethLine_buckets)-1):
        if curWid >= widthTeethLine_buckets[i] and curWid < widthTeethLine_buckets[i+1]:
            curWid_bucket = widthTeethLine_buckets[i]
    
    
    
    if not (curYCen_bucket == -1 or curWid_bucket == -1):
        
        #3. Record original and smoothed tooth lengths
        for toothKey in teethKeys_toLookFor:

            if toothKey in teethSeperated_resultsDic[frameNb]:

                
                
                
                origToothLength = teethSeperated_resultsDic[frameNb][toothKey][3] -teethSeperated_resultsDic[frameNb][toothKey][2]
                
                
                
                smoothedToothLength = getMovingAverage3(
                    toothLengthsDic[toothKey][curYCen_bucket][curWid_bucket]['deque'],
                    origToothLength,
                    movingAvgLength,
                    posJumpMin,
                    negJumpMax)

                
                    
                toothLengthsDic[toothKey][curYCen_bucket][curWid_bucket]['frameNbs'].append(frameNb)
                
                toothLengthsDic[toothKey][curYCen_bucket][curWid_bucket]['originalLength'].append(origToothLength)
                
                toothLengthsDic[toothKey][curYCen_bucket][curWid_bucket]['smoothedLength'].append(smoothedToothLength)
                
                
                
                
    else:
        #print('could not find curYCen_bucket or curWid_bucket for frame: ' + str(frameNb) + '\n\n')
        noBucketCount +=1
            
            
            
            
            
#4. Print results
'''
print('\n\n\n')
for toothKey in toothLengthsDic.keys():
    print('\n\nfor ' + toothKey + ' found')
    
    for yBuck in toothLengthsDic[toothKey].keys():
        print(str(yBuck) + ' :\n')
        for wBuck in toothLengthsDic[toothKey][yBuck]:
            print('       '+str(wBuck) + ': ' + str(len(toothLengthsDic[toothKey][yBuck][wBuck]['originalLength'])) )
            
print('\n\n')
'''
    

print('out of ' + str(len(teethSeperated_resultsDic)) + "  frames in teethSeperated_resultsDic, could not find an appropriate yCent or width teethLine bucket for     " + str(noBucketCount) + ' frames')

In [ ]:
# get statistics on bin distribution

for key in toothLengthsDic.keys():
    print("\n\n")
    print(key)
    
    toothCounter = 0
    for yBuck in toothLengthsDic[key].keys():
        #print('\nyLoc  ' + str(yBuck) + ' :')
        
        yLocCounter = 0
        for wBuck in toothLengthsDic[key][yBuck]:
            if yBuck == 206 or yBuck == 186:
                print('wBuck ' + str(wBuck) + ' :  ' + str(len(toothLengthsDic[key][yBuck][wBuck]['frameNbs'])))
                yLocCounter += len(toothLengthsDic[key][yBuck][wBuck]['frameNbs'])
            
        if yLocCounter > 5:
            print('\nyLoc  ' + str(yBuck) + ' :')
            print('yLoc total:  ' + str(yLocCounter))
        
        toothCounter += yLocCounter
        
    print('\n ****tooth total:  ' + str(toothCounter))


In [ ]:
# calculate ratios for selected bins, here yLoc 186, 206 and Width (500, 450) and (550, 450) respectively

teethKeys_toLookFor = ['tooth0','tooth1','tooth2','tooth3','tooth4','tooth5','tooth6','tooth7','tooth8']


    
#setup toothLengthsDic
calibMeans = {}

for toothKey in teethKeys_toLookFor:
    calibMeans[toothKey] = {}
    calibMeans[toothKey][186] = {}
    calibMeans[toothKey][186][450] = 0
    calibMeans[toothKey][186][500] = 0
    calibMeans[toothKey][206] = {}
    calibMeans[toothKey][206][450] = 0
    calibMeans[toothKey][206][550] = 0
    
    
for toothKey in toothLengthsDic.keys():
    print('for tooth: ' + toothKey + ': ')
    print('[186][450]: ' + str(len(toothLengthsDic[toothKey][186][450]['originalLength'])))
    print('[186][500]: ' + str(len(toothLengthsDic[toothKey][186][500]['originalLength'])))
    print('[206][450]: ' + str(len(toothLengthsDic[toothKey][206][450]['originalLength'])))
    print('[206][550]: ' + str(len(toothLengthsDic[toothKey][206][550]['originalLength'])))
    
    for tl in toothLengthsDic[toothKey][186][450]['originalLength']:
        calibMeans[toothKey][186][450]+= tl
        
    calibMeans[toothKey][186][450] = calibMeans[toothKey][186][450] / len(toothLengthsDic[toothKey][186][450]['originalLength'])
    
        
    for tl in toothLengthsDic[toothKey][186][500]['originalLength']:
        calibMeans[toothKey][186][500]+= tl
        
    calibMeans[toothKey][186][500] = calibMeans[toothKey][186][500] / len(toothLengthsDic[toothKey][186][500]['originalLength'])
    
        
    for tl in toothLengthsDic[toothKey][206][450]['originalLength']:
        calibMeans[toothKey][206][450]+= tl
        
    calibMeans[toothKey][206][450] = calibMeans[toothKey][206][450]/ len(toothLengthsDic[toothKey][206][450]['originalLength'])
    
        
    for tl in toothLengthsDic[toothKey][206][550]['originalLength']:
        calibMeans[toothKey][206][550]+= tl
        
    calibMeans[toothKey][206][550] = calibMeans[toothKey][206][550] / len(toothLengthsDic[toothKey][206][550]['originalLength'])
        

        
    print('\n\n\n')
    
    
    
    
    
for key in calibMeans.keys():
    print(key)
    print(calibMeans[key])
    print('\n')

In [ ]:
# Bin statistics for calibration sequence

In [ ]:
# Bin statistics for every 100frames video

### Plotting out the detections

In [ ]:
# Combining two bins together

key = 'tooth3'
print(key)



print('\n186-450:')
yBuck1 = 186
wBuck1 = 450

yBuck2 = 206
wBuck2 = 450


plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(toothLengthsDic[key][yBuck1][wBuck1]['frameNbs'],
         toothLengthsDic[key][yBuck1][wBuck1]['originalLength'])
plt.plot(toothLengthsDic[key][yBuck1][wBuck1]['frameNbs'],
         toothLengthsDic[key][yBuck1][wBuck1]['originalLength'],'ro')

plt.show()


print('\n206-450:')


plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(toothLengthsDic[key][yBuck2][wBuck2]['frameNbs'],
         toothLengthsDic[key][yBuck2][wBuck2]['originalLength'])
plt.plot(toothLengthsDic[key][yBuck2][wBuck2]['frameNbs'],
         toothLengthsDic[key][yBuck2][wBuck2]['originalLength'],'ro')

plt.show()





print('\ncombined:')

combFrames = []
comLengs = []

for i in range(len(toothLengthsDic[key][yBuck1][wBuck1]['frameNbs'])):
    combFrames.append(toothLengthsDic[key][yBuck1][wBuck1]['frameNbs'][i])
    comLengs.append(toothLengthsDic[key][yBuck1][wBuck1]['originalLength'][i])
    
for i in range(len(toothLengthsDic[key][yBuck2][wBuck2]['frameNbs'])):
    combFrames.append(toothLengthsDic[key][yBuck2][wBuck2]['frameNbs'][i])
    comLengs.append(toothLengthsDic[key][yBuck2][wBuck2]['originalLength'][i])


inx = np.argsort(combFrames)
sorted_combFrames = []
sorted_comLengs = []
for i in range(len(inx)):
    sorted_combFrames.append(combFrames[inx[i]])
    sorted_comLengs.append(comLengs[inx[i]])

plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(sorted_combFrames,
         sorted_comLengs)
plt.plot(sorted_combFrames,
         sorted_comLengs,'ro')

plt.show()







posJumpMin = 10 #for t0:7 ,t3: 7
negJumpMax = 7  #for t0:7 ,t3: 10
movingAvgLength = 1 #for t0:1 ,t3: 1


newDeq = deque()
smoothed_sorted_comLengs = []
for toothLen in sorted_comLengs:
    smoothed_sorted_comLengs.append(getMovingAverage3(
        newDeq,
        toothLen,
        movingAvgLength,
        posJumpMin,
        negJumpMax))  


plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()


print('\ncombined and smoothed:')
plt.plot(sorted_combFrames,
         smoothed_sorted_comLengs)
plt.plot(sorted_combFrames,
         smoothed_sorted_comLengs,'ro')

plt.show()

In [ ]:
# Just yLoc 186, 206 and Width (500, 450) and (550, 450) 

#for key in toothLengthsDic.keys():
key = 'tooth3'
print(key)



print('186-450')
yBuck = 186
wBuck = 450


plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
         toothLengthsDic[key][yBuck][wBuck]['originalLength'])
plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
         toothLengthsDic[key][yBuck][wBuck]['originalLength'],'ro')

plt.show()




print('186-500')
yBuck = 186
wBuck = 500

plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()


plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
         toothLengthsDic[key][yBuck][wBuck]['originalLength'])
plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
         toothLengthsDic[key][yBuck][wBuck]['originalLength'],'ro')

plt.show()




print('206-450')
yBuck = 206
wBuck = 450

plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
         toothLengthsDic[key][yBuck][wBuck]['originalLength'])
plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
         toothLengthsDic[key][yBuck][wBuck]['originalLength'],'ro')

plt.show()




print('206-550')
yBuck = 206
wBuck = 550

plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
         toothLengthsDic[key][yBuck][wBuck]['originalLength'])
plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
         toothLengthsDic[key][yBuck][wBuck]['originalLength'],'ro')

plt.show()

print("\n\n")

In [ ]:
# V2 Plot all teeth, original and smooth

for key in toothLengthsDic.keys():
    print("\n\n")
    print(key)
    
    plt.figure(figsize=(30,10))
    ax = plt.axes()
    loc = plticker.MultipleLocator(base=300000.0)
    ax.xaxis.set_major_locator(loc)
    ax.grid()
    
    for yBuck in toothLengthsDic[key].keys():
        
        for wBuck in toothLengthsDic[key][yBuck]:

            if(len(toothLengthsDic[key][yBuck][wBuck]['frameNbs']) > 0):

                plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
                         toothLengthsDic[key][yBuck][wBuck]['originalLength'])
                plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
                         toothLengthsDic[key][yBuck][wBuck]['originalLength'],'ro')

                plt.plot(toothLengthsDic[key][yBuck][wBuck]['frameNbs'],
                         toothLengthsDic[key][yBuck][wBuck]['smoothedLength'])

    plt.show()

In [ ]:
# V2 Plot just tooth3 (i.e tooth nb 4), original and smooth

plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

for yBuck in toothLengthsDic['tooth3'].keys():
        
    for wBuck in toothLengthsDic['tooth3'][yBuck]:
        
        if(len(toothLengthsDic['tooth3'][yBuck][wBuck]['frameNbs']) > 12):
        
            plt.plot(toothLengthsDic['tooth3'][yBuck][wBuck]['frameNbs'],
                     toothLengthsDic['tooth3'][yBuck][wBuck]['originalLength'])
            plt.plot(toothLengthsDic['tooth3'][yBuck][wBuck]['frameNbs'],
                     toothLengthsDic['tooth3'][yBuck][wBuck]['originalLength'],'ro')
            
            plt.plot(toothLengthsDic['tooth3'][yBuck][wBuck]['frameNbs'],
                     toothLengthsDic['tooth3'][yBuck][wBuck]['smoothedLength'])
        
plt.show()

In [ ]:
# V2 Plot just tooth3, playing around with different smoothness

plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()


posJumpMin = 7 #for t0:7 ,t3: 7
negJumpMax = 10  #for t0:7 ,t3: 10
movingAvgLength = 1 #for t0:1 ,t3: 1


for yBuck in toothLengthsDic['tooth0'].keys():
        
    for wBuck in toothLengthsDic['tooth0'][yBuck]:
        
        if(len(toothLengthsDic['tooth0'][yBuck][wBuck]['frameNbs']) > 30):
        
            plt.plot(toothLengthsDic['tooth0'][yBuck][wBuck]['frameNbs'],
                     toothLengthsDic['tooth0'][yBuck][wBuck]['originalLength'])
            plt.plot(toothLengthsDic['tooth0'][yBuck][wBuck]['frameNbs'],
                     toothLengthsDic['tooth0'][yBuck][wBuck]['originalLength'], 'ro')
            
            toothLengthsDic['tooth0'][yBuck][wBuck]['sm2'] = []
            toothLengthsDic['tooth0'][yBuck][wBuck]['deque'].clear()
            
            for toothLen in toothLengthsDic['tooth0'][yBuck][wBuck]['originalLength']:
                toothLengthsDic['tooth0'][yBuck][wBuck]['sm2'].append(getMovingAverage3(
                    toothLengthsDic['tooth0'][yBuck][wBuck]['deque'],
                    toothLen,
                    movingAvgLength,
                    posJumpMin,
                    negJumpMax))  
                
            
            plt.plot(toothLengthsDic['tooth0'][yBuck][wBuck]['frameNbs'],
                     toothLengthsDic['tooth0'][yBuck][wBuck]['sm2'])
        
plt.show()

In [ ]:
# Plotting individual tooth Lengths Vs. Frame Number just original

tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()


tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()


tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()


tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()

In [ ]:
# Plotting individual tooth Lengths Vs. Frame Number just nornmalized

tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth0Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth0Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth1Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth1Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth2Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth2Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth3Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth3Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth4Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth4Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth5Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth5Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth6Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth6Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth7Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth7Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth8Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth8Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()

In [ ]:
# Plotting individual tooth Lengths Vs. Frame Number both nornmalized and original

tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth0Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth0Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth1Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth1Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth2Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth2Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth3Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth3Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth4Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth4Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth5Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth5Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth6Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth6Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth7Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth7Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth8Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth8Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)


plt.show()

In [ ]:
# Plotting individual tooth Lengths_113_115 Vs. Frame Number just original, both 112-114 and 113-115

tempLengths_113_115 = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths_113_115):
    tempFrameNbs.append(frameNb)
    tempLengths_113_115.append(tooth0Lengths_113_115[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths_113_115)

tempLengths_112_114 = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths_112_114):
    tempFrameNbs.append(frameNb)
    tempLengths_112_114.append(tooth0Lengths_112_114[frameNb])
    
plt.plot(tempFrameNbs, tempLengths_112_114)


plt.show()






tempLengths_113_115 = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths_113_115):
    tempFrameNbs.append(frameNb)
    tempLengths_113_115.append(tooth1Lengths_113_115[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths_113_115)

tempLengths_112_114 = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths_112_114):
    tempFrameNbs.append(frameNb)
    tempLengths_112_114.append(tooth1Lengths_112_114[frameNb])
    
plt.plot(tempFrameNbs, tempLengths_112_114)

plt.show()





tempLengths_113_115 = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths_113_115):
    tempFrameNbs.append(frameNb)
    tempLengths_113_115.append(tooth2Lengths_113_115[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths_113_115)

tempLengths_112_114 = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths_112_114):
    tempFrameNbs.append(frameNb)
    tempLengths_112_114.append(tooth2Lengths_112_114[frameNb])
    
plt.plot(tempFrameNbs, tempLengths_112_114)

plt.show()





tempLengths_113_115 = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths_113_115):
    tempFrameNbs.append(frameNb)
    tempLengths_113_115.append(tooth3Lengths_113_115[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths_113_115)

tempLengths_112_114 = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths_112_114):
    tempFrameNbs.append(frameNb)
    tempLengths_112_114.append(tooth3Lengths_112_114[frameNb])

plt.plot(tempFrameNbs, tempLengths_112_114)

plt.show()






tempLengths_113_115 = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths_113_115):
    tempFrameNbs.append(frameNb)
    tempLengths_113_115.append(tooth4Lengths_113_115[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths_113_115)

tempLengths_112_114 = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths_112_114):
    tempFrameNbs.append(frameNb)
    tempLengths_112_114.append(tooth4Lengths_112_114[frameNb])

plt.plot(tempFrameNbs, tempLengths_112_114)

plt.show()






tempLengths_113_115 = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths_113_115):
    tempFrameNbs.append(frameNb)
    tempLengths_113_115.append(tooth5Lengths_113_115[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths_113_115)

tempLengths_112_114 = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths_112_114):
    tempFrameNbs.append(frameNb)
    tempLengths_112_114.append(tooth5Lengths_112_114[frameNb])

plt.plot(tempFrameNbs, tempLengths_112_114)

plt.show()






tempLengths_113_115 = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths_113_115):
    tempFrameNbs.append(frameNb)
    tempLengths_113_115.append(tooth6Lengths_113_115[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths_113_115)

tempLengths_112_114 = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths_112_114):
    tempFrameNbs.append(frameNb)
    tempLengths_112_114.append(tooth6Lengths_112_114[frameNb])

plt.plot(tempFrameNbs, tempLengths_112_114)

plt.show()






tempLengths_113_115 = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths_113_115):
    tempFrameNbs.append(frameNb)
    tempLengths_113_115.append(tooth7Lengths_113_115[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths_113_115)

tempLengths_112_114 = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths_112_114):
    tempFrameNbs.append(frameNb)
    tempLengths_112_114.append(tooth7Lengths_112_114[frameNb])

plt.plot(tempFrameNbs, tempLengths_112_114)

plt.show()



tempLengths_113_115 = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths_113_115):
    tempFrameNbs.append(frameNb)
    tempLengths_113_115.append(tooth8Lengths_113_115[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths_113_115)

tempLengths_112_114 = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths_112_114):
    tempFrameNbs.append(frameNb)
    tempLengths_112_114.append(tooth8Lengths_112_114[frameNb])

plt.plot(tempFrameNbs, tempLengths_112_114)

plt.show()


## <font color='blue'>  Finding the calibration function 

In [ ]:
#Finding average tooth lengths and correspondng teethLineWidth or Y of middle with/out recovered
imHeight = 480
imWidth = 720

averageToothLength = []
midTeethLineY = []
teethLineWidths = []
teethLineY2WidthRatio = []


for key in sorted(teethSeperated_resultsDic):
    minY = 1000
    maxY = -1
    toothLengths = []
    
    if not ('tooth0' in teethSeperated_resultsDic[key] or 'tooth1' in teethSeperated_resultsDic[key] or 'tooth2' in teethSeperated_resultsDic[key] or 'tooth3' in teethSeperated_resultsDic[key] or 'tooth4' in teethSeperated_resultsDic[key] or 'tooth5' in teethSeperated_resultsDic[key] or 'tooth6' in teethSeperated_resultsDic[key] or 'tooth7' in teethSeperated_resultsDic[key] or 'tooth8' in teethSeperated_resultsDic[key]):
        print("no teeth for image:  " + str(key) + "\n")
    
    
    else:

        
        if 'tooth0' in teethSeperated_resultsDic[key]:
            toothLengths.append( (teethSeperated_resultsDic[key]['tooth0'][3] - teethSeperated_resultsDic[key]['tooth0'][2]) )

            if  teethSeperated_resultsDic[key]['tooth0'][3] > maxY:
                maxY = teethSeperated_resultsDic[key]['tooth0'][3]
            if teethSeperated_resultsDic[key]['tooth0'][2] < minY:
                minY = teethSeperated_resultsDic[key]['tooth0'][2]


        if 'tooth1' in teethSeperated_resultsDic[key]:
            toothLengths.append( (teethSeperated_resultsDic[key]['tooth1'][3] - teethSeperated_resultsDic[key]['tooth1'][2]) )

            if  teethSeperated_resultsDic[key]['tooth1'][3] > maxY:
                maxY = teethSeperated_resultsDic[key]['tooth1'][3]
            if teethSeperated_resultsDic[key]['tooth1'][2] < minY:
                minY = teethSeperated_resultsDic[key]['tooth1'][2]


        if 'tooth2' in teethSeperated_resultsDic[key]:
            toothLengths.append( (teethSeperated_resultsDic[key]['tooth2'][3] - teethSeperated_resultsDic[key]['tooth2'][2]) )

            if  teethSeperated_resultsDic[key]['tooth2'][3] > maxY:
                maxY = teethSeperated_resultsDic[key]['tooth2'][3]
            if teethSeperated_resultsDic[key]['tooth2'][2] < minY:
                minY = teethSeperated_resultsDic[key]['tooth2'][2]


        if 'tooth3' in teethSeperated_resultsDic[key]:
            toothLengths.append( (teethSeperated_resultsDic[key]['tooth3'][3] - teethSeperated_resultsDic[key]['tooth3'][2]) )

            if  teethSeperated_resultsDic[key]['tooth3'][3] > maxY:
                maxY = teethSeperated_resultsDic[key]['tooth3'][3]
            if teethSeperated_resultsDic[key]['tooth3'][2] < minY:
                minY = teethSeperated_resultsDic[key]['tooth3'][2]


        if 'tooth4' in teethSeperated_resultsDic[key]:
            toothLengths.append( (teethSeperated_resultsDic[key]['tooth4'][3] - teethSeperated_resultsDic[key]['tooth4'][2]) )

            if  teethSeperated_resultsDic[key]['tooth4'][3] > maxY:
                maxY = teethSeperated_resultsDic[key]['tooth4'][3]
            if teethSeperated_resultsDic[key]['tooth4'][2] < minY:
                minY = teethSeperated_resultsDic[key]['tooth4'][2]


        if 'tooth5' in teethSeperated_resultsDic[key]:
            toothLengths.append( (teethSeperated_resultsDic[key]['tooth5'][3] - teethSeperated_resultsDic[key]['tooth5'][2]) )

            if  teethSeperated_resultsDic[key]['tooth5'][3] > maxY:
                maxY = teethSeperated_resultsDic[key]['tooth5'][3]
            if teethSeperated_resultsDic[key]['tooth5'][2] < minY:
                minY = teethSeperated_resultsDic[key]['tooth5'][2]


        if 'tooth6' in teethSeperated_resultsDic[key]:
            toothLengths.append( (teethSeperated_resultsDic[key]['tooth6'][3] - teethSeperated_resultsDic[key]['tooth6'][2]) )

            if  teethSeperated_resultsDic[key]['tooth6'][3] > maxY:
                maxY = teethSeperated_resultsDic[key]['tooth6'][3]
            if teethSeperated_resultsDic[key]['tooth6'][2] < minY:
                minY = teethSeperated_resultsDic[key]['tooth6'][2]


        if 'tooth7' in teethSeperated_resultsDic[key]:
            toothLengths.append( (teethSeperated_resultsDic[key]['tooth7'][3] - teethSeperated_resultsDic[key]['tooth7'][2]) )

            if  teethSeperated_resultsDic[key]['tooth7'][3] > maxY:
                maxY = teethSeperated_resultsDic[key]['tooth7'][3]
            if teethSeperated_resultsDic[key]['tooth7'][2] < minY:
                minY = teethSeperated_resultsDic[key]['tooth7'][2]


        if 'tooth8' in teethSeperated_resultsDic[key]:
            toothLengths.append( (teethSeperated_resultsDic[key]['tooth8'][3] - teethSeperated_resultsDic[key]['tooth8'][2]) )

            if  teethSeperated_resultsDic[key]['tooth8'][3] > maxY:
                maxY = teethSeperated_resultsDic[key]['tooth8'][3]
            if teethSeperated_resultsDic[key]['tooth8'][2] < minY:
                minY = teethSeperated_resultsDic[key]['tooth8'][2]


        if 'teethLine' in teethSeperated_resultsDic[key] and 'box' in teethSeperated_resultsDic[key]['teethLine']:
            xmin, xmax, ymin, ymax = teethSeperated_resultsDic[key]['teethLine']['box']

            yminConv = ymin * imHeight
            ymaxConv = ymax * imHeight
            
            xminConv = xmin * imWidth
            xmaxConv = xmax * imWidth

            teethLineHeight = ymaxConv - yminConv

            midTeethLineY.append( (yminConv + teethLineHeight/2) )
            
            teethLineWidths.append( (xmaxConv - xminConv) )
            
            teethLineY2WidthRatio.append((yminConv + teethLineHeight/2) / (xmaxConv - xminConv))
        #else:
        #    teethLineHeight = maxY - minY

        #    midTeethLineY.append( (minY + teethLineHeight/2) )


            if len(toothLengths) > 0:
                sumToothLengths = 0
                for toothLen in toothLengths:
                    sumToothLengths = sumToothLengths + toothLen

                averageToothLength.append(sumToothLengths/ len(toothLengths))

print("found: " + str(len(midTeethLineY)) + "   teethLines and  " + str(len(averageToothLength)) + "   average teeth lengths")

In [ ]:
# Loading individual tooth lengths into dict keyed by frame without any filtering or normalization

imHeight = 480


tooth0Lengths = {}
tooth1Lengths = {}
tooth2Lengths = {}
tooth3Lengths = {}
tooth4Lengths = {}
tooth5Lengths = {}
tooth6Lengths = {}
tooth7Lengths = {}
tooth8Lengths = {}

listOfFrameNbs = []


processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width']


            
        
        if 'tooth0' in teethSeperated_resultsDic[key]:
            actualLength = teethSeperated_resultsDic[key]['tooth0'][3] - teethSeperated_resultsDic[key]['tooth0'][2]

            tooth0Lengths[key] = actualLength





        if 'tooth1' in teethSeperated_resultsDic[key]:
            actualLength = teethSeperated_resultsDic[key]['tooth1'][3] - teethSeperated_resultsDic[key]['tooth1'][2]

            tooth1Lengths[key] = actualLength





        if 'tooth2' in teethSeperated_resultsDic[key]:
            actualLength = teethSeperated_resultsDic[key]['tooth2'][3] - teethSeperated_resultsDic[key]['tooth2'][2]

            tooth2Lengths[key] = actualLength




        if 'tooth3' in teethSeperated_resultsDic[key]:
            actualLength = teethSeperated_resultsDic[key]['tooth3'][3] - teethSeperated_resultsDic[key]['tooth3'][2]

            tooth3Lengths[key] = actualLength




        if 'tooth4' in teethSeperated_resultsDic[key]:
            actualLength = teethSeperated_resultsDic[key]['tooth4'][3] - teethSeperated_resultsDic[key]['tooth4'][2]

            tooth4Lengths[key] = actualLength



        if 'tooth5' in teethSeperated_resultsDic[key]:
            actualLength = teethSeperated_resultsDic[key]['tooth5'][3] - teethSeperated_resultsDic[key]['tooth5'][2]

            tooth5Lengths[key] = actualLength




        if 'tooth6' in teethSeperated_resultsDic[key]:
            actualLength = teethSeperated_resultsDic[key]['tooth6'][3] - teethSeperated_resultsDic[key]['tooth6'][2]

            tooth6Lengths[key] = actualLength




        if 'tooth7' in teethSeperated_resultsDic[key]:
            actualLength = teethSeperated_resultsDic[key]['tooth7'][3] - teethSeperated_resultsDic[key]['tooth7'][2]

            tooth7Lengths[key] = actualLength




        if 'tooth8' in teethSeperated_resultsDic[key]:
            actualLength = teethSeperated_resultsDic[key]['tooth8'][3] - teethSeperated_resultsDic[key]['tooth8'][2]

            tooth8Lengths[key] = actualLength





        processedFrames+=1
        listOfFrameNbs.append(key)



print("processes: " + str(processedFrames) + " frames and Found:\n\n" + str(len(tooth0Lengths)) + '  tooth0Lengths\n' + str(len(tooth1Lengths)) + '  tooth1Lengths\n' + str(len(tooth2Lengths)) + '  tooth2Lengths\n' + str(len(tooth3Lengths)) + '  tooth3Lengths\n' + str(len(tooth4Lengths)) + '  tooth4Lengths\n' + str(len(tooth5Lengths)) + '  tooth5Lengths\n' + str(len(tooth6Lengths)) + '  tooth6Lengths\n' + str(len(tooth7Lengths)) + '  tooth7Lengths\n' + str(len(tooth8Lengths)) + '  tooth8Lengths\n\n\n')

### Plotting average and indidual teeth lengths VS. y of centure of teethLine

In [ ]:
#plotting average tooth lengths vs midPoint y of teethline

plt.plot(midTeethLineY, averageToothLength, 'ro')
plt.ylabel('average ToothLength In Frame')
plt.xlabel('y location of middle of teethLine')
plt.show()

In [ ]:
# Plotting individual tooth Lengths Vs. midPoint y of teethline

tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth0Lengths):
    if 'yOfMiddle' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'])
        tempLengths.append(tooth0Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth1Lengths):
    if 'yOfMiddle' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'])
        tempLengths.append(tooth1Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth2Lengths):
    if 'yOfMiddle' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'])
        tempLengths.append(tooth2Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth3Lengths):
    if 'yOfMiddle' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'])
        tempLengths.append(tooth3Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth4Lengths):
    if 'yOfMiddle' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'])
        tempLengths.append(tooth4Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth5Lengths):
    if 'yOfMiddle' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'])
        tempLengths.append(tooth5Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth6Lengths):
    if 'yOfMiddle' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'])
        tempLengths.append(tooth6Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth7Lengths):
    if 'yOfMiddle' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'])
        tempLengths.append(tooth7Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth8Lengths):
    if 'yOfMiddle' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'])
        tempLengths.append(tooth8Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()

### Plotting average and indidual teeth lengths VS. width of teethLine

In [ ]:
#plotting average tooth lengths vs teethline width

plt.plot(teethLineWidths, averageToothLength, 'ro')
plt.ylabel('average ToothLength In Frame')
plt.xlabel('width of teethLine')
plt.show()

In [ ]:
# Plotting individual tooth Lengths Vs. teethline width

tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth0Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth0Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth1Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth1Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth2Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth2Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth3Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth3Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth4Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth4Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth5Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth5Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth6Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth6Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth7Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth7Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()



tempLengths = []
tempMiddleOfToothLine = []

for frameNb in sorted(tooth8Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempMiddleOfToothLine.append(teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth8Lengths[frameNb])
    
plt.plot(tempMiddleOfToothLine, tempLengths, 'ro')
plt.show()

### Plotting average and indidual teeth lengths VS. (y of centre of teethLine) / (width of teethLine)

In [ ]:
# Plotting average teeth lengths VS. (y of centre of teethLine) / (width of teethLine)

plt.plot(teethLineY2WidthRatio, averageToothLength, 'ro')
plt.ylabel('average ToothLength In Frame')
plt.xlabel('teethLineY2WidthRatio')
plt.show()

In [ ]:
# Plotting indidual teeth lengths VS. (y of centre of teethLine) / (width of teethLine)

tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth0Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth0Lengths[frameNb])
    
plt.plot(tempCentreToWidthRatio, tempLengths, 'ro')
plt.show()



tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth1Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth1Lengths[frameNb])
    
plt.plot(tempCentreToWidthRatio, tempLengths, 'ro')
plt.show()



tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth2Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth2Lengths[frameNb])
    
plt.plot(tempCentreToWidthRatio, tempLengths, 'ro')
plt.show()



tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth3Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth3Lengths[frameNb])
    
plt.plot(tempCentreToWidthRatio, tempLengths, 'ro')
plt.show()



tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth4Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth4Lengths[frameNb])
    
plt.plot(tempCentreToWidthRatio, tempLengths, 'ro')
plt.show()



tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth5Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth5Lengths[frameNb])
    
plt.plot(tempCentreToWidthRatio, tempLengths, 'ro')
plt.show()



tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth6Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth6Lengths[frameNb])
    
plt.plot(tempCentreToWidthRatio, tempLengths, 'ro')
plt.show()



tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth7Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth7Lengths[frameNb])
    
plt.plot(tempCentreToWidthRatio, tempLengths, 'ro')
plt.show()



tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth8Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth8Lengths[frameNb])
    
plt.plot(tempCentreToWidthRatio, tempLengths, 'ro')
plt.show()

In [ ]:
# Plotting CORRECTED average teeth lengths VS. (y of centre of teethLine) / (width of teethLine)

tempLengths = []
tempCentreToWidthRatio = []

for frameNb in sorted(tooth3Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        tempCentreToWidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        tempLengths.append(tooth3Lengths[frameNb])

        


correctedAverageToothLengths = []

for ratio, length in zip(tempCentreToWidthRatio, tempLengths):
    correctedAverageToothLengths.append(length + polyNomialFunction_y2WidthRatio(ratio))

plt.plot(tempCentreToWidthRatio, correctedAverageToothLengths, 'ro')
plt.ylabel('correctedTooth3Lengths')
plt.xlabel('teethLineY2WidthRatio')
plt.show()

### Plotting max/current average toothLengtsh Vs. Y of centre, width of teethline, and y to width ratio

In [ ]:
# Plotting normalized toothLength (for average lengths) (using largest) with sorting VS. Middle of TeethLine y

indexOfHighestFrame = np.argmax(averageToothLength)

normalizezdAverageToothLengths_midTeethLine = []
for tl in averageToothLength:
    normalizezdAverageToothLengths_midTeethLine.append(averageToothLength[indexOfHighestFrame] / tl)


inx = np.argsort(midTeethLineY)
sorted_midTeethLineY = []
sorted_normalizezdAverageToothLength_midTeethLine = []
for i in range(len(inx)):
    sorted_midTeethLineY.append(midTeethLineY[inx[i]])
    sorted_normalizezdAverageToothLength_midTeethLine.append(normalizezdAverageToothLengths_midTeethLine[inx[i]])
    
    
plt.plot(sorted_midTeethLineY, sorted_normalizezdAverageToothLength_midTeethLine)
plt.ylabel('ratio of biggest average tooth length to average tooth length')
plt.xlabel('y location of middle of teethLine')
plt.show()

In [ ]:
# Plotting normalized toothLength (for average lengths) (using largest) with sorting VS. width of TeethLine

indexOfHighestFrame = np.argmax(averageToothLength)

normalizezdAverageToothLengths = []
for tl in averageToothLength:
    normalizezdAverageToothLengths.append(averageToothLength[indexOfHighestFrame] / tl)


inx = np.argsort(teethLineWidths)
sorted_teethLineWidths = []
sorted_normalizezdAverageToothLength = []
for i in range(len(inx)):
    sorted_teethLineWidths.append(teethLineWidths[inx[i]])
    sorted_normalizezdAverageToothLength.append(normalizezdAverageToothLengths[inx[i]])
    
    
plt.plot(sorted_teethLineWidths, sorted_normalizezdAverageToothLength)
plt.ylabel('ratio of biggest average tooth length to average tooth length')
plt.xlabel('width of teethLine')
plt.show()

In [ ]:
# Plotting normalized toothLength (for average lengths) (using largest) with sorting VS. y of teetthLine / width

indexOfHighestFrame = np.argmax(averageToothLength)

normalizezdAverageToothLengths = []
for tl in averageToothLength:
    normalizezdAverageToothLengths.append(averageToothLength[indexOfHighestFrame] / tl)


inx = np.argsort(teethLineY2WidthRatio)
sorted_teethLineY2WidthRatio = []
sorted_normalizezdAverageToothLength = []
for i in range(len(inx)):
    sorted_teethLineY2WidthRatio.append(teethLineY2WidthRatio[inx[i]])
    sorted_normalizezdAverageToothLength.append(normalizezdAverageToothLengths[inx[i]])
    

plt.figure(figsize=(30,10))
ax = plt.axes()
ax.grid()
plt.plot(sorted_teethLineY2WidthRatio, sorted_normalizezdAverageToothLength)
plt.ylabel('ratio of biggest average tooth length to average tooth length')
plt.xlabel('teethLineY2WidthRatio')
plt.show()

In [ ]:
# Plotting normalized toothLength for tooth4 (using largest) with sorting VS. y of teetthLine / width

averageToothLength = []
teethLineY2WidthRatio = []

for frameNb in sorted(tooth3Lengths):
    if 'width' in teethSeperated_resultsDic[frameNb]['teethLine']:
        teethLineY2WidthRatio.append(teethSeperated_resultsDic[frameNb]['teethLine']['yOfMiddle'] / teethSeperated_resultsDic[frameNb]['teethLine']['width'])
        averageToothLength.append(tooth3Lengths[frameNb])


indexOfHighestFrame = np.argmax(averageToothLength)

normalizezdAverageToothLengths = []
for tl in averageToothLength:
    normalizezdAverageToothLengths.append(averageToothLength[indexOfHighestFrame] / tl)


inx = np.argsort(teethLineY2WidthRatio)
sorted_teethLineY2WidthRatio = []
sorted_normalizezdtooth3Lengths = []
for i in range(len(inx)):
    sorted_teethLineY2WidthRatio.append(teethLineY2WidthRatio[inx[i]])
    sorted_normalizezdtooth3Lengths.append(normalizezdAverageToothLengths[inx[i]])
    
    
    
plt.figure(figsize=(30,10))
ax = plt.axes()
ax.grid()

plt.plot(sorted_teethLineY2WidthRatio, sorted_normalizezdtooth3Lengths)
plt.ylabel('ratio of biggest average tooth length to average tooth length')
plt.xlabel('teethLineY2WidthRatio')
plt.show()

In [ ]:
# Plotting CORRECTED normalized toothLength (for average lengths) (using largest) with sorting VS. y of teetthLine / width

indexOfHighestFrame = np.argmax(correctedAverageToothLengths)

normalizezdAverageToothLengths = []
for tl in correctedAverageToothLengths:
    normalizezdAverageToothLengths.append(correctedAverageToothLengths[indexOfHighestFrame] / tl)


inx = np.argsort(teethLineY2WidthRatio)
sorted_teethLineY2WidthRatio = []
sorted_normalizezdAverageToothLength = []
for i in range(len(inx)):
    sorted_teethLineY2WidthRatio.append(teethLineY2WidthRatio[inx[i]])
    sorted_normalizezdAverageToothLength.append(normalizezdAverageToothLengths[inx[i]])
    

plt.figure(figsize=(30,10))
ax = plt.axes()
ax.grid()
plt.plot(sorted_teethLineY2WidthRatio, sorted_normalizezdAverageToothLength)
plt.ylabel('ratio of biggest average tooth length to average tooth length')
plt.xlabel('teethLineY2WidthRatio')
plt.show()

### Plotting the distance btw max/current average toothLengtsh Vs. y to width ratio

In [ ]:
# Plotting distance btw max and current average tooth lengths with sorting VS. y of teetthLine / width

indexOfHighestFrame = np.argmax(averageToothLength)

normalizezdAverageToothLengths = []
for tl in averageToothLength:
    normalizezdAverageToothLengths.append(averageToothLength[indexOfHighestFrame] - tl)


inx = np.argsort(teethLineY2WidthRatio)
sorted_teethLineY2WidthRatio = []
sorted_normalizezdAverageToothLength = []
for i in range(len(inx)):
    sorted_teethLineY2WidthRatio.append(teethLineY2WidthRatio[inx[i]])
    sorted_normalizezdAverageToothLength.append(normalizezdAverageToothLengths[inx[i]])
    

plt.figure(figsize=(30,10))
ax = plt.axes()
ax.grid()
plt.plot(sorted_teethLineY2WidthRatio, sorted_normalizezdAverageToothLength)
plt.ylabel('distance btw biggest and current average tooth lengths')
plt.xlabel('teethLineY2WidthRatio')
plt.show()

### Fitting savgol_filter polynomials to data

In [ ]:
#(average of lengths) 1 deg sorted using savgol_filter VS.  Middle of TeethLine y

x_midTeethLine = np.array(sorted_midTeethLineY)
y_midTeethLine = np.array(sorted_normalizezdAverageToothLength_midTeethLine)


from scipy.signal import savgol_filter

yhat_midTeethLine = savgol_filter(y_midTeethLine, 51, 2) # window size 51, polynomial order 4

plt.plot(x_midTeethLine,y_midTeethLine)
plt.plot(x_midTeethLine,yhat_midTeethLine, color='red')
plt.show()

In [ ]:
#(average of lengths) 1 deg sorted using savgol_filter VS. width of TeethLine

x_widthTeethLine = np.array(sorted_teethLineWidths)
y_widthTeethLine = np.array(sorted_normalizezdAverageToothLength)


from scipy.signal import savgol_filter

yhat_widthTeethLine = savgol_filter(y_widthTeethLine, 51, 2) # window size 51, polynomial order 4

plt.plot(x_widthTeethLine,y_widthTeethLine)
plt.plot(x_widthTeethLine,yhat_widthTeethLine, color='red')
plt.show()

In [ ]:
#(average of lengths) 1 deg sorted using savgol_filter VS.  (y of centure of teethLine) / width

x_y2WidthRatio_average = np.array(sorted_teethLineY2WidthRatio)
y_y2WidthRatio_average = np.array(sorted_normalizezdAverageToothLength)


from scipy.signal import savgol_filter

yhat_y2WidthRatio_average = savgol_filter(y_y2WidthRatio_average, 51, 2) # window size 51, polynomial order 4

plt.plot(x_y2WidthRatio_average,y_y2WidthRatio_average)
plt.plot(x_y2WidthRatio_average,yhat_y2WidthRatio_average, color='red')
plt.show()

In [ ]:
#(tooth4 lengths) 1 deg sorted using savgol_filter VS.  (y of centure of teethLine) / width

x_y2WidthRatio_tooth3 = np.array(sorted_teethLineY2WidthRatio)
y_y2WidthRatio_tooth3 = np.array(sorted_normalizezdtooth3Lengths)


from scipy.signal import savgol_filter

yhat_y2WidthRatio_tooth3 = savgol_filter(y_y2WidthRatio_tooth3, 51, 2) # window size 51, polynomial order 4

plt.plot(x_y2WidthRatio_tooth3,y_y2WidthRatio_tooth3)
plt.plot(x_y2WidthRatio_tooth3,yhat_y2WidthRatio_tooth3, color='red')
plt.show()

In [ ]:
#( CORRECTED tooth4 lengths) 1 deg sorted using savgol_filter VS.  (y of centure of teethLine) / width

x_y2WidthRatio_tooth3 = np.array(sorted_teethLineY2WidthRatio)
y_y2WidthRatio_tooth3 = np.array(sorted_normalizezdAverageToothLength)


from scipy.signal import savgol_filter

yhat_y2WidthRatio_tooth3 = savgol_filter(y_y2WidthRatio_tooth3, 51, 2) # window size 51, polynomial order 4

plt.plot(x_y2WidthRatio_tooth3,y_y2WidthRatio_tooth3)
plt.plot(x_y2WidthRatio_tooth3,yhat_y2WidthRatio_tooth3, color='red')
plt.show()


### Fitting pyFit polynomials to data

In [ ]:
#(average of lengths) fitting to the sorted x and y using pyFit Middle of TeethLine y

fittedWeights_midTeethLine = np.polyfit(x_midTeethLine, y_midTeethLine, 2)

polyNomialFunction_midTeethLine = np.poly1d(fittedWeights_midTeethLine) 

plt.plot(polyNomialFunction_midTeethLine(x_midTeethLine))

In [ ]:
#(average of lengths) fitting to the sorted x and y using pyFit width of TeethLine

fittedWeights_widthTeethLine = np.polyfit(x_widthTeethLine, y_widthTeethLine, 2)

polyNomialFunction_widthTeethLine = np.poly1d(fittedWeights_widthTeethLine) 

plt.plot(polyNomialFunction_widthTeethLine(x_widthTeethLine))

In [ ]:
#(average of lengths) fitting to the sorted x and y using pyFit (y of centre of teethline) / (width of teethLine) 

fittedWeights_widthTeethLine = np.polyfit(x_y2WidthRatio_average, y_y2WidthRatio_average, 2)

polyNomialFunction_y2WidthRatio = np.poly1d(fittedWeights_widthTeethLine) 

plt.plot(polyNomialFunction_y2WidthRatio(x_y2WidthRatio_average))

In [ ]:
#(tooth4 lengths) fitting to the sorted x and y using pyFit (y of centre of teethline) / (width of teethLine) 

fittedWeights_widthTeethLine = np.polyfit(x_y2WidthRatio_tooth3, y_y2WidthRatio_tooth3, 2)

polyNomialFunction_widthTeethLine = np.poly1d(fittedWeights_widthTeethLine) 

plt.plot(polyNomialFunction_widthTeethLine(x_y2WidthRatio_tooth3))

### Results to keep

In [ ]:
# Statistics for the calibratinSequenceALL   any bin > 5

### Saving and Loading weights

In [ ]:
# saving the fitted weights

locationToSaveTheWights = '/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/'

np.save(locationToSaveTheWights + 'fittedWeights_Seq1_7Deg', fittedWeights)

In [ ]:
# loading fitted weights
fittedWeights = np.load('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/avg_fittedWeights_3rdDeg.npy')

polyNomialFunction = np.poly1d(fittedWeights) 

### Combining several normalizationCurves

In [ ]:
# loading fitted weights

fittedWeights1 = np.load('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/fittedWeightsSeq1.npy')


fittedWeights2 = np.load('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/fittedWeightsSeq1.npy')


fittedWeights3 = np.load('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/fittedWeightsSeq1.npy')


fittedWeights4 = np.load('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/fittedWeightsSeq1.npy')


fittedWeights5 = np.load('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/fittedWeightsSeq1.npy')


fittedWeights6 = np.load('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/fittedWeightsSeq1.npy')

fittedWeights = fittedWeights1 + fittedWeights2 + fittedWeights3 + fittedWeights4 + fittedWeights5 + fittedWeights6 

fittedWeights = fittedWeights/6

np.save('/media/hooman/961293e3-04a5-40c5-afc0-2b205d0a7067/WM_PROJECT--thermal_Bucyrus--try6--trainingLogs--results--backups/calibSequence/' + 'avg_fittedWeights', fittedWeights)

In [ ]:
# putting the weights in polynomial

polyNomialFunction = np.poly1d(fittedWeights) 

plt.plot(polyNomialFunction(midTeethLineY))

## <font color='blue'>  Results Archive

### <font color='red'>RESULTS FOR WMDL LOGS FROM MAHDI

####  wmdlLogsFromMahdi_Kearl

In [ ]:
#MULTIPLE Final Tooth 1, -- 20,3

imHeight = 480

posJumpMin = 20 #14
negJumpMax = 3 #4
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth0Lengths = {}
tooth0Lengths_normalized = {}
tooth0Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth0Lengths[resultKey] = {}
    tooth0Lengths_normalized[resultKey] = {}
    tooth0Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth0' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth0'][3] - teethSeperated_resultsDic[resultKey][key]['tooth0'][2]

                    tooth0Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth0Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth0Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth0Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth0Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth0Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth0Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth0Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth0Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
#MULTIPLE Final Tooth 2, -- 17,2,1

imHeight = 480

posJumpMin = 17 #20 #30
negJumpMax = 2 #4  #3
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth1Lengths = {}
tooth1Lengths_normalized = {}
tooth1Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth1Lengths[resultKey] = {}
    tooth1Lengths_normalized[resultKey] = {}
    tooth1Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth1' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth1'][3] - teethSeperated_resultsDic[resultKey][key]['tooth1'][2]

                    tooth1Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth1Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth1Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth1Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth1Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth1Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth1Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth1Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth1Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
#MULTIPLE Final Tooth 3 -- 20,3

imHeight = 480

posJumpMin = 20  #20#15
negJumpMax = 3  #4#3
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth2Lengths = {}
tooth2Lengths_normalized = {}
tooth2Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth2Lengths[resultKey] = {}
    tooth2Lengths_normalized[resultKey] = {}
    tooth2Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth2' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth2'][3] - teethSeperated_resultsDic[resultKey][key]['tooth2'][2]

                    tooth2Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth2Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth2Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth2Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth2Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth2Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth2Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth2Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth2Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
#MULTIPLE Final Tooth 4 -- 23,5,2

imHeight = 480

posJumpMin = 23  #20 
negJumpMax = 5   #4
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth3Lengths = {}
tooth3Lengths_normalized = {}
tooth3Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth3Lengths[resultKey] = {}
    tooth3Lengths_normalized[resultKey] = {}
    tooth3Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth3' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth3'][3] - teethSeperated_resultsDic[resultKey][key]['tooth3'][2]

                    tooth3Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth3Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth3Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth3Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth3Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth3Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth3Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth3Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth3Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
#MULTIPLE Final Tooth 5 -- 21,4,2

imHeight = 480

posJumpMin = 21#20 for 2ndDec #18
negJumpMax = 4 #7  for 2ndDec #5
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth4Lengths = {}
tooth4Lengths_normalized = {}
tooth4Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth4Lengths[resultKey] = {}
    tooth4Lengths_normalized[resultKey] = {}
    tooth4Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth4' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth4'][3] - teethSeperated_resultsDic[resultKey][key]['tooth4'][2]

                    tooth4Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth4Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth4Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth4Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth4Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth4Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth4Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth4Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth4Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
#MULTIPLE Final Tooth 6 -- 20,4

imHeight = 480

posJumpMin = 20#18 for 2ndDec  #20 #-1
negJumpMax = 4#8  for 2ndDec  #4  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth5Lengths = {}
tooth5Lengths_normalized = {}
tooth5Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth5Lengths[resultKey] = {}
    tooth5Lengths_normalized[resultKey] = {}
    tooth5Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth5' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth5'][3] - teethSeperated_resultsDic[resultKey][key]['tooth5'][2]

                    tooth5Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth5Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth5Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth5Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth5Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth5Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth5Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth5Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth5Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
#MULTIPLE Final Tooth 7 -- 24,5

imHeight = 480

posJumpMin = 24 #20 for Dec all  #-1
negJumpMax = 5 #4 for Dec all  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth6Lengths = {}
tooth6Lengths_normalized = {}
tooth6Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth6Lengths[resultKey] = {}
    tooth6Lengths_normalized[resultKey] = {}
    tooth6Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth6' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth6'][3] - teethSeperated_resultsDic[resultKey][key]['tooth6'][2]

                    tooth6Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth6Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth6Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth6Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth6Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth6Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth6Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth6Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth6Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
#MULTIPLE Final Tooth 8 -- 23,3,2

imHeight = 480

posJumpMin = 23 #20  for Dec2nd#20
negJumpMax = 3  #5   for Dec2nd#4
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth7Lengths = {}
tooth7Lengths_normalized = {}
tooth7Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth7Lengths[resultKey] = {}
    tooth7Lengths_normalized[resultKey] = {}
    tooth7Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth7' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth7'][3] - teethSeperated_resultsDic[resultKey][key]['tooth7'][2]

                    tooth7Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth7Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth7Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth7Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth7Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth7Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth7Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth7Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth7Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

In [ ]:
#MULTIPLE Final Tooth 9 -- 24,4,2

imHeight = 480

posJumpMin = 24 #18  for Dec2nd   #22 #-1
negJumpMax = 4  #4   for Dec2nd   #2  #1000
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth8Lengths = {}
tooth8Lengths_normalized = {}
tooth8Lengths_movingAverage = {}


for resultKey in resultsDic:
    processedFrames = 0
    tooth8Lengths[resultKey] = {}
    tooth8Lengths_normalized[resultKey] = {}
    tooth8Lengths_movingAverage[resultKey] = deque()



    
    for key in sorted(teethSeperated_resultsDic[resultKey]):        

        if 'teethLine' in teethSeperated_resultsDic[resultKey][key] and 'yOfMiddle' in teethSeperated_resultsDic[resultKey][key]['teethLine']:

            yMiddleOfTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['yOfMiddle']
            widthTeethLine= teethSeperated_resultsDic[resultKey][key]['teethLine']['width'] 


            if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:


                if 'tooth8' in teethSeperated_resultsDic[resultKey][key]:
                    actualLength = teethSeperated_resultsDic[resultKey][key]['tooth8'][3] - teethSeperated_resultsDic[resultKey][key]['tooth8'][2]

                    tooth8Lengths[resultKey][key] = actualLength


                    movingAbverage = getMovingAverage4(tooth8Lengths_movingAverage[resultKey], actualLength, movingAvgLength,posJumpMin,negJumpMax)


                    tooth8Lengths_normalized[resultKey][key] = movingAbverage


                    
    
    
    
###########PLOTTING ORIGINALS#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()

for resultKey in resultsDic:
    
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth8Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth8Lengths[resultKey][frameNb])

    print(str(len(tempLengths)) + '  original tooth lengths')

    plt.plot(tempFrameNbs, tempLengths, label=resultKey)
    plt.plot(tempFrameNbs, tempLengths,'ro')


ax.legend()
plt.show()


    
    
    
###########PLOTTING SMOOTHED#########################
plt.figure(figsize=(30,10))
ax = plt.axes()
#loc = plticker.MultipleLocator(base=300000.0)
#ax.xaxis.set_major_locator(loc)
ax.grid()


for resultKey in resultsDic:
    tempLengths = []
    tempFrameNbs = []

    for frameNb in sorted(tooth8Lengths[resultKey]):
        tempFrameNbs.append(frameNb)
        tempLengths.append(tooth8Lengths[resultKey][frameNb])



    tempLengths_normalized = []
    tempFrameNbs_normalized = []

    for frameNb in sorted(tooth8Lengths_normalized[resultKey]):
        tempFrameNbs_normalized.append(frameNb)
        tempLengths_normalized.append(tooth8Lengths_normalized[resultKey][frameNb])

    print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, label=resultKey)
    plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

ax.legend() 
plt.show()

#### FOR NOV

In [ ]:
# Final Tooth 0

imHeight = 480

posJumpMin = 15#15#22 #-1
negJumpMax = 3 #3#2  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth0Lengths = {}


tooth0Lengths_normalized = {}


tooth0Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth0' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth0'][3] - teethSeperated_resultsDic[key]['tooth0'][2]
                
                tooth0Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth0Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth0Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth0Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth0Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth1

imHeight = 480

posJumpMin = 15#18#14#14 #-1
negJumpMax = 3#2#1  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth1Lengths = {}


tooth1Lengths_normalized = {}


tooth1Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth1' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth1'][3] - teethSeperated_resultsDic[key]['tooth1'][2]
                
                tooth1Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth1Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth1Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth1Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth1Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')

plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth2

imHeight = 480

posJumpMin = 15#32 #-1
negJumpMax = 3#3  #1000
movingAvgLength = 1#3

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth2Lengths = {}


tooth2Lengths_normalized = {}


tooth2Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth2' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth2'][3] - teethSeperated_resultsDic[key]['tooth2'][2]
                
                tooth2Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth2Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth2Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()
print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth2Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth2Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')


plt.show()

In [ ]:
# Final Tooth3

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 3#4  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth3Lengths = {}


tooth3Lengths_normalized = {}


tooth3Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth3' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth3'][3] - teethSeperated_resultsDic[key]['tooth3'][2]
                
                tooth3Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth3Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth3Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()
print(str(len(tempLengths)) + '  original tooth lengths')
plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth3Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth3Lengths_normalized[frameNb])
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth4

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 3#3  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth4Lengths = {}


tooth4Lengths_normalized = {}


tooth4Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth4' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth4'][3] - teethSeperated_resultsDic[key]['tooth4'][2]
                
                tooth4Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth4Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth4Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth4Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth4Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth5

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 3#3  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth5Lengths = {}


tooth5Lengths_normalized = {}


tooth5Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth5' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth5'][3] - teethSeperated_resultsDic[key]['tooth5'][2]
                
                tooth5Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth5Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth5Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()


print(str(len(tempLengths)) + '  original tooth lengths')
plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth5Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth5Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth6

imHeight = 480

posJumpMin = 15#32 #-1
negJumpMax = 3#3  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth6Lengths = {}


tooth6Lengths_normalized = {}


tooth6Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth6' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth6'][3] - teethSeperated_resultsDic[key]['tooth6'][2]
                
                tooth6Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth6Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth6Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth6Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth6Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth7

imHeight = 480

posJumpMin = 16#32 #-1
negJumpMax = 2#4 #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth7Lengths = {}


tooth7Lengths_normalized = {}


tooth7Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth7' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth7'][3] - teethSeperated_resultsDic[key]['tooth7'][2]
                
                tooth7Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth7Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth7Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth7Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth7Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth8

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 3#3 #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth8Lengths = {}


tooth8Lengths_normalized = {}


tooth8Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth8' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth8'][3] - teethSeperated_resultsDic[key]['tooth8'][2]
                
                tooth8Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth8Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth8Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth8Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth8Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

#### FOR DEC

In [ ]:
# Final Tooth 0

imHeight = 480

posJumpMin = 15#22 #-1
negJumpMax = 3#2  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth0Lengths = {}


tooth0Lengths_normalized = {}


tooth0Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth0' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth0'][3] - teethSeperated_resultsDic[key]['tooth0'][2]
                
                tooth0Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth0Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth0Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth0Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth0Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth1

imHeight = 480

posJumpMin = 15#18#15#14 #-1
negJumpMax = 3#3#3#1  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth1Lengths = {}


tooth1Lengths_normalized = {}


tooth1Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth1' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth1'][3] - teethSeperated_resultsDic[key]['tooth1'][2]
                
                tooth1Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth1Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth1Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth1Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth1Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')

plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth2

imHeight = 480

posJumpMin = 15 #-1
negJumpMax = 3  #1000
movingAvgLength = 1#3

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth2Lengths = {}


tooth2Lengths_normalized = {}


tooth2Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth2' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth2'][3] - teethSeperated_resultsDic[key]['tooth2'][2]
                
                tooth2Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth2Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth2Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()
print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth2Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth2Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')


plt.show()

In [ ]:
# Final Tooth3

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 3#4  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth3Lengths = {}


tooth3Lengths_normalized = {}


tooth3Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth3' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth3'][3] - teethSeperated_resultsDic[key]['tooth3'][2]
                
                tooth3Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth3Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth3Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()
print(str(len(tempLengths)) + '  original tooth lengths')
plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth3Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth3Lengths_normalized[frameNb])
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth4

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 3#3  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth4Lengths = {}


tooth4Lengths_normalized = {}


tooth4Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth4' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth4'][3] - teethSeperated_resultsDic[key]['tooth4'][2]
                
                tooth4Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth4Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth4Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth4Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth4Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth5

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 3#3  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth5Lengths = {}


tooth5Lengths_normalized = {}


tooth5Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth5' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth5'][3] - teethSeperated_resultsDic[key]['tooth5'][2]
                
                tooth5Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth5Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth5Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()


print(str(len(tempLengths)) + '  original tooth lengths')
plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth5Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth5Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth6

imHeight = 480

posJumpMin = 15#32 #-1
negJumpMax = 2#3  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth6Lengths = {}


tooth6Lengths_normalized = {}


tooth6Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth6' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth6'][3] - teethSeperated_resultsDic[key]['tooth6'][2]
                
                tooth6Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth6Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth6Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth6Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth6Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth7

imHeight = 480

posJumpMin = 15#32 #-1
negJumpMax = 3#4 #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth7Lengths = {}


tooth7Lengths_normalized = {}


tooth7Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth7' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth7'][3] - teethSeperated_resultsDic[key]['tooth7'][2]
                
                tooth7Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth7Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth7Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth7Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth7Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth8

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 3#3 #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth8Lengths = {}


tooth8Lengths_normalized = {}


tooth8Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth8' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth8'][3] - teethSeperated_resultsDic[key]['tooth8'][2]
                
                tooth8Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth8Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth8Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth8Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth8Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

#### FOR MAY

In [ ]:
# Final Tooth 0

imHeight = 480

posJumpMin = 15#22 #-1
negJumpMax = 2#2#2  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth0Lengths = {}


tooth0Lengths_normalized = {}


tooth0Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth0' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth0'][3] - teethSeperated_resultsDic[key]['tooth0'][2]
                
                tooth0Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth0Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth0Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth0Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth0Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth1

imHeight = 480

posJumpMin = 15#18#14#14 #-1
negJumpMax = 2#3#2#1  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth1Lengths = {}


tooth1Lengths_normalized = {}


tooth1Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth1' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth1'][3] - teethSeperated_resultsDic[key]['tooth1'][2]
                
                tooth1Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth1Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth1Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth1Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth1Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')

plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth2

imHeight = 480

posJumpMin = 15#32 #-1
negJumpMax = 2#3  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth2Lengths = {}


tooth2Lengths_normalized = {}


tooth2Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth2' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth2'][3] - teethSeperated_resultsDic[key]['tooth2'][2]
                
                tooth2Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth2Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth2Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()
print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth2Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth2Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')


plt.show()

In [ ]:
# Final Tooth3

imHeight = 480

posJumpMin = 15#18#35 #-1
negJumpMax = 3#4#4  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth3Lengths = {}


tooth3Lengths_normalized = {}


tooth3Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth3' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth3'][3] - teethSeperated_resultsDic[key]['tooth3'][2]
                
                tooth3Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth3Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth3Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()
print(str(len(tempLengths)) + '  original tooth lengths')
plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth3Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth3Lengths_normalized[frameNb])
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth4

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 2#3  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth4Lengths = {}


tooth4Lengths_normalized = {}


tooth4Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth4' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth4'][3] - teethSeperated_resultsDic[key]['tooth4'][2]
                
                tooth4Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth4Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth4Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth4Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth4Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth5

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 3#3  #1000
movingAvgLength = 1#2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth5Lengths = {}


tooth5Lengths_normalized = {}


tooth5Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth5' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth5'][3] - teethSeperated_resultsDic[key]['tooth5'][2]
                
                tooth5Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth5Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth5Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()


print(str(len(tempLengths)) + '  original tooth lengths')
plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth5Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth5Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth6

imHeight = 480

posJumpMin = 15#32 #-1
negJumpMax = 2#3  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth6Lengths = {}


tooth6Lengths_normalized = {}


tooth6Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth6' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth6'][3] - teethSeperated_resultsDic[key]['tooth6'][2]
                
                tooth6Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth6Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth6Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth6Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth6Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth7

imHeight = 480

posJumpMin = 15#32 #-1
negJumpMax = 3#4 #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth7Lengths = {}


tooth7Lengths_normalized = {}


tooth7Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth7' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth7'][3] - teethSeperated_resultsDic[key]['tooth7'][2]
                
                tooth7Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth7Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth7Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth7Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth7Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth8

imHeight = 480

posJumpMin = 15#35 #-1
negJumpMax = 2#3 #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth8Lengths = {}


tooth8Lengths_normalized = {}


tooth8Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth8' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth8'][3] - teethSeperated_resultsDic[key]['tooth8'][2]
                
                tooth8Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth8Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth8Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=3000000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth8Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth8Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

### <font color='red'>FINAL RESULTS FOR first video captured every 100 frames

In [ ]:
# Final Tooth 0

imHeight = 480

posJumpMin = 22 #-1
negJumpMax = 2  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth0Lengths = {}


tooth0Lengths_normalized = {}


tooth0Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth0' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth0'][3] - teethSeperated_resultsDic[key]['tooth0'][2]
                
                tooth0Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth0Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth0Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth0Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth0Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth0Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth0Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth1

imHeight = 480

posJumpMin = 14 #-1
negJumpMax = 1  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth1Lengths = {}


tooth1Lengths_normalized = {}


tooth1Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth1' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth1'][3] - teethSeperated_resultsDic[key]['tooth1'][2]
                
                tooth1Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth1Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth1Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth1Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth1Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth1Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth1Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')

plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth2

imHeight = 480

posJumpMin = 32 #-1
negJumpMax = 3  #1000
movingAvgLength = 3

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth2Lengths = {}


tooth2Lengths_normalized = {}


tooth2Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth2' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth2'][3] - teethSeperated_resultsDic[key]['tooth2'][2]
                
                tooth2Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth2Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth2Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()
print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth2Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth2Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth2Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth2Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')


plt.show()

In [ ]:
# Final Tooth3

imHeight = 480

posJumpMin = 35 #-1
negJumpMax = 4  #1000
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth3Lengths = {}


tooth3Lengths_normalized = {}


tooth3Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth3' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth3'][3] - teethSeperated_resultsDic[key]['tooth3'][2]
                
                tooth3Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth3Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth3Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()
print(str(len(tempLengths)) + '  original tooth lengths')
plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth3Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth3Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth3Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth3Lengths_normalized[frameNb])
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth4

imHeight = 480

posJumpMin = 35 #-1
negJumpMax = 3  #1000
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth4Lengths = {}


tooth4Lengths_normalized = {}


tooth4Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth4' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth4'][3] - teethSeperated_resultsDic[key]['tooth4'][2]
                
                tooth4Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth4Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth4Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

print(str(len(tempLengths)) + '  original tooth lengths')

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth4Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth4Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth4Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth4Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth5

imHeight = 480

posJumpMin = 35 #-1
negJumpMax = 3  #1000
movingAvgLength = 2

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth5Lengths = {}


tooth5Lengths_normalized = {}


tooth5Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth5' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth5'][3] - teethSeperated_resultsDic[key]['tooth5'][2]
                
                tooth5Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth5Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth5Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()


print(str(len(tempLengths)) + '  original tooth lengths')
plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth5Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth5Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth5Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth5Lengths_normalized[frameNb])
    
print(str(len(tempLengths_normalized)) + '  normalized tooth lengths')
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth6

imHeight = 480

posJumpMin = 32 #-1
negJumpMax = 3  #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth6Lengths = {}


tooth6Lengths_normalized = {}


tooth6Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth6' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth6'][3] - teethSeperated_resultsDic[key]['tooth6'][2]
                
                tooth6Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth6Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth6Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth6Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth6Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth6Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth6Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth7

imHeight = 480

posJumpMin = 32 #-1
negJumpMax = 4 #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth7Lengths = {}


tooth7Lengths_normalized = {}


tooth7Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth7' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth7'][3] - teethSeperated_resultsDic[key]['tooth7'][2]
                
                tooth7Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth7Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth7Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth7Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth7Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth7Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth7Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

In [ ]:
# Final Tooth8

imHeight = 480

posJumpMin = 35 #-1
negJumpMax = 3 #1000
movingAvgLength = 1

ymiddleLimitMax = 210 #120 #400 #115
ymiddleLimitMin = 180#100 #60 #113

widthToFilterFor = 500
widthTolerance = 100



tooth8Lengths = {}


tooth8Lengths_normalized = {}


tooth8Lengths_movingAverage = deque()



processedFrames = 0
for key in sorted(teethSeperated_resultsDic):        

    if 'teethLine' in teethSeperated_resultsDic[key] and 'yOfMiddle' in teethSeperated_resultsDic[key]['teethLine']:
        
        yMiddleOfTeethLine= teethSeperated_resultsDic[key]['teethLine']['yOfMiddle']
        widthTeethLine= teethSeperated_resultsDic[key]['teethLine']['width'] 


        if yMiddleOfTeethLine < ymiddleLimitMax and yMiddleOfTeethLine > ymiddleLimitMin:

        
            if 'tooth8' in teethSeperated_resultsDic[key]:
                actualLength = teethSeperated_resultsDic[key]['tooth8'][3] - teethSeperated_resultsDic[key]['tooth8'][2]
                
                tooth8Lengths[key] = actualLength

            
                movingAbverage = getMovingAverage4(tooth8Lengths_movingAverage, actualLength, movingAvgLength,posJumpMin,negJumpMax)

                
                tooth8Lengths_normalized[key] = movingAbverage

                
                
                
tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

plt.plot(tempFrameNbs, tempLengths)
plt.plot(tempFrameNbs, tempLengths,'ro')
plt.show()



tempLengths = []
tempFrameNbs = []

for frameNb in sorted(tooth8Lengths):
    tempFrameNbs.append(frameNb)
    tempLengths.append(tooth8Lengths[frameNb])
    
plt.figure(figsize=(30,10))
ax = plt.axes()
loc = plticker.MultipleLocator(base=300000.0)
ax.xaxis.set_major_locator(loc)
ax.grid()

#plt.plot(tempFrameNbs, tempLengths)


tempLengths_normalized = []
tempFrameNbs_normalized = []

for frameNb in sorted(tooth8Lengths_normalized):
    tempFrameNbs_normalized.append(frameNb)
    tempLengths_normalized.append(tooth8Lengths_normalized[frameNb])
    
plt.plot(tempFrameNbs_normalized, tempLengths_normalized)
plt.plot(tempFrameNbs_normalized, tempLengths_normalized, 'ro')

plt.show()

### <font color='green'>  Results 2nd round for showing to Mahdi start

In [ ]:
# every 100 frames. all locations

In [ ]:
Individual Tooth Lengths * (yOfCentreOfTeethLine / widthOfTeethLine)

In [ ]:
# frame to hour convertor
def convH(x):
    x = x * 10000000 / 30 / 60 / 60
    return x
    

#frames = [0.03, 0.06, 0.9,0.12, 0.30,0.31, 0.39,0.40, 0.55,0.57, 0.65,0.66, 1.04, 1.05, 1.08, 1.11]
#for x in frames:

print(convH(5939170/10000000))

In [ ]:
# every 100 frames, only frames that have teethline, with center btw y 115 and 113, Xaxis tick size: 2.7Hours

In [ ]:
# JUSTPOINTS every 100 frames, only frames that have teethline, with center btw y 115 and 113, Xaxis tick size: 2.7Hours

In [ ]:
# USING RECOVERED TEETHLINE TOO, every 100 frames, only frames that have teethline, with center btw y 115 and 113, Xaxis tick size: 2.7Hours

In [ ]:
# every 100 frames,frames that have teethline, with center btw y 115 and 113 (blue) and 129-131 (orange)

In [ ]:
# every 100 frames,frames that have teethline, with center btw y 115 and 113 (blue), and widths btween 400-500

In [ ]:
# JUSTPOINTS every 100 frames,frames that have teethline, with center btw y 115 and 113 (blue), and widths btween 400-500

In [ ]:
# every 100 frames, only frames that have teethline, with center btw y 110 and 120, Xaxis tick size: 2.7Hours

In [ ]:
# every 100 frames, only frames that have teethline, with center btw y 100 and 120, Xaxis tick size: 2.7Hours

In [ ]:
# (Lengths / centreYOFTeehtline * widhtOfTeethLine) every 100 frames, only frames that have teethline, with center btw y 100 and 120, Xaxis tick size: 2.7Hours

In [ ]:
# (normalized) every 100 frames, only frames that have teethline, with center btw y 100 and 120, Xaxis tick size: 2.7Hours

In [ ]:
# Number of frames at each y location of the centre of teethLine

ymiddleLimitMax = 107
ymiddleLimitMin = 105
processes: 56 frames and Found:


ymiddleLimitMax = 109
ymiddleLimitMin = 107
processes: 94 frames and Found:


ymiddleLimitMax = 111
ymiddleLimitMin = 109
processes: 81 frames and Found:


ymiddleLimitMax = 113
ymiddleLimitMin = 111
processes: 97 frames and Found:


ymiddleLimitMax = 114
ymiddleLimitMin = 112
processes: 128 frames and Found: ***


ymiddleLimitMax = 115
ymiddleLimitMin = 113
processes: 134 frames and Found: ****************************


ymiddleLimitMax = 116
ymiddleLimitMin = 114
processes: 110 frames and Found:


ymiddleLimitMax = 117
ymiddleLimitMin = 115
processes: 80 frames and Found:


ymiddleLimitMax = 118
ymiddleLimitMin = 116
processes: 60 frames and Found:


ymiddleLimitMax = 120
ymiddleLimitMin = 118
processes: 46 frames and Found:





ymiddleLimitMax = 130
ymiddleLimitMin = 128
processes: 109 frames and Found:


ymiddleLimitMax = 131
ymiddleLimitMin = 129
processes: 132 frames and Found:  ********


ymiddleLimitMax = 132
ymiddleLimitMin = 130
processes: 127 frames and Found: **


ymiddleLimitMax = 133
ymiddleLimitMin = 131
processes: 96 frames and Found:


ymiddleLimitMax = 134
ymiddleLimitMin = 132
processes: 84 frames and Found:  


ymiddleLimitMax = 140
ymiddleLimitMin = 138
processes: 116 frames and Found:


ymiddleLimitMax = 141
ymiddleLimitMin = 139
processes: 127 frames and Found: **


ymiddleLimitMax = 142
ymiddleLimitMin = 140
processes: 122 frames and Found:


ymiddleLimitMax = 144
ymiddleLimitMin = 142
processes: 120 frames and Found:

#### Correcting with fitted Line (BiggestAvgToothLengths/ avgToothLengths) Vs. (YofCenture / width  OfTeethLine )

In [ ]:
Average toothLengths Vs. (YofCenture / width  OfTeethLine )

In [ ]:
INDIVIDUAL toothLengths Vs. (YofCenture / width  OfTeethLine )

In [ ]:
Corrected average toothLengths Vs. (YofCenture / width  OfTeethLine )

In [ ]:
Corrected Tooth4Lengths  Vs. (YofCenture / width  OfTeethLine )

In [ ]:
(BiggestAvgToothLengths/ avgToothLengths) Vs. (YofCenture / width  OfTeethLine )

In [ ]:
(BiggestTooth4Lengths/ currentTooth4Lengths) Vs. (YofCenture / width  OfTeethLine )

In [ ]:
(Corrected BiggestAvgToothLengths/ avgToothLengths) Vs. (YofCenture / width  OfTeethLine )

In [ ]:
( Corrected BiggestTooth4Lengths/ currentTooth4Lengths) Vs. (YofCenture / width  OfTeethLine )

In [ ]:
Original Tooth Lengths Vs. Frame Number For the selectedSequences

In [ ]:
Corrected using tooth4Correction graph, Tooth Lengths Vs. Frame Number For the selectedSequences

### <font color='green'> Results 2nd round for showing to Mahdi end

### MISC Results

In [ ]:
#ymiddleLimit = 400   every 10000

In [ ]:
#none messy

In [ ]:
# messy 

##### Results: every 18000

In [ ]:
# normalizedLength > 0 normalizedLength < 200

In [ ]:
#== normalizedLength if normalizedLength > 0 and normalizedLength < 200 else actualLength

In [ ]:
#== normalizedLength if normalizedLength > 0 else Nothing

In [ ]:
#== normalizedLength if normalizedLength > 0 else actualLength

#### Results: every 10000

In [ ]:
# No normalization

In [ ]:
# With Forshortenning normalization selectedSequence normalization Deg 5

In [ ]:
# With seq1 7Deg normalization

In [ ]:
# With seq1 16Deg normalization

In [ ]:
# With average 16Deg normalization

In [ ]:
# With average 7Deg normalization

#### Results for normalization curves using differenct sequences

### Fitting a degree 7 curve

In [ ]:
### Results: selectedSequence1

In [ ]:
### Results: selectedSequence2

In [ ]:
### Results: selectedSequence3

In [ ]:
### Results: selectedSequence4

In [ ]:
### Results: selectedSequence5

In [ ]:
### Results: selectedSequence6

In [ ]:
### Results: average

### Fitting a degree 16 curve

In [ ]:
### Results: selectedSequence1

In [ ]:
### Results: selectedSequence2

In [ ]:
### Results: selectedSequence3

In [ ]:
### Results: selectedSequence4

In [ ]:
### Results: selectedSequence5

In [ ]:
### results: selectSequence6

In [ ]:
### Results: average

### Fitting a degree 3 curve

In [ ]:
### Results: selectedSequence1

In [ ]:
### Results: selectedSequence2

In [ ]:
### Results: selectedSequence3

In [ ]:
### Results: selectedSequence4

In [ ]:
### Results: selectedSequence5

In [ ]:
### Results: selectedSequence6

In [ ]:
### Results: selectedSequence6

### Fitting a degree 5 curve 

In [ ]:
### Results: selectedSequence1

In [ ]:
### Results: selectedSequence2

In [ ]:
### Results: selectedSequence3

In [ ]:
### Results: selectedSequence4